In [1]:
import gym
# Tensorflow Library
import tensorflow as tf
import numpy as np
from collections import deque
from time import time, strftime, localtime
import os
tf.get_logger().setLevel('ERROR')
import matplotlib.pyplot as plt

In [3]:
!mkdir test

In [6]:
import gym, random, tempfile
import numpy as np
from gym import wrappers
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.max_e = 1
        self.min_e = 0.0001
        self.epsilon_decay = 0.001
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    tdir = tempfile.mkdtemp()
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.min_e:
            agent.epsilon *= agent.epsilon_decay
            agent.epsilon = agent.min_e + (agent.max_e - agent.min_e)*np.exp(-agent.epsilon_decay*e)

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()


#agent.model.save_weights('weights_DQN.h5')
#agent.model.load_weights('weights_DQN.h5')

episode:  0  score:  -171.66  avg_score:  -171.66  frames:  105  epsilon:  1.00
episode:  1  score:  -209.66  avg_score:  -190.66  frames:  87  epsilon:  1.00
episode:  2  score:  -139.12  avg_score:  -173.48  frames:  115  epsilon:  1.00
episode:  3  score:  -8.48  avg_score:  -132.23  frames:  119  epsilon:  1.00
episode:  4  score:  -238.79  avg_score:  -153.54  frames:  104  epsilon:  1.00
episode:  5  score:  -110.54  avg_score:  -146.38  frames:  84  epsilon:  1.00
episode:  6  score:  -64.30  avg_score:  -134.65  frames:  126  epsilon:  0.99
episode:  7  score:  -182.64  avg_score:  -140.65  frames:  117  epsilon:  0.99
episode:  8  score:  -329.27  avg_score:  -161.61  frames:  98  epsilon:  0.99
episode:  9  score:  -88.07  avg_score:  -154.25  frames:  79  epsilon:  0.99
episode:  10  score:  -93.18  avg_score:  -148.70  frames:  69  epsilon:  0.99
episode:  11  score:  -163.08  avg_score:  -149.90  frames:  77  epsilon:  0.99
episode:  12  score:  -125.12  avg_score:  -147.9

episode:  103  score:  -125.48  avg_score:  -167.72  frames:  94  epsilon:  0.90
episode:  104  score:  -83.38  avg_score:  -166.16  frames:  68  epsilon:  0.90
episode:  105  score:  -297.36  avg_score:  -168.03  frames:  119  epsilon:  0.90
episode:  106  score:  -92.27  avg_score:  -168.31  frames:  108  epsilon:  0.90
episode:  107  score:  -88.50  avg_score:  -167.37  frames:  81  epsilon:  0.90
episode:  108  score:  -132.13  avg_score:  -165.40  frames:  95  epsilon:  0.90
episode:  109  score:  -207.72  avg_score:  -166.60  frames:  70  epsilon:  0.90
episode:  110  score:  -478.10  avg_score:  -170.45  frames:  99  epsilon:  0.90
episode:  111  score:  -137.82  avg_score:  -170.19  frames:  110  epsilon:  0.89
episode:  112  score:  -182.86  avg_score:  -170.77  frames:  119  epsilon:  0.89
episode:  113  score:  -217.04  avg_score:  -171.29  frames:  83  epsilon:  0.89
episode:  114  score:  -371.79  avg_score:  -173.26  frames:  105  epsilon:  0.89
episode:  115  score:  -22

episode:  204  score:  -36.46  avg_score:  -150.57  frames:  127  epsilon:  0.82
episode:  205  score:  -79.34  avg_score:  -148.39  frames:  79  epsilon:  0.81
episode:  206  score:  -83.49  avg_score:  -148.31  frames:  141  epsilon:  0.81
episode:  207  score:  -133.55  avg_score:  -148.76  frames:  113  epsilon:  0.81
episode:  208  score:  -63.33  avg_score:  -148.07  frames:  94  epsilon:  0.81
episode:  209  score:  -88.52  avg_score:  -146.88  frames:  87  epsilon:  0.81
episode:  210  score:  -82.10  avg_score:  -142.92  frames:  126  epsilon:  0.81
episode:  211  score:  -85.97  avg_score:  -142.40  frames:  63  epsilon:  0.81
episode:  212  score:  -102.78  avg_score:  -141.60  frames:  83  epsilon:  0.81
episode:  213  score:  -26.30  avg_score:  -139.69  frames:  93  epsilon:  0.81
episode:  214  score:  -143.87  avg_score:  -137.41  frames:  70  epsilon:  0.81
episode:  215  score:  -75.28  avg_score:  -135.95  frames:  96  epsilon:  0.81
episode:  216  score:  -105.72  a

episode:  306  score:  -151.56  avg_score:  -90.19  frames:  129  epsilon:  0.74
episode:  307  score:  -61.59  avg_score:  -89.47  frames:  108  epsilon:  0.74
episode:  308  score:  -238.05  avg_score:  -91.21  frames:  99  epsilon:  0.73
episode:  309  score:  -105.24  avg_score:  -91.38  frames:  76  epsilon:  0.73
episode:  310  score:  -99.72  avg_score:  -91.56  frames:  127  epsilon:  0.73
episode:  311  score:  -75.10  avg_score:  -91.45  frames:  78  epsilon:  0.73
episode:  312  score:  -65.71  avg_score:  -91.08  frames:  125  epsilon:  0.73
episode:  313  score:  -50.35  avg_score:  -91.32  frames:  130  epsilon:  0.73
episode:  314  score:  -116.97  avg_score:  -91.05  frames:  107  epsilon:  0.73
episode:  315  score:  -105.54  avg_score:  -91.35  frames:  111  epsilon:  0.73
episode:  316  score:  -57.50  avg_score:  -90.87  frames:  158  epsilon:  0.73
episode:  317  score:  -38.06  avg_score:  -90.56  frames:  157  epsilon:  0.73
episode:  318  score:  -150.55  avg_sc

episode:  409  score:  45.84  avg_score:  -76.24  frames:  174  epsilon:  0.66
episode:  410  score:  -106.75  avg_score:  -76.31  frames:  89  epsilon:  0.66
episode:  411  score:  -71.47  avg_score:  -76.27  frames:  126  epsilon:  0.66
episode:  412  score:  -42.48  avg_score:  -76.04  frames:  184  epsilon:  0.66
episode:  413  score:  -62.27  avg_score:  -76.16  frames:  61  epsilon:  0.66
episode:  414  score:  -112.95  avg_score:  -76.12  frames:  88  epsilon:  0.66
episode:  415  score:  -38.95  avg_score:  -75.45  frames:  203  epsilon:  0.66
episode:  416  score:  -54.51  avg_score:  -75.42  frames:  167  epsilon:  0.66
episode:  417  score:  14.02  avg_score:  -74.90  frames:  86  epsilon:  0.66
episode:  418  score:  -188.36  avg_score:  -75.28  frames:  193  epsilon:  0.66
episode:  419  score:  -17.48  avg_score:  -75.40  frames:  148  epsilon:  0.66
episode:  420  score:  -69.41  avg_score:  -74.63  frames:  132  epsilon:  0.66
episode:  421  score:  -49.14  avg_score:  

...

episode:  2774  score:  223.05  avg_score:  253.74  frames:  264  epsilon:  0.06
episode:  2775  score:  258.13  avg_score:  253.74  frames:  246  epsilon:  0.06
episode:  2776  score:  -96.50  avg_score:  250.81  frames:  57  epsilon:  0.06
episode:  2777  score:  269.26  avg_score:  251.52  frames:  272  epsilon:  0.06
episode:  2778  score:  294.64  avg_score:  251.72  frames:  319  epsilon:  0.06
episode:  2779  score:  245.99  avg_score:  251.52  frames:  261  epsilon:  0.06
episode:  2780  score:  244.17  avg_score:  251.38  frames:  211  epsilon:  0.06
episode:  2781  score:  226.91  avg_score:  251.21  frames:  469  epsilon:  0.06
episode:  2782  score:  268.16  avg_score:  251.57  frames:  176  epsilon:  0.06
episode:  2783  score:  275.64  avg_score:  252.06  frames:  269  epsilon:  0.06
episode:  2784  score:  239.48  avg_score:  251.40  frames:  209  epsilon:  0.06
episode:  2785  score:  286.70  avg_score:  251.94  frames:  169  epsilon:  0.06
episode:  2786  score:  302.7

episode:  2876  score:  293.88  avg_score:  235.06  frames:  234  epsilon:  0.06
episode:  2877  score:  292.27  avg_score:  235.29  frames:  259  epsilon:  0.06
episode:  2878  score:  267.08  avg_score:  235.02  frames:  205  epsilon:  0.06
episode:  2879  score:  284.89  avg_score:  235.41  frames:  291  epsilon:  0.06
episode:  2880  score:  279.96  avg_score:  235.76  frames:  238  epsilon:  0.06
episode:  2881  score:  279.15  avg_score:  236.29  frames:  352  epsilon:  0.06
episode:  2882  score:  284.44  avg_score:  236.45  frames:  246  epsilon:  0.06
episode:  2883  score:  273.74  avg_score:  236.43  frames:  232  epsilon:  0.06
episode:  2884  score:  248.80  avg_score:  236.52  frames:  195  epsilon:  0.06
episode:  2885  score:  222.25  avg_score:  235.88  frames:  189  epsilon:  0.06
episode:  2886  score:  281.06  avg_score:  235.66  frames:  370  epsilon:  0.06
episode:  2887  score:  288.78  avg_score:  238.75  frames:  226  epsilon:  0.06
episode:  2888  score:  244.

episode:  2978  score:  -4.46  avg_score:  212.10  frames:  112  epsilon:  0.05
episode:  2979  score:  257.19  avg_score:  211.82  frames:  257  epsilon:  0.05
episode:  2980  score:  241.48  avg_score:  211.44  frames:  262  epsilon:  0.05
episode:  2981  score:  306.13  avg_score:  211.71  frames:  233  epsilon:  0.05
episode:  2982  score:  63.82  avg_score:  209.50  frames:  183  epsilon:  0.05
episode:  2983  score:  -186.61  avg_score:  204.90  frames:  56  epsilon:  0.05
episode:  2984  score:  242.10  avg_score:  204.83  frames:  306  epsilon:  0.05
episode:  2985  score:  311.77  avg_score:  205.73  frames:  261  epsilon:  0.05
episode:  2986  score:  261.77  avg_score:  205.53  frames:  253  epsilon:  0.05
episode:  2987  score:  20.91  avg_score:  202.86  frames:  172  epsilon:  0.05
episode:  2988  score:  46.30  avg_score:  200.87  frames:  189  epsilon:  0.05
episode:  2989  score:  160.61  avg_score:  199.66  frames:  264  epsilon:  0.05
episode:  2990  score:  31.86  a

Exception ignored in: <function ScopedTFFunction.__del__ at 0x00000280D2469B80>
Traceback (most recent call last):
  File "C:\Apps\Anaconda\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 96, in __del__
    self.deleter(self.func)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
import pickle

with open('log/rewards_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                         +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5', 'wb') as fp:
    pickle.dump(total_rewards, fp)

In [7]:



agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                         +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

In [ ]:
string="""
episode:  0  score:  -171.66  avg_score:  -171.66  frames:  105  epsilon:  1.00
episode:  1  score:  -209.66  avg_score:  -190.66  frames:  87  epsilon:  1.00
episode:  2  score:  -139.12  avg_score:  -173.48  frames:  115  epsilon:  1.00
episode:  3  score:  -8.48  avg_score:  -132.23  frames:  119  epsilon:  1.00
episode:  4  score:  -238.79  avg_score:  -153.54  frames:  104  epsilon:  1.00
episode:  5  score:  -110.54  avg_score:  -146.38  frames:  84  epsilon:  1.00
episode:  6  score:  -64.30  avg_score:  -134.65  frames:  126  epsilon:  0.99
episode:  7  score:  -182.64  avg_score:  -140.65  frames:  117  epsilon:  0.99
episode:  8  score:  -329.27  avg_score:  -161.61  frames:  98  epsilon:  0.99
episode:  9  score:  -88.07  avg_score:  -154.25  frames:  79  epsilon:  0.99
episode:  10  score:  -93.18  avg_score:  -148.70  frames:  69  epsilon:  0.99
episode:  11  score:  -163.08  avg_score:  -149.90  frames:  77  epsilon:  0.99
episode:  12  score:  -125.12  avg_score:  -147.99  frames:  85  epsilon:  0.99
episode:  13  score:  -164.90  avg_score:  -149.20  frames:  80  epsilon:  0.99
episode:  14  score:  -175.17  avg_score:  -150.93  frames:  102  epsilon:  0.99
episode:  15  score:  -84.80  avg_score:  -146.80  frames:  60  epsilon:  0.99
episode:  16  score:  -423.26  avg_score:  -163.06  frames:  140  epsilon:  0.98
episode:  17  score:  -127.01  avg_score:  -161.06  frames:  90  epsilon:  0.98
episode:  18  score:  -233.46  avg_score:  -164.87  frames:  107  epsilon:  0.98
episode:  19  score:  -154.08  avg_score:  -164.33  frames:  70  epsilon:  0.98
episode:  20  score:  -203.16  avg_score:  -166.18  frames:  81  epsilon:  0.98
episode:  21  score:  -115.82  avg_score:  -163.89  frames:  92  epsilon:  0.98
episode:  22  score:  -23.03  avg_score:  -157.76  frames:  111  epsilon:  0.98
episode:  23  score:  -486.80  avg_score:  -171.47  frames:  126  epsilon:  0.98
episode:  24  score:  -87.75  avg_score:  -168.13  frames:  71  epsilon:  0.98
episode:  25  score:  -96.98  avg_score:  -165.39  frames:  112  epsilon:  0.98
episode:  26  score:  -7.23  avg_score:  -159.53  frames:  90  epsilon:  0.97
episode:  27  score:  -124.30  avg_score:  -158.27  frames:  78  epsilon:  0.97
episode:  28  score:  -345.14  avg_score:  -164.72  frames:  81  epsilon:  0.97
episode:  29  score:  -182.77  avg_score:  -165.32  frames:  105  epsilon:  0.97
episode:  30  score:  -156.99  avg_score:  -165.05  frames:  75  epsilon:  0.97
episode:  31  score:  -148.18  avg_score:  -164.52  frames:  80  epsilon:  0.97
episode:  32  score:  -101.39  avg_score:  -162.61  frames:  91  epsilon:  0.97
episode:  33  score:  -83.36  avg_score:  -160.28  frames:  113  epsilon:  0.97
episode:  34  score:  -62.39  avg_score:  -157.48  frames:  91  epsilon:  0.97
episode:  35  score:  -222.16  avg_score:  -159.28  frames:  87  epsilon:  0.97
episode:  36  score:  -98.01  avg_score:  -157.62  frames:  72  epsilon:  0.96
episode:  37  score:  -123.51  avg_score:  -156.73  frames:  111  epsilon:  0.96
episode:  38  score:  -81.74  avg_score:  -154.80  frames:  105  epsilon:  0.96
episode:  39  score:  -69.10  avg_score:  -152.66  frames:  71  epsilon:  0.96
episode:  40  score:  -476.05  avg_score:  -160.55  frames:  107  epsilon:  0.96
episode:  41  score:  -258.38  avg_score:  -162.88  frames:  83  epsilon:  0.96
episode:  42  score:  -110.97  avg_score:  -161.67  frames:  70  epsilon:  0.96
episode:  43  score:  10.49  avg_score:  -157.76  frames:  77  epsilon:  0.96
episode:  44  score:  -47.46  avg_score:  -155.31  frames:  62  epsilon:  0.96
episode:  45  score:  -134.25  avg_score:  -154.85  frames:  103  epsilon:  0.96
episode:  46  score:  -132.58  avg_score:  -154.37  frames:  103  epsilon:  0.96
episode:  47  score:  -153.53  avg_score:  -154.36  frames:  88  epsilon:  0.95
episode:  48  score:  -227.33  avg_score:  -155.85  frames:  98  epsilon:  0.95
episode:  49  score:  -94.10  avg_score:  -154.61  frames:  87  epsilon:  0.95
episode:  50  score:  -128.76  avg_score:  -154.10  frames:  96  epsilon:  0.95
episode:  51  score:  -391.69  avg_score:  -158.67  frames:  83  epsilon:  0.95
episode:  52  score:  -347.60  avg_score:  -162.24  frames:  118  epsilon:  0.95
episode:  53  score:  -87.05  avg_score:  -160.85  frames:  83  epsilon:  0.95
episode:  54  score:  -130.31  avg_score:  -160.29  frames:  65  epsilon:  0.95
episode:  55  score:  -296.65  avg_score:  -162.73  frames:  120  epsilon:  0.95
episode:  56  score:  -267.75  avg_score:  -164.57  frames:  109  epsilon:  0.95
episode:  57  score:  -102.62  avg_score:  -163.50  frames:  79  epsilon:  0.94
episode:  58  score:  -331.08  avg_score:  -166.34  frames:  87  epsilon:  0.94
episode:  59  score:  -134.62  avg_score:  -165.81  frames:  66  epsilon:  0.94
episode:  60  score:  -128.09  avg_score:  -165.19  frames:  65  epsilon:  0.94
episode:  61  score:  -299.85  avg_score:  -167.37  frames:  118  epsilon:  0.94
episode:  62  score:  -132.96  avg_score:  -166.82  frames:  92  epsilon:  0.94
episode:  63  score:  -269.49  avg_score:  -168.42  frames:  112  epsilon:  0.94
episode:  64  score:  -206.02  avg_score:  -169.00  frames:  66  epsilon:  0.94
episode:  65  score:  -135.67  avg_score:  -168.50  frames:  69  epsilon:  0.94
episode:  66  score:  -142.92  avg_score:  -168.11  frames:  70  epsilon:  0.94
episode:  67  score:  -2.84  avg_score:  -165.68  frames:  82  epsilon:  0.94
episode:  68  score:  -123.67  avg_score:  -165.08  frames:  99  epsilon:  0.93
episode:  69  score:  -65.55  avg_score:  -163.65  frames:  124  epsilon:  0.93
episode:  70  score:  -130.97  avg_score:  -163.19  frames:  83  epsilon:  0.93
episode:  71  score:  -324.43  avg_score:  -165.43  frames:  117  epsilon:  0.93
episode:  72  score:  -137.54  avg_score:  -165.05  frames:  105  epsilon:  0.93
episode:  73  score:  -317.68  avg_score:  -167.11  frames:  111  epsilon:  0.93
episode:  74  score:  -131.77  avg_score:  -166.64  frames:  117  epsilon:  0.93
episode:  75  score:  -148.44  avg_score:  -166.40  frames:  114  epsilon:  0.93
episode:  76  score:  -93.32  avg_score:  -165.45  frames:  83  epsilon:  0.93
episode:  77  score:  -142.11  avg_score:  -165.15  frames:  84  epsilon:  0.93
episode:  78  score:  -97.18  avg_score:  -164.29  frames:  91  epsilon:  0.92
episode:  79  score:  -61.39  avg_score:  -163.01  frames:  72  epsilon:  0.92
episode:  80  score:  -331.18  avg_score:  -165.08  frames:  121  epsilon:  0.92
episode:  81  score:  -47.98  avg_score:  -163.66  frames:  74  epsilon:  0.92
episode:  82  score:  -338.30  avg_score:  -165.76  frames:  140  epsilon:  0.92
episode:  83  score:  -390.98  avg_score:  -168.44  frames:  105  epsilon:  0.92
episode:  84  score:  -113.65  avg_score:  -167.80  frames:  81  epsilon:  0.92
episode:  85  score:  -268.62  avg_score:  -168.97  frames:  97  epsilon:  0.92
episode:  86  score:  -150.34  avg_score:  -168.75  frames:  117  epsilon:  0.92
episode:  87  score:  -82.98  avg_score:  -167.78  frames:  77  epsilon:  0.92
episode:  88  score:  -251.18  avg_score:  -168.72  frames:  90  epsilon:  0.92
episode:  89  score:  30.15  avg_score:  -166.51  frames:  98  epsilon:  0.91
episode:  90  score:  -118.51  avg_score:  -165.98  frames:  60  epsilon:  0.91
episode:  91  score:  -353.02  avg_score:  -168.01  frames:  105  epsilon:  0.91
episode:  92  score:  -371.96  avg_score:  -170.21  frames:  92  epsilon:  0.91
episode:  93  score:  -153.01  avg_score:  -170.02  frames:  74  epsilon:  0.91
episode:  94  score:  -137.49  avg_score:  -169.68  frames:  84  epsilon:  0.91
episode:  95  score:  -151.25  avg_score:  -169.49  frames:  90  epsilon:  0.91
episode:  96  score:  -82.49  avg_score:  -168.59  frames:  117  epsilon:  0.91
episode:  97  score:  -75.71  avg_score:  -167.64  frames:  72  epsilon:  0.91
episode:  98  score:  -126.55  avg_score:  -167.23  frames:  109  epsilon:  0.91
episode:  99  score:  -148.32  avg_score:  -167.04  frames:  101  epsilon:  0.91
episode:  100  score:  -248.03  avg_score:  -167.80  frames:  124  epsilon:  0.90
episode:  101  score:  -129.50  avg_score:  -167.00  frames:  80  epsilon:  0.90
episode:  102  score:  -93.85  avg_score:  -166.55  frames:  86  epsilon:  0.90
episode:  103  score:  -125.48  avg_score:  -167.72  frames:  94  epsilon:  0.90
episode:  104  score:  -83.38  avg_score:  -166.16  frames:  68  epsilon:  0.90
episode:  105  score:  -297.36  avg_score:  -168.03  frames:  119  epsilon:  0.90
episode:  106  score:  -92.27  avg_score:  -168.31  frames:  108  epsilon:  0.90
episode:  107  score:  -88.50  avg_score:  -167.37  frames:  81  epsilon:  0.90
episode:  108  score:  -132.13  avg_score:  -165.40  frames:  95  epsilon:  0.90
episode:  109  score:  -207.72  avg_score:  -166.60  frames:  70  epsilon:  0.90
episode:  110  score:  -478.10  avg_score:  -170.45  frames:  99  epsilon:  0.90
episode:  111  score:  -137.82  avg_score:  -170.19  frames:  110  epsilon:  0.89
episode:  112  score:  -182.86  avg_score:  -170.77  frames:  119  epsilon:  0.89
episode:  113  score:  -217.04  avg_score:  -171.29  frames:  83  epsilon:  0.89
episode:  114  score:  -371.79  avg_score:  -173.26  frames:  105  epsilon:  0.89
episode:  115  score:  -221.06  avg_score:  -174.62  frames:  118  epsilon:  0.89
episode:  116  score:  -283.56  avg_score:  -173.22  frames:  144  epsilon:  0.89
episode:  117  score:  4.58  avg_score:  -171.91  frames:  141  epsilon:  0.89
episode:  118  score:  -116.74  avg_score:  -170.74  frames:  59  epsilon:  0.89
episode:  119  score:  -22.38  avg_score:  -169.42  frames:  157  epsilon:  0.89
episode:  120  score:  -118.07  avg_score:  -168.57  frames:  70  epsilon:  0.89
episode:  121  score:  -545.14  avg_score:  -172.87  frames:  146  epsilon:  0.89
episode:  122  score:  -103.89  avg_score:  -173.67  frames:  128  epsilon:  0.89
episode:  123  score:  -180.72  avg_score:  -170.61  frames:  80  epsilon:  0.88
episode:  124  score:  -25.39  avg_score:  -169.99  frames:  150  epsilon:  0.88
episode:  125  score:  -86.99  avg_score:  -169.89  frames:  131  epsilon:  0.88
episode:  126  score:  -457.28  avg_score:  -174.39  frames:  93  epsilon:  0.88
episode:  127  score:  -165.75  avg_score:  -174.81  frames:  116  epsilon:  0.88
episode:  128  score:  -15.78  avg_score:  -171.51  frames:  81  epsilon:  0.88
episode:  129  score:  -58.30  avg_score:  -170.27  frames:  79  epsilon:  0.88
episode:  130  score:  -88.39  avg_score:  -169.58  frames:  76  epsilon:  0.88
episode:  131  score:  -154.70  avg_score:  -169.65  frames:  82  epsilon:  0.88
episode:  132  score:  -170.97  avg_score:  -170.34  frames:  111  epsilon:  0.88
episode:  133  score:  -81.27  avg_score:  -170.32  frames:  86  epsilon:  0.88
episode:  134  score:  -214.85  avg_score:  -171.85  frames:  107  epsilon:  0.87
episode:  135  score:  -155.66  avg_score:  -171.18  frames:  102  epsilon:  0.87
episode:  136  score:  -15.46  avg_score:  -170.35  frames:  76  epsilon:  0.87
episode:  137  score:  -6.43  avg_score:  -169.18  frames:  79  epsilon:  0.87
episode:  138  score:  -191.82  avg_score:  -170.28  frames:  105  epsilon:  0.87
episode:  139  score:  -163.99  avg_score:  -171.23  frames:  81  epsilon:  0.87
episode:  140  score:  -142.04  avg_score:  -167.89  frames:  110  epsilon:  0.87
episode:  141  score:  -52.02  avg_score:  -165.83  frames:  64  epsilon:  0.87
episode:  142  score:  -263.49  avg_score:  -167.36  frames:  132  epsilon:  0.87
episode:  143  score:  -87.24  avg_score:  -168.33  frames:  141  epsilon:  0.87
episode:  144  score:  -129.83  avg_score:  -169.16  frames:  118  epsilon:  0.87
episode:  145  score:  -263.22  avg_score:  -170.45  frames:  122  epsilon:  0.87
episode:  146  score:  -244.52  avg_score:  -171.57  frames:  125  epsilon:  0.86
episode:  147  score:  -359.75  avg_score:  -173.63  frames:  86  epsilon:  0.86
episode:  148  score:  -255.11  avg_score:  -173.91  frames:  98  epsilon:  0.86
episode:  149  score:  -111.28  avg_score:  -174.08  frames:  81  epsilon:  0.86
episode:  150  score:  -85.24  avg_score:  -173.64  frames:  68  epsilon:  0.86
episode:  151  score:  -208.94  avg_score:  -171.81  frames:  144  epsilon:  0.86
episode:  152  score:  -12.31  avg_score:  -168.46  frames:  161  epsilon:  0.86
episode:  153  score:  -27.65  avg_score:  -167.87  frames:  86  epsilon:  0.86
episode:  154  score:  -146.13  avg_score:  -168.03  frames:  103  epsilon:  0.86
episode:  155  score:  -365.83  avg_score:  -168.72  frames:  157  epsilon:  0.86
episode:  156  score:  -189.33  avg_score:  -167.93  frames:  80  epsilon:  0.86
episode:  157  score:  -105.14  avg_score:  -167.96  frames:  105  epsilon:  0.85
episode:  158  score:  -279.36  avg_score:  -167.44  frames:  95  epsilon:  0.85
episode:  159  score:  -268.92  avg_score:  -168.78  frames:  125  epsilon:  0.85
episode:  160  score:  -211.02  avg_score:  -169.61  frames:  117  epsilon:  0.85
episode:  161  score:  -254.01  avg_score:  -169.15  frames:  146  epsilon:  0.85
episode:  162  score:  -143.24  avg_score:  -169.26  frames:  118  epsilon:  0.85
episode:  163  score:  -84.90  avg_score:  -167.41  frames:  103  epsilon:  0.85
episode:  164  score:  -124.07  avg_score:  -166.59  frames:  112  epsilon:  0.85
episode:  165  score:  -122.16  avg_score:  -166.46  frames:  123  epsilon:  0.85
episode:  166  score:  -84.22  avg_score:  -165.87  frames:  102  epsilon:  0.85
episode:  167  score:  -71.41  avg_score:  -166.56  frames:  69  epsilon:  0.85
episode:  168  score:  -85.84  avg_score:  -166.18  frames:  85  epsilon:  0.85
episode:  169  score:  -145.56  avg_score:  -166.98  frames:  112  epsilon:  0.84
episode:  170  score:  -63.57  avg_score:  -166.30  frames:  100  epsilon:  0.84
episode:  171  score:  -159.41  avg_score:  -164.65  frames:  115  epsilon:  0.84
episode:  172  score:  -186.29  avg_score:  -165.14  frames:  75  epsilon:  0.84
episode:  173  score:  -112.75  avg_score:  -163.09  frames:  99  epsilon:  0.84
episode:  174  score:  -243.20  avg_score:  -164.21  frames:  120  epsilon:  0.84
episode:  175  score:  17.53  avg_score:  -162.55  frames:  105  epsilon:  0.84
episode:  176  score:  -182.31  avg_score:  -163.44  frames:  95  epsilon:  0.84
episode:  177  score:  -117.19  avg_score:  -163.19  frames:  86  epsilon:  0.84
episode:  178  score:  -86.66  avg_score:  -163.08  frames:  126  epsilon:  0.84
episode:  179  score:  -159.18  avg_score:  -164.06  frames:  103  epsilon:  0.84
episode:  180  score:  -108.06  avg_score:  -161.83  frames:  112  epsilon:  0.84
episode:  181  score:  -236.11  avg_score:  -163.71  frames:  121  epsilon:  0.83
episode:  182  score:  -272.13  avg_score:  -163.05  frames:  99  epsilon:  0.83
episode:  183  score:  -136.20  avg_score:  -160.50  frames:  103  epsilon:  0.83
episode:  184  score:  -173.35  avg_score:  -161.10  frames:  122  epsilon:  0.83
episode:  185  score:  -43.66  avg_score:  -158.85  frames:  97  epsilon:  0.83
episode:  186  score:  -147.41  avg_score:  -158.82  frames:  78  epsilon:  0.83
episode:  187  score:  -76.41  avg_score:  -158.75  frames:  101  epsilon:  0.83
episode:  188  score:  -214.91  avg_score:  -158.39  frames:  103  epsilon:  0.83
episode:  189  score:  -199.64  avg_score:  -160.69  frames:  107  epsilon:  0.83
episode:  190  score:  -97.84  avg_score:  -160.48  frames:  88  epsilon:  0.83
episode:  191  score:  -77.17  avg_score:  -157.72  frames:  65  epsilon:  0.83
episode:  192  score:  -126.31  avg_score:  -155.27  frames:  74  epsilon:  0.83
episode:  193  score:  -52.29  avg_score:  -154.26  frames:  69  epsilon:  0.82
episode:  194  score:  -167.01  avg_score:  -154.55  frames:  82  epsilon:  0.82
episode:  195  score:  -128.64  avg_score:  -154.33  frames:  88  epsilon:  0.82
episode:  196  score:  -73.02  avg_score:  -154.23  frames:  89  epsilon:  0.82
episode:  197  score:  -160.67  avg_score:  -155.08  frames:  143  epsilon:  0.82
episode:  198  score:  -41.14  avg_score:  -154.23  frames:  91  epsilon:  0.82
episode:  199  score:  -109.21  avg_score:  -153.84  frames:  127  epsilon:  0.82
episode:  200  score:  -49.15  avg_score:  -151.85  frames:  63  epsilon:  0.82
episode:  201  score:  -67.42  avg_score:  -151.23  frames:  94  epsilon:  0.82
episode:  202  score:  -96.23  avg_score:  -151.25  frames:  71  epsilon:  0.82
episode:  203  score:  -104.55  avg_score:  -151.04  frames:  107  epsilon:  0.82
episode:  204  score:  -36.46  avg_score:  -150.57  frames:  127  epsilon:  0.82
episode:  205  score:  -79.34  avg_score:  -148.39  frames:  79  epsilon:  0.81
episode:  206  score:  -83.49  avg_score:  -148.31  frames:  141  epsilon:  0.81
episode:  207  score:  -133.55  avg_score:  -148.76  frames:  113  epsilon:  0.81
episode:  208  score:  -63.33  avg_score:  -148.07  frames:  94  epsilon:  0.81
episode:  209  score:  -88.52  avg_score:  -146.88  frames:  87  epsilon:  0.81
episode:  210  score:  -82.10  avg_score:  -142.92  frames:  126  epsilon:  0.81
episode:  211  score:  -85.97  avg_score:  -142.40  frames:  63  epsilon:  0.81
episode:  212  score:  -102.78  avg_score:  -141.60  frames:  83  epsilon:  0.81
episode:  213  score:  -26.30  avg_score:  -139.69  frames:  93  epsilon:  0.81
episode:  214  score:  -143.87  avg_score:  -137.41  frames:  70  epsilon:  0.81
episode:  215  score:  -75.28  avg_score:  -135.95  frames:  96  epsilon:  0.81
episode:  216  score:  -105.72  avg_score:  -134.17  frames:  80  epsilon:  0.81
episode:  217  score:  -68.73  avg_score:  -134.91  frames:  99  epsilon:  0.80
episode:  218  score:  -188.81  avg_score:  -135.63  frames:  112  epsilon:  0.80
episode:  219  score:  -40.65  avg_score:  -135.81  frames:  80  epsilon:  0.80
episode:  220  score:  -105.43  avg_score:  -135.68  frames:  126  epsilon:  0.80
episode:  221  score:  -80.56  avg_score:  -131.04  frames:  84  epsilon:  0.80
episode:  222  score:  -60.35  avg_score:  -130.60  frames:  109  epsilon:  0.80
episode:  223  score:  -104.19  avg_score:  -129.84  frames:  106  epsilon:  0.80
episode:  224  score:  -115.02  avg_score:  -130.73  frames:  152  epsilon:  0.80
episode:  225  score:  -99.75  avg_score:  -130.86  frames:  91  epsilon:  0.80
episode:  226  score:  -89.39  avg_score:  -127.18  frames:  122  epsilon:  0.80
episode:  227  score:  -158.26  avg_score:  -127.11  frames:  96  epsilon:  0.80
episode:  228  score:  -109.27  avg_score:  -128.04  frames:  65  epsilon:  0.80
episode:  229  score:  -84.62  avg_score:  -128.31  frames:  93  epsilon:  0.80
episode:  230  score:  -68.43  avg_score:  -128.11  frames:  101  epsilon:  0.79
episode:  231  score:  -142.63  avg_score:  -127.99  frames:  121  epsilon:  0.79
episode:  232  score:  -43.57  avg_score:  -126.71  frames:  153  epsilon:  0.79
episode:  233  score:  -46.95  avg_score:  -126.37  frames:  95  epsilon:  0.79
episode:  234  score:  -145.73  avg_score:  -125.68  frames:  87  epsilon:  0.79
episode:  235  score:  -73.78  avg_score:  -124.86  frames:  124  epsilon:  0.79
episode:  236  score:  -155.96  avg_score:  -126.26  frames:  121  epsilon:  0.79
episode:  237  score:  -50.98  avg_score:  -126.71  frames:  79  epsilon:  0.79
episode:  238  score:  -73.85  avg_score:  -125.53  frames:  90  epsilon:  0.79
episode:  239  score:  -81.98  avg_score:  -124.71  frames:  117  epsilon:  0.79
episode:  240  score:  -36.49  avg_score:  -123.65  frames:  145  epsilon:  0.79
episode:  241  score:  -175.32  avg_score:  -124.89  frames:  122  epsilon:  0.79
episode:  242  score:  -49.99  avg_score:  -122.75  frames:  104  epsilon:  0.79
episode:  243  score:  -77.90  avg_score:  -122.66  frames:  94  epsilon:  0.78
episode:  244  score:  -64.26  avg_score:  -122.00  frames:  156  epsilon:  0.78
episode:  245  score:  -46.49  avg_score:  -119.84  frames:  74  epsilon:  0.78
episode:  246  score:  -78.84  avg_score:  -118.18  frames:  76  epsilon:  0.78
episode:  247  score:  -137.71  avg_score:  -115.96  frames:  103  epsilon:  0.78
episode:  248  score:  -54.37  avg_score:  -113.95  frames:  112  epsilon:  0.78
episode:  249  score:  -64.93  avg_score:  -113.49  frames:  102  epsilon:  0.78
episode:  250  score:  -56.56  avg_score:  -113.20  frames:  102  epsilon:  0.78
episode:  251  score:  -36.85  avg_score:  -111.48  frames:  85  epsilon:  0.78
episode:  252  score:  -49.63  avg_score:  -111.85  frames:  108  epsilon:  0.78
episode:  253  score:  -117.97  avg_score:  -112.76  frames:  99  epsilon:  0.78
episode:  254  score:  -17.95  avg_score:  -111.47  frames:  121  epsilon:  0.78
episode:  255  score:  -66.68  avg_score:  -108.48  frames:  117  epsilon:  0.77
episode:  256  score:  -107.66  avg_score:  -107.67  frames:  66  epsilon:  0.77
episode:  257  score:  -109.51  avg_score:  -107.71  frames:  99  epsilon:  0.77
episode:  258  score:  -172.89  avg_score:  -106.64  frames:  127  epsilon:  0.77
episode:  259  score:  -128.77  avg_score:  -105.24  frames:  89  epsilon:  0.77
episode:  260  score:  -69.33  avg_score:  -103.83  frames:  79  epsilon:  0.77
episode:  261  score:  -140.42  avg_score:  -102.69  frames:  82  epsilon:  0.77
episode:  262  score:  -89.95  avg_score:  -102.16  frames:  112  epsilon:  0.77
episode:  263  score:  -78.45  avg_score:  -102.09  frames:  159  epsilon:  0.77
episode:  264  score:  -80.15  avg_score:  -101.65  frames:  79  epsilon:  0.77
episode:  265  score:  -139.35  avg_score:  -101.83  frames:  125  epsilon:  0.77
episode:  266  score:  -84.17  avg_score:  -101.83  frames:  89  epsilon:  0.77
episode:  267  score:  -52.98  avg_score:  -101.64  frames:  87  epsilon:  0.77
episode:  268  score:  -52.57  avg_score:  -101.31  frames:  82  epsilon:  0.76
episode:  269  score:  -143.54  avg_score:  -101.29  frames:  79  epsilon:  0.76
episode:  270  score:  -68.60  avg_score:  -101.34  frames:  136  epsilon:  0.76
episode:  271  score:  -60.45  avg_score:  -100.35  frames:  89  epsilon:  0.76
episode:  272  score:  -96.52  avg_score:  -99.45  frames:  98  epsilon:  0.76
episode:  273  score:  -98.52  avg_score:  -99.31  frames:  120  epsilon:  0.76
episode:  274  score:  -66.25  avg_score:  -97.54  frames:  125  epsilon:  0.76
episode:  275  score:  -42.14  avg_score:  -98.14  frames:  117  epsilon:  0.76
episode:  276  score:  -115.08  avg_score:  -97.46  frames:  89  epsilon:  0.76
episode:  277  score:  -141.08  avg_score:  -97.70  frames:  66  epsilon:  0.76
episode:  278  score:  -70.93  avg_score:  -97.55  frames:  78  epsilon:  0.76
episode:  279  score:  -118.98  avg_score:  -97.14  frames:  138  epsilon:  0.76
episode:  280  score:  -57.81  avg_score:  -96.64  frames:  147  epsilon:  0.76
episode:  281  score:  30.73  avg_score:  -93.97  frames:  155  epsilon:  0.76
episode:  282  score:  -204.05  avg_score:  -93.29  frames:  158  epsilon:  0.75
episode:  283  score:  -75.83  avg_score:  -92.69  frames:  65  epsilon:  0.75
episode:  284  score:  -122.25  avg_score:  -92.18  frames:  138  epsilon:  0.75
episode:  285  score:  -178.49  avg_score:  -93.53  frames:  92  epsilon:  0.75
episode:  286  score:  -71.60  avg_score:  -92.77  frames:  76  epsilon:  0.75
episode:  287  score:  -47.34  avg_score:  -92.48  frames:  96  epsilon:  0.75
episode:  288  score:  -75.40  avg_score:  -91.08  frames:  80  epsilon:  0.75
episode:  289  score:  -60.59  avg_score:  -89.69  frames:  111  epsilon:  0.75
episode:  290  score:  -87.91  avg_score:  -89.59  frames:  66  epsilon:  0.75
episode:  291  score:  -62.08  avg_score:  -89.44  frames:  102  epsilon:  0.75
episode:  292  score:  -196.80  avg_score:  -90.15  frames:  99  epsilon:  0.75
episode:  293  score:  -100.70  avg_score:  -90.63  frames:  122  epsilon:  0.75
episode:  294  score:  -78.57  avg_score:  -89.75  frames:  90  epsilon:  0.75
episode:  295  score:  -104.40  avg_score:  -89.50  frames:  126  epsilon:  0.74
episode:  296  score:  -82.82  avg_score:  -89.60  frames:  148  epsilon:  0.74
episode:  297  score:  -125.78  avg_score:  -89.25  frames:  112  epsilon:  0.74
episode:  298  score:  -111.04  avg_score:  -89.95  frames:  118  epsilon:  0.74
episode:  299  score:  -66.03  avg_score:  -89.52  frames:  141  epsilon:  0.74
episode:  300  score:  -80.47  avg_score:  -89.83  frames:  193  epsilon:  0.74
episode:  301  score:  -76.41  avg_score:  -89.92  frames:  139  epsilon:  0.74
episode:  302  score:  -76.70  avg_score:  -89.73  frames:  100  epsilon:  0.74
episode:  303  score:  24.26  avg_score:  -88.44  frames:  84  epsilon:  0.74
episode:  304  score:  -171.41  avg_score:  -89.79  frames:  112  epsilon:  0.74
episode:  305  score:  -51.03  avg_score:  -89.51  frames:  130  epsilon:  0.74
episode:  306  score:  -151.56  avg_score:  -90.19  frames:  129  epsilon:  0.74
episode:  307  score:  -61.59  avg_score:  -89.47  frames:  108  epsilon:  0.74
episode:  308  score:  -238.05  avg_score:  -91.21  frames:  99  epsilon:  0.73
episode:  309  score:  -105.24  avg_score:  -91.38  frames:  76  epsilon:  0.73
episode:  310  score:  -99.72  avg_score:  -91.56  frames:  127  epsilon:  0.73
episode:  311  score:  -75.10  avg_score:  -91.45  frames:  78  epsilon:  0.73
episode:  312  score:  -65.71  avg_score:  -91.08  frames:  125  epsilon:  0.73
episode:  313  score:  -50.35  avg_score:  -91.32  frames:  130  epsilon:  0.73
episode:  314  score:  -116.97  avg_score:  -91.05  frames:  107  epsilon:  0.73
episode:  315  score:  -105.54  avg_score:  -91.35  frames:  111  epsilon:  0.73
episode:  316  score:  -57.50  avg_score:  -90.87  frames:  158  epsilon:  0.73
episode:  317  score:  -38.06  avg_score:  -90.56  frames:  157  epsilon:  0.73
episode:  318  score:  -150.55  avg_score:  -90.18  frames:  118  epsilon:  0.73
episode:  319  score:  -5.89  avg_score:  -89.83  frames:  85  epsilon:  0.73
episode:  320  score:  -145.79  avg_score:  -90.24  frames:  96  epsilon:  0.73
episode:  321  score:  -101.05  avg_score:  -90.44  frames:  111  epsilon:  0.73
episode:  322  score:  -29.36  avg_score:  -90.13  frames:  106  epsilon:  0.72
episode:  323  score:  -102.50  avg_score:  -90.11  frames:  156  epsilon:  0.72
episode:  324  score:  -78.53  avg_score:  -89.75  frames:  112  epsilon:  0.72
episode:  325  score:  10.51  avg_score:  -88.65  frames:  170  epsilon:  0.72
episode:  326  score:  3.36  avg_score:  -87.72  frames:  137  epsilon:  0.72
episode:  327  score:  -82.47  avg_score:  -86.96  frames:  90  epsilon:  0.72
episode:  328  score:  -139.92  avg_score:  -87.27  frames:  121  epsilon:  0.72
episode:  329  score:  -87.34  avg_score:  -87.30  frames:  94  epsilon:  0.72
episode:  330  score:  -50.03  avg_score:  -87.11  frames:  137  epsilon:  0.72
episode:  331  score:  -29.98  avg_score:  -85.99  frames:  94  epsilon:  0.72
episode:  332  score:  -81.56  avg_score:  -86.36  frames:  99  epsilon:  0.72
episode:  333  score:  -46.19  avg_score:  -86.36  frames:  106  epsilon:  0.72
episode:  334  score:  -50.75  avg_score:  -85.41  frames:  147  epsilon:  0.72
episode:  335  score:  -58.94  avg_score:  -85.26  frames:  119  epsilon:  0.72
episode:  336  score:  -80.96  avg_score:  -84.51  frames:  116  epsilon:  0.71
episode:  337  score:  -109.29  avg_score:  -85.09  frames:  111  epsilon:  0.71
episode:  338  score:  -85.10  avg_score:  -85.20  frames:  116  epsilon:  0.71
episode:  339  score:  -70.96  avg_score:  -85.09  frames:  101  epsilon:  0.71
episode:  340  score:  -126.73  avg_score:  -86.00  frames:  96  epsilon:  0.71
episode:  341  score:  -58.08  avg_score:  -84.82  frames:  169  epsilon:  0.71
episode:  342  score:  -109.03  avg_score:  -85.42  frames:  108  epsilon:  0.71
episode:  343  score:  11.64  avg_score:  -84.52  frames:  82  epsilon:  0.71
episode:  344  score:  -91.35  avg_score:  -84.79  frames:  108  epsilon:  0.71
episode:  345  score:  -104.78  avg_score:  -85.37  frames:  117  epsilon:  0.71
episode:  346  score:  -40.72  avg_score:  -84.99  frames:  78  epsilon:  0.71
episode:  347  score:  -36.94  avg_score:  -83.98  frames:  91  epsilon:  0.71
episode:  348  score:  -50.12  avg_score:  -83.94  frames:  204  epsilon:  0.71
episode:  349  score:  -78.32  avg_score:  -84.08  frames:  179  epsilon:  0.71
episode:  350  score:  -88.08  avg_score:  -84.39  frames:  156  epsilon:  0.70
episode:  351  score:  -75.85  avg_score:  -84.78  frames:  118  epsilon:  0.70
episode:  352  score:  -71.19  avg_score:  -85.00  frames:  180  epsilon:  0.70
episode:  353  score:  -149.82  avg_score:  -85.31  frames:  122  epsilon:  0.70
episode:  354  score:  -48.28  avg_score:  -85.62  frames:  141  epsilon:  0.70
episode:  355  score:  -100.33  avg_score:  -85.95  frames:  82  epsilon:  0.70
episode:  356  score:  -98.82  avg_score:  -85.87  frames:  100  epsilon:  0.70
episode:  357  score:  -101.89  avg_score:  -85.79  frames:  188  epsilon:  0.70
episode:  358  score:  -59.40  avg_score:  -84.66  frames:  150  epsilon:  0.70
episode:  359  score:  -124.44  avg_score:  -84.61  frames:  120  epsilon:  0.70
episode:  360  score:  -130.96  avg_score:  -85.23  frames:  147  epsilon:  0.70
episode:  361  score:  -87.91  avg_score:  -84.70  frames:  94  epsilon:  0.70
episode:  362  score:  -62.05  avg_score:  -84.42  frames:  139  epsilon:  0.70
episode:  363  score:  -65.36  avg_score:  -84.29  frames:  89  epsilon:  0.70
episode:  364  score:  -141.57  avg_score:  -84.91  frames:  135  epsilon:  0.69
episode:  365  score:  -83.31  avg_score:  -84.35  frames:  79  epsilon:  0.69
episode:  366  score:  -50.04  avg_score:  -84.01  frames:  76  epsilon:  0.69
episode:  367  score:  -64.62  avg_score:  -84.12  frames:  85  epsilon:  0.69
episode:  368  score:  -60.48  avg_score:  -84.20  frames:  88  epsilon:  0.69
episode:  369  score:  -125.34  avg_score:  -84.02  frames:  60  epsilon:  0.69
episode:  370  score:  -77.15  avg_score:  -84.10  frames:  96  epsilon:  0.69
episode:  371  score:  -104.52  avg_score:  -84.55  frames:  105  epsilon:  0.69
episode:  372  score:  70.41  avg_score:  -82.88  frames:  999  epsilon:  0.69
episode:  373  score:  -161.72  avg_score:  -83.51  frames:  110  epsilon:  0.69
episode:  374  score:  -110.56  avg_score:  -83.95  frames:  97  epsilon:  0.69
episode:  375  score:  -40.41  avg_score:  -83.93  frames:  116  epsilon:  0.69
episode:  376  score:  -68.86  avg_score:  -83.47  frames:  82  epsilon:  0.69
episode:  377  score:  -107.85  avg_score:  -83.14  frames:  88  epsilon:  0.69
episode:  378  score:  -47.42  avg_score:  -82.90  frames:  158  epsilon:  0.69
episode:  379  score:  -113.82  avg_score:  -82.85  frames:  87  epsilon:  0.68
episode:  380  score:  -37.60  avg_score:  -82.65  frames:  156  epsilon:  0.68
episode:  381  score:  -91.17  avg_score:  -83.87  frames:  187  epsilon:  0.68
episode:  382  score:  -115.79  avg_score:  -82.99  frames:  95  epsilon:  0.68
episode:  383  score:  -71.78  avg_score:  -82.95  frames:  121  epsilon:  0.68
episode:  384  score:  -79.37  avg_score:  -82.52  frames:  100  epsilon:  0.68
episode:  385  score:  -11.00  avg_score:  -80.84  frames:  169  epsilon:  0.68
episode:  386  score:  -121.04  avg_score:  -81.34  frames:  118  epsilon:  0.68
episode:  387  score:  -62.02  avg_score:  -81.48  frames:  167  epsilon:  0.68
episode:  388  score:  -31.06  avg_score:  -81.04  frames:  143  epsilon:  0.68
episode:  389  score:  -99.22  avg_score:  -81.43  frames:  139  epsilon:  0.68
episode:  390  score:  -90.31  avg_score:  -81.45  frames:  132  epsilon:  0.68
episode:  391  score:  -110.74  avg_score:  -81.94  frames:  87  epsilon:  0.68
episode:  392  score:  -59.03  avg_score:  -80.56  frames:  83  epsilon:  0.68
episode:  393  score:  -13.95  avg_score:  -79.69  frames:  153  epsilon:  0.68
episode:  394  score:  -78.06  avg_score:  -79.69  frames:  109  epsilon:  0.67
episode:  395  score:  -94.54  avg_score:  -79.59  frames:  111  epsilon:  0.67
episode:  396  score:  -84.12  avg_score:  -79.60  frames:  95  epsilon:  0.67
episode:  397  score:  -71.12  avg_score:  -79.06  frames:  76  epsilon:  0.67
episode:  398  score:  -71.83  avg_score:  -78.66  frames:  151  epsilon:  0.67
episode:  399  score:  -64.34  avg_score:  -78.65  frames:  89  epsilon:  0.67
episode:  400  score:  -196.09  avg_score:  -79.80  frames:  144  epsilon:  0.67
episode:  401  score:  -69.20  avg_score:  -79.73  frames:  88  epsilon:  0.67
episode:  402  score:  -83.51  avg_score:  -79.80  frames:  116  epsilon:  0.67
episode:  403  score:  -64.20  avg_score:  -80.68  frames:  102  epsilon:  0.67
episode:  404  score:  -51.62  avg_score:  -79.49  frames:  95  epsilon:  0.67
episode:  405  score:  -59.24  avg_score:  -79.57  frames:  170  epsilon:  0.67
episode:  406  score:  -118.00  avg_score:  -79.23  frames:  106  epsilon:  0.67
episode:  407  score:  -43.14  avg_score:  -79.05  frames:  113  epsilon:  0.67
episode:  408  score:  -108.44  avg_score:  -77.75  frames:  122  epsilon:  0.67
episode:  409  score:  45.84  avg_score:  -76.24  frames:  174  epsilon:  0.66
episode:  410  score:  -106.75  avg_score:  -76.31  frames:  89  epsilon:  0.66
episode:  411  score:  -71.47  avg_score:  -76.27  frames:  126  epsilon:  0.66
episode:  412  score:  -42.48  avg_score:  -76.04  frames:  184  epsilon:  0.66
episode:  413  score:  -62.27  avg_score:  -76.16  frames:  61  epsilon:  0.66
episode:  414  score:  -112.95  avg_score:  -76.12  frames:  88  epsilon:  0.66
episode:  415  score:  -38.95  avg_score:  -75.45  frames:  203  epsilon:  0.66
episode:  416  score:  -54.51  avg_score:  -75.42  frames:  167  epsilon:  0.66
episode:  417  score:  14.02  avg_score:  -74.90  frames:  86  epsilon:  0.66
episode:  418  score:  -188.36  avg_score:  -75.28  frames:  193  epsilon:  0.66
episode:  419  score:  -17.48  avg_score:  -75.40  frames:  148  epsilon:  0.66
episode:  420  score:  -69.41  avg_score:  -74.63  frames:  132  epsilon:  0.66
episode:  421  score:  -49.14  avg_score:  -74.12  frames:  131  epsilon:  0.66
episode:  422  score:  -97.47  avg_score:  -74.80  frames:  160  epsilon:  0.66
episode:  423  score:  -25.80  avg_score:  -74.03  frames:  113  epsilon:  0.66
episode:  424  score:  -69.25  avg_score:  -73.94  frames:  117  epsilon:  0.65
episode:  425  score:  -44.82  avg_score:  -74.49  frames:  160  epsilon:  0.65
episode:  426  score:  -12.54  avg_score:  -74.65  frames:  140  epsilon:  0.65
episode:  427  score:  -165.14  avg_score:  -75.48  frames:  147  epsilon:  0.65
episode:  428  score:  -134.26  avg_score:  -75.42  frames:  120  epsilon:  0.65
episode:  429  score:  -92.59  avg_score:  -75.47  frames:  77  epsilon:  0.65
episode:  430  score:  -94.69  avg_score:  -75.92  frames:  65  epsilon:  0.65
episode:  431  score:  -32.36  avg_score:  -75.94  frames:  140  epsilon:  0.65
episode:  432  score:  -55.43  avg_score:  -75.68  frames:  88  epsilon:  0.65
episode:  433  score:  -46.17  avg_score:  -75.68  frames:  208  epsilon:  0.65
episode:  434  score:  -40.06  avg_score:  -75.57  frames:  158  epsilon:  0.65
episode:  435  score:  -76.20  avg_score:  -75.75  frames:  106  epsilon:  0.65
episode:  436  score:  -85.74  avg_score:  -75.79  frames:  99  epsilon:  0.65
episode:  437  score:  -50.26  avg_score:  -75.20  frames:  163  epsilon:  0.65
episode:  438  score:  -38.89  avg_score:  -74.74  frames:  127  epsilon:  0.65
episode:  439  score:  -67.46  avg_score:  -74.71  frames:  153  epsilon:  0.64
episode:  440  score:  -13.93  avg_score:  -73.58  frames:  128  epsilon:  0.64
episode:  441  score:  -94.34  avg_score:  -73.94  frames:  149  epsilon:  0.64
episode:  442  score:  -74.30  avg_score:  -73.59  frames:  80  epsilon:  0.64
episode:  443  score:  -117.20  avg_score:  -74.88  frames:  219  epsilon:  0.64
episode:  444  score:  -95.25  avg_score:  -74.92  frames:  104  epsilon:  0.64
episode:  445  score:  -123.72  avg_score:  -75.11  frames:  120  epsilon:  0.64
episode:  446  score:  -72.18  avg_score:  -75.43  frames:  180  epsilon:  0.64
episode:  447  score:  -84.39  avg_score:  -75.90  frames:  81  epsilon:  0.64
episode:  448  score:  -36.44  avg_score:  -75.76  frames:  197  epsilon:  0.64
episode:  449  score:  -15.14  avg_score:  -75.13  frames:  95  epsilon:  0.64
episode:  450  score:  -65.42  avg_score:  -74.90  frames:  130  epsilon:  0.64
episode:  451  score:  -91.57  avg_score:  -75.06  frames:  81  epsilon:  0.64
episode:  452  score:  23.58  avg_score:  -74.11  frames:  128  epsilon:  0.64
episode:  453  score:  -86.64  avg_score:  -73.48  frames:  85  epsilon:  0.64
episode:  454  score:  -72.92  avg_score:  -73.73  frames:  82  epsilon:  0.64
episode:  455  score:  -65.85  avg_score:  -73.38  frames:  114  epsilon:  0.63
episode:  456  score:  -46.51  avg_score:  -72.86  frames:  92  epsilon:  0.63
episode:  457  score:  -69.16  avg_score:  -72.53  frames:  113  epsilon:  0.63
episode:  458  score:  -40.08  avg_score:  -72.34  frames:  144  epsilon:  0.63
episode:  459  score:  -90.66  avg_score:  -72.00  frames:  107  epsilon:  0.63
episode:  460  score:  -88.48  avg_score:  -71.58  frames:  134  epsilon:  0.63
episode:  461  score:  -112.33  avg_score:  -71.82  frames:  158  epsilon:  0.63
episode:  462  score:  -92.75  avg_score:  -72.13  frames:  78  epsilon:  0.63
episode:  463  score:  -91.31  avg_score:  -72.39  frames:  91  epsilon:  0.63
episode:  464  score:  -159.48  avg_score:  -72.57  frames:  186  epsilon:  0.63
episode:  465  score:  -168.05  avg_score:  -73.41  frames:  121  epsilon:  0.63
episode:  466  score:  -71.37  avg_score:  -73.63  frames:  103  epsilon:  0.63
episode:  467  score:  -38.63  avg_score:  -73.37  frames:  92  epsilon:  0.63
episode:  468  score:  -7.22  avg_score:  -72.84  frames:  169  epsilon:  0.63
episode:  469  score:  -73.29  avg_score:  -72.31  frames:  127  epsilon:  0.63
episode:  470  score:  -49.64  avg_score:  -72.04  frames:  193  epsilon:  0.63
episode:  471  score:  -113.57  avg_score:  -72.13  frames:  110  epsilon:  0.62
episode:  472  score:  -188.82  avg_score:  -74.72  frames:  129  epsilon:  0.62
episode:  473  score:  -119.32  avg_score:  -74.30  frames:  144  epsilon:  0.62
episode:  474  score:  -81.49  avg_score:  -74.01  frames:  71  epsilon:  0.62
episode:  475  score:  -79.29  avg_score:  -74.40  frames:  170  epsilon:  0.62
episode:  476  score:  -88.40  avg_score:  -74.59  frames:  96  epsilon:  0.62
episode:  477  score:  -58.20  avg_score:  -74.10  frames:  75  epsilon:  0.62
episode:  478  score:  -95.00  avg_score:  -74.57  frames:  130  epsilon:  0.62
episode:  479  score:  -91.65  avg_score:  -74.35  frames:  181  epsilon:  0.62
episode:  480  score:  -40.00  avg_score:  -74.37  frames:  135  epsilon:  0.62
episode:  481  score:  -105.88  avg_score:  -74.52  frames:  160  epsilon:  0.62
episode:  482  score:  -57.61  avg_score:  -73.94  frames:  103  epsilon:  0.62
episode:  483  score:  -34.42  avg_score:  -73.57  frames:  102  epsilon:  0.62
episode:  484  score:  26.16  avg_score:  -72.51  frames:  176  epsilon:  0.62
episode:  485  score:  -28.13  avg_score:  -72.68  frames:  90  epsilon:  0.62
episode:  486  score:  -42.63  avg_score:  -71.90  frames:  204  epsilon:  0.62
episode:  487  score:  -70.33  avg_score:  -71.98  frames:  130  epsilon:  0.61
episode:  488  score:  -65.78  avg_score:  -72.33  frames:  142  epsilon:  0.61
episode:  489  score:  -31.37  avg_score:  -71.65  frames:  120  epsilon:  0.61
episode:  490  score:  -63.84  avg_score:  -71.38  frames:  65  epsilon:  0.61
episode:  491  score:  -65.62  avg_score:  -70.93  frames:  113  epsilon:  0.61
episode:  492  score:  -74.13  avg_score:  -71.08  frames:  139  epsilon:  0.61
episode:  493  score:  -83.65  avg_score:  -71.78  frames:  104  epsilon:  0.61
episode:  494  score:  -67.39  avg_score:  -71.67  frames:  138  epsilon:  0.61
episode:  495  score:  -111.56  avg_score:  -71.84  frames:  106  epsilon:  0.61
episode:  496  score:  -71.35  avg_score:  -71.72  frames:  121  epsilon:  0.61
episode:  497  score:  -13.01  avg_score:  -71.14  frames:  164  epsilon:  0.61
episode:  498  score:  -46.76  avg_score:  -70.88  frames:  160  epsilon:  0.61
episode:  499  score:  -51.35  avg_score:  -70.76  frames:  217  epsilon:  0.61
episode:  500  score:  -43.64  avg_score:  -69.23  frames:  207  epsilon:  0.61
episode:  501  score:  -61.47  avg_score:  -69.15  frames:  112  epsilon:  0.61
episode:  502  score:  -27.80  avg_score:  -68.60  frames:  181  epsilon:  0.61
episode:  503  score:  -22.97  avg_score:  -68.18  frames:  128  epsilon:  0.60
episode:  504  score:  40.37  avg_score:  -67.26  frames:  183  epsilon:  0.60
episode:  505  score:  -65.32  avg_score:  -67.32  frames:  117  epsilon:  0.60
episode:  506  score:  -72.22  avg_score:  -66.87  frames:  75  epsilon:  0.60
episode:  507  score:  -0.04  avg_score:  -66.44  frames:  137  epsilon:  0.60
episode:  508  score:  -66.88  avg_score:  -66.02  frames:  126  epsilon:  0.60
episode:  509  score:  -114.58  avg_score:  -67.62  frames:  156  epsilon:  0.60
episode:  510  score:  -31.82  avg_score:  -66.88  frames:  169  epsilon:  0.60
episode:  511  score:  -16.03  avg_score:  -66.32  frames:  161  epsilon:  0.60
episode:  512  score:  -52.12  avg_score:  -66.42  frames:  148  epsilon:  0.60
episode:  513  score:  -167.82  avg_score:  -67.47  frames:  116  epsilon:  0.60
episode:  514  score:  -16.82  avg_score:  -66.51  frames:  97  epsilon:  0.60
episode:  515  score:  -76.57  avg_score:  -66.89  frames:  146  epsilon:  0.60
episode:  516  score:  -33.31  avg_score:  -66.68  frames:  106  epsilon:  0.60
episode:  517  score:  -57.23  avg_score:  -67.39  frames:  120  epsilon:  0.60
episode:  518  score:  -74.41  avg_score:  -66.25  frames:  140  epsilon:  0.60
episode:  519  score:  -19.22  avg_score:  -66.27  frames:  213  epsilon:  0.60
episode:  520  score:  -107.61  avg_score:  -66.65  frames:  141  epsilon:  0.59
episode:  521  score:  -92.95  avg_score:  -67.09  frames:  95  epsilon:  0.59
episode:  522  score:  -103.87  avg_score:  -67.15  frames:  88  epsilon:  0.59
episode:  523  score:  -49.23  avg_score:  -67.38  frames:  150  epsilon:  0.59
episode:  524  score:  -42.13  avg_score:  -67.11  frames:  95  epsilon:  0.59
episode:  525  score:  -71.21  avg_score:  -67.38  frames:  96  epsilon:  0.59
episode:  526  score:  -70.28  avg_score:  -67.95  frames:  97  epsilon:  0.59
episode:  527  score:  -48.16  avg_score:  -66.78  frames:  183  epsilon:  0.59
episode:  528  score:  -51.58  avg_score:  -65.96  frames:  113  epsilon:  0.59
episode:  529  score:  5.83  avg_score:  -64.97  frames:  354  epsilon:  0.59
episode:  530  score:  -49.97  avg_score:  -64.53  frames:  93  epsilon:  0.59
episode:  531  score:  -25.19  avg_score:  -64.45  frames:  139  epsilon:  0.59
episode:  532  score:  -37.33  avg_score:  -64.27  frames:  159  epsilon:  0.59
episode:  533  score:  -68.43  avg_score:  -64.50  frames:  125  epsilon:  0.59
episode:  534  score:  -4.64  avg_score:  -64.14  frames:  195  epsilon:  0.59
episode:  535  score:  -0.99  avg_score:  -63.39  frames:  192  epsilon:  0.59
episode:  536  score:  16.66  avg_score:  -62.37  frames:  107  epsilon:  0.59
episode:  537  score:  -17.57  avg_score:  -62.04  frames:  185  epsilon:  0.58
episode:  538  score:  -31.73  avg_score:  -61.97  frames:  77  epsilon:  0.58
episode:  539  score:  -247.57  avg_score:  -63.77  frames:  691  epsilon:  0.58
episode:  540  score:  -25.74  avg_score:  -63.89  frames:  168  epsilon:  0.58
episode:  541  score:  -60.03  avg_score:  -63.54  frames:  81  epsilon:  0.58
episode:  542  score:  16.50  avg_score:  -62.64  frames:  226  epsilon:  0.58
episode:  543  score:  -76.82  avg_score:  -62.23  frames:  168  epsilon:  0.58
episode:  544  score:  -130.41  avg_score:  -62.58  frames:  141  epsilon:  0.58
episode:  545  score:  -198.04  avg_score:  -63.33  frames:  99  epsilon:  0.58
episode:  546  score:  -81.45  avg_score:  -63.42  frames:  99  epsilon:  0.58
episode:  547  score:  -51.63  avg_score:  -63.09  frames:  124  epsilon:  0.58
episode:  548  score:  -78.70  avg_score:  -63.51  frames:  131  epsilon:  0.58
episode:  549  score:  17.10  avg_score:  -63.19  frames:  277  epsilon:  0.58
episode:  550  score:  -21.56  avg_score:  -62.75  frames:  81  epsilon:  0.58
episode:  551  score:  -57.06  avg_score:  -62.41  frames:  76  epsilon:  0.58
episode:  552  score:  -73.98  avg_score:  -63.38  frames:  135  epsilon:  0.58
episode:  553  score:  -85.03  avg_score:  -63.37  frames:  67  epsilon:  0.58
episode:  554  score:  -183.71  avg_score:  -64.48  frames:  194  epsilon:  0.57
episode:  555  score:  -30.71  avg_score:  -64.12  frames:  203  epsilon:  0.57
episode:  556  score:  -67.34  avg_score:  -64.33  frames:  88  epsilon:  0.57
episode:  557  score:  -41.44  avg_score:  -64.06  frames:  138  epsilon:  0.57
episode:  558  score:  -117.99  avg_score:  -64.83  frames:  150  epsilon:  0.57
episode:  559  score:  -73.99  avg_score:  -64.67  frames:  66  epsilon:  0.57
episode:  560  score:  -34.09  avg_score:  -64.12  frames:  91  epsilon:  0.57
episode:  561  score:  -90.76  avg_score:  -63.91  frames:  117  epsilon:  0.57
episode:  562  score:  -54.18  avg_score:  -63.52  frames:  71  epsilon:  0.57
episode:  563  score:  -66.40  avg_score:  -63.27  frames:  111  epsilon:  0.57
episode:  564  score:  0.30  avg_score:  -61.68  frames:  186  epsilon:  0.57
episode:  565  score:  -1.12  avg_score:  -60.01  frames:  101  epsilon:  0.57
episode:  566  score:  -87.07  avg_score:  -60.16  frames:  101  epsilon:  0.57
episode:  567  score:  -11.48  avg_score:  -59.89  frames:  81  epsilon:  0.57
episode:  568  score:  -17.87  avg_score:  -60.00  frames:  160  epsilon:  0.57
episode:  569  score:  -98.81  avg_score:  -60.25  frames:  163  epsilon:  0.57
episode:  570  score:  -32.36  avg_score:  -60.08  frames:  151  epsilon:  0.57
episode:  571  score:  -52.37  avg_score:  -59.47  frames:  111  epsilon:  0.57
episode:  572  score:  -45.90  avg_score:  -58.04  frames:  134  epsilon:  0.56
episode:  573  score:  -74.96  avg_score:  -57.60  frames:  57  epsilon:  0.56
episode:  574  score:  -81.78  avg_score:  -57.60  frames:  88  epsilon:  0.56
episode:  575  score:  -54.47  avg_score:  -57.35  frames:  142  epsilon:  0.56
episode:  576  score:  -36.23  avg_score:  -56.83  frames:  125  epsilon:  0.56
episode:  577  score:  -43.08  avg_score:  -56.68  frames:  86  epsilon:  0.56
episode:  578  score:  -47.60  avg_score:  -56.20  frames:  145  epsilon:  0.56
episode:  579  score:  -85.36  avg_score:  -56.14  frames:  122  epsilon:  0.56
episode:  580  score:  -71.89  avg_score:  -56.46  frames:  129  epsilon:  0.56
episode:  581  score:  -71.18  avg_score:  -56.11  frames:  107  epsilon:  0.56
episode:  582  score:  -29.31  avg_score:  -55.83  frames:  151  epsilon:  0.56
episode:  583  score:  -57.16  avg_score:  -56.06  frames:  131  epsilon:  0.56
episode:  584  score:  -44.06  avg_score:  -56.76  frames:  177  epsilon:  0.56
episode:  585  score:  -64.01  avg_score:  -57.12  frames:  79  epsilon:  0.56
episode:  586  score:  -40.74  avg_score:  -57.10  frames:  107  epsilon:  0.56
episode:  587  score:  -69.62  avg_score:  -57.09  frames:  173  epsilon:  0.56
episode:  588  score:  -22.06  avg_score:  -56.65  frames:  138  epsilon:  0.56
episode:  589  score:  -178.03  avg_score:  -58.12  frames:  106  epsilon:  0.55
episode:  590  score:  -20.73  avg_score:  -57.69  frames:  120  epsilon:  0.55
episode:  591  score:  -40.32  avg_score:  -57.44  frames:  154  epsilon:  0.55
episode:  592  score:  -78.32  avg_score:  -57.48  frames:  999  epsilon:  0.55
episode:  593  score:  2.73  avg_score:  -56.62  frames:  144  epsilon:  0.55
episode:  594  score:  -42.26  avg_score:  -56.36  frames:  140  epsilon:  0.55
episode:  595  score:  -40.21  avg_score:  -55.65  frames:  139  epsilon:  0.55
episode:  596  score:  -30.76  avg_score:  -55.24  frames:  143  epsilon:  0.55
episode:  597  score:  -30.21  avg_score:  -55.42  frames:  152  epsilon:  0.55
episode:  598  score:  -9.95  avg_score:  -55.05  frames:  182  epsilon:  0.55
episode:  599  score:  22.96  avg_score:  -54.31  frames:  999  epsilon:  0.55
episode:  600  score:  -47.04  avg_score:  -54.34  frames:  141  epsilon:  0.55
episode:  601  score:  17.06  avg_score:  -53.55  frames:  999  epsilon:  0.55
episode:  602  score:  -19.62  avg_score:  -53.47  frames:  181  epsilon:  0.55
episode:  603  score:  -36.95  avg_score:  -53.61  frames:  66  epsilon:  0.55
episode:  604  score:  -62.37  avg_score:  -54.64  frames:  96  epsilon:  0.55
episode:  605  score:  -42.41  avg_score:  -54.41  frames:  197  epsilon:  0.55
episode:  606  score:  -81.05  avg_score:  -54.50  frames:  140  epsilon:  0.55
episode:  607  score:  -157.60  avg_score:  -56.07  frames:  127  epsilon:  0.55
episode:  608  score:  -25.86  avg_score:  -55.66  frames:  111  epsilon:  0.54
episode:  609  score:  -41.65  avg_score:  -54.93  frames:  99  epsilon:  0.54
episode:  610  score:  -70.71  avg_score:  -55.32  frames:  128  epsilon:  0.54
episode:  611  score:  -85.60  avg_score:  -56.02  frames:  115  epsilon:  0.54
episode:  612  score:  -175.14  avg_score:  -57.25  frames:  172  epsilon:  0.54
episode:  613  score:  -31.49  avg_score:  -55.89  frames:  999  epsilon:  0.54
episode:  614  score:  -24.86  avg_score:  -55.97  frames:  164  epsilon:  0.54
episode:  615  score:  -55.35  avg_score:  -55.75  frames:  101  epsilon:  0.54
episode:  616  score:  -5.28  avg_score:  -55.47  frames:  197  epsilon:  0.54
episode:  617  score:  -2.80  avg_score:  -54.93  frames:  154  epsilon:  0.54
episode:  618  score:  -165.27  avg_score:  -55.84  frames:  124  epsilon:  0.54
episode:  619  score:  -30.29  avg_score:  -55.95  frames:  228  epsilon:  0.54
episode:  620  score:  -35.44  avg_score:  -55.23  frames:  134  epsilon:  0.54
episode:  621  score:  -27.14  avg_score:  -54.57  frames:  172  epsilon:  0.54
episode:  622  score:  -67.09  avg_score:  -54.20  frames:  84  epsilon:  0.54
episode:  623  score:  -59.42  avg_score:  -54.30  frames:  83  epsilon:  0.54
episode:  624  score:  25.43  avg_score:  -53.63  frames:  999  epsilon:  0.54
episode:  625  score:  -26.71  avg_score:  -53.18  frames:  237  epsilon:  0.54
episode:  626  score:  -6.98  avg_score:  -52.55  frames:  134  epsilon:  0.53
episode:  627  score:  -33.00  avg_score:  -52.40  frames:  164  epsilon:  0.53
episode:  628  score:  -105.94  avg_score:  -52.94  frames:  155  epsilon:  0.53
episode:  629  score:  -14.48  avg_score:  -53.15  frames:  206  epsilon:  0.53
episode:  630  score:  33.29  avg_score:  -52.31  frames:  246  epsilon:  0.53
episode:  631  score:  -16.76  avg_score:  -52.23  frames:  97  epsilon:  0.53
episode:  632  score:  -126.89  avg_score:  -53.12  frames:  426  epsilon:  0.53
episode:  633  score:  6.12  avg_score:  -52.38  frames:  999  epsilon:  0.53
episode:  634  score:  9.16  avg_score:  -52.24  frames:  238  epsilon:  0.53
episode:  635  score:  -17.67  avg_score:  -52.41  frames:  231  epsilon:  0.53
episode:  636  score:  -29.35  avg_score:  -52.87  frames:  158  epsilon:  0.53
episode:  637  score:  -26.60  avg_score:  -52.96  frames:  182  epsilon:  0.53
episode:  638  score:  -31.83  avg_score:  -52.96  frames:  179  epsilon:  0.53
episode:  639  score:  -99.32  avg_score:  -51.48  frames:  185  epsilon:  0.53
episode:  640  score:  -58.73  avg_score:  -51.81  frames:  198  epsilon:  0.53
episode:  641  score:  -34.23  avg_score:  -51.55  frames:  112  epsilon:  0.53
episode:  642  score:  3.57  avg_score:  -51.68  frames:  248  epsilon:  0.53
episode:  643  score:  -46.01  avg_score:  -51.37  frames:  194  epsilon:  0.53
episode:  644  score:  -11.86  avg_score:  -50.18  frames:  207  epsilon:  0.53
episode:  645  score:  -12.01  avg_score:  -48.32  frames:  148  epsilon:  0.52
episode:  646  score:  -16.49  avg_score:  -47.67  frames:  158  epsilon:  0.52
episode:  647  score:  -61.41  avg_score:  -47.77  frames:  82  epsilon:  0.52
episode:  648  score:  -72.80  avg_score:  -47.71  frames:  150  epsilon:  0.52
episode:  649  score:  -18.62  avg_score:  -48.07  frames:  162  epsilon:  0.52
episode:  650  score:  -50.85  avg_score:  -48.36  frames:  119  epsilon:  0.52
episode:  651  score:  -34.78  avg_score:  -48.14  frames:  167  epsilon:  0.52
episode:  652  score:  -9.52  avg_score:  -47.50  frames:  213  epsilon:  0.52
episode:  653  score:  -56.93  avg_score:  -47.21  frames:  112  epsilon:  0.52
episode:  654  score:  -85.56  avg_score:  -46.23  frames:  123  epsilon:  0.52
episode:  655  score:  32.29  avg_score:  -45.60  frames:  291  epsilon:  0.52
episode:  656  score:  -9.00  avg_score:  -45.02  frames:  121  epsilon:  0.52
episode:  657  score:  56.63  avg_score:  -44.04  frames:  999  epsilon:  0.52
episode:  658  score:  -52.31  avg_score:  -43.38  frames:  119  epsilon:  0.52
episode:  659  score:  32.08  avg_score:  -42.32  frames:  145  epsilon:  0.52
episode:  660  score:  -43.82  avg_score:  -42.42  frames:  203  epsilon:  0.52
episode:  661  score:  -17.70  avg_score:  -41.69  frames:  143  epsilon:  0.52
episode:  662  score:  -3.86  avg_score:  -41.19  frames:  187  epsilon:  0.52
episode:  663  score:  16.96  avg_score:  -40.35  frames:  999  epsilon:  0.52
episode:  664  score:  -59.25  avg_score:  -40.95  frames:  173  epsilon:  0.51
episode:  665  score:  7.41  avg_score:  -40.86  frames:  999  epsilon:  0.51
episode:  666  score:  -149.31  avg_score:  -41.48  frames:  204  epsilon:  0.51
episode:  667  score:  -94.09  avg_score:  -42.31  frames:  104  epsilon:  0.51
episode:  668  score:  -35.69  avg_score:  -42.49  frames:  132  epsilon:  0.51
episode:  669  score:  -19.69  avg_score:  -41.70  frames:  195  epsilon:  0.51
episode:  670  score:  -53.46  avg_score:  -41.91  frames:  88  epsilon:  0.51
episode:  671  score:  -4.85  avg_score:  -41.43  frames:  177  epsilon:  0.51
episode:  672  score:  -27.10  avg_score:  -41.25  frames:  170  epsilon:  0.51
episode:  673  score:  -76.28  avg_score:  -41.26  frames:  102  epsilon:  0.51
episode:  674  score:  -35.86  avg_score:  -40.80  frames:  123  epsilon:  0.51
episode:  675  score:  -34.94  avg_score:  -40.60  frames:  999  epsilon:  0.51
episode:  676  score:  -60.09  avg_score:  -40.84  frames:  124  epsilon:  0.51
episode:  677  score:  -199.99  avg_score:  -42.41  frames:  164  epsilon:  0.51
episode:  678  score:  -161.30  avg_score:  -43.55  frames:  929  epsilon:  0.51
episode:  679  score:  -165.15  avg_score:  -44.35  frames:  110  epsilon:  0.51
episode:  680  score:  -38.43  avg_score:  -44.01  frames:  152  epsilon:  0.51
episode:  681  score:  -116.77  avg_score:  -44.47  frames:  134  epsilon:  0.51
episode:  682  score:  -25.04  avg_score:  -44.43  frames:  99  epsilon:  0.51
episode:  683  score:  -188.02  avg_score:  -45.73  frames:  160  epsilon:  0.51
episode:  684  score:  -100.72  avg_score:  -46.30  frames:  133  epsilon:  0.50
episode:  685  score:  -75.77  avg_score:  -46.42  frames:  147  epsilon:  0.50
episode:  686  score:  -95.20  avg_score:  -46.96  frames:  106  epsilon:  0.50
episode:  687  score:  -39.10  avg_score:  -46.66  frames:  199  epsilon:  0.50
episode:  688  score:  -7.92  avg_score:  -46.52  frames:  125  epsilon:  0.50
episode:  689  score:  30.50  avg_score:  -44.43  frames:  133  epsilon:  0.50
episode:  690  score:  68.97  avg_score:  -43.53  frames:  999  epsilon:  0.50
episode:  691  score:  -27.54  avg_score:  -43.41  frames:  122  epsilon:  0.50
episode:  692  score:  -66.18  avg_score:  -43.28  frames:  100  epsilon:  0.50
episode:  693  score:  -75.22  avg_score:  -44.06  frames:  79  epsilon:  0.50
episode:  694  score:  -139.12  avg_score:  -45.03  frames:  389  epsilon:  0.50
episode:  695  score:  -22.41  avg_score:  -44.85  frames:  185  epsilon:  0.50
episode:  696  score:  3.93  avg_score:  -44.51  frames:  220  epsilon:  0.50
episode:  697  score:  -58.74  avg_score:  -44.79  frames:  99  epsilon:  0.50
episode:  698  score:  -69.76  avg_score:  -45.39  frames:  111  epsilon:  0.50
episode:  699  score:  -89.65  avg_score:  -46.52  frames:  168  epsilon:  0.50
episode:  700  score:  -16.49  avg_score:  -46.21  frames:  137  epsilon:  0.50
episode:  701  score:  -188.48  avg_score:  -48.27  frames:  321  epsilon:  0.50
episode:  702  score:  -83.12  avg_score:  -48.90  frames:  115  epsilon:  0.50
episode:  703  score:  -2.85  avg_score:  -48.56  frames:  150  epsilon:  0.50
episode:  704  score:  -4.10  avg_score:  -47.98  frames:  135  epsilon:  0.49
episode:  705  score:  -48.54  avg_score:  -48.04  frames:  174  epsilon:  0.49
episode:  706  score:  -18.31  avg_score:  -47.41  frames:  84  epsilon:  0.49
episode:  707  score:  -1.96  avg_score:  -45.86  frames:  124  epsilon:  0.49
episode:  708  score:  66.53  avg_score:  -44.93  frames:  999  epsilon:  0.49
episode:  709  score:  -36.19  avg_score:  -44.88  frames:  192  epsilon:  0.49
episode:  710  score:  5.84  avg_score:  -44.11  frames:  163  epsilon:  0.49
episode:  711  score:  9.47  avg_score:  -43.16  frames:  239  epsilon:  0.49
episode:  712  score:  42.47  avg_score:  -40.99  frames:  246  epsilon:  0.49
episode:  713  score:  18.48  avg_score:  -40.49  frames:  164  epsilon:  0.49
episode:  714  score:  -13.55  avg_score:  -40.37  frames:  131  epsilon:  0.49
episode:  715  score:  59.26  avg_score:  -39.23  frames:  999  epsilon:  0.49
episode:  716  score:  -4.06  avg_score:  -39.21  frames:  214  epsilon:  0.49
episode:  717  score:  -37.92  avg_score:  -39.57  frames:  223  epsilon:  0.49
episode:  718  score:  -85.09  avg_score:  -38.76  frames:  159  epsilon:  0.49
episode:  719  score:  3.73  avg_score:  -38.42  frames:  211  epsilon:  0.49
episode:  720  score:  -55.16  avg_score:  -38.62  frames:  167  epsilon:  0.49
episode:  721  score:  -17.80  avg_score:  -38.53  frames:  146  epsilon:  0.49
episode:  722  score:  42.89  avg_score:  -37.43  frames:  999  epsilon:  0.49
episode:  723  score:  -5.85  avg_score:  -36.89  frames:  251  epsilon:  0.49
episode:  724  score:  89.40  avg_score:  -36.25  frames:  999  epsilon:  0.48
episode:  725  score:  -116.72  avg_score:  -37.15  frames:  298  epsilon:  0.48
episode:  726  score:  -1.09  avg_score:  -37.09  frames:  147  epsilon:  0.48
episode:  727  score:  -48.35  avg_score:  -37.25  frames:  129  epsilon:  0.48
episode:  728  score:  -13.41  avg_score:  -36.32  frames:  142  epsilon:  0.48
episode:  729  score:  -35.47  avg_score:  -36.53  frames:  156  epsilon:  0.48
episode:  730  score:  21.47  avg_score:  -36.65  frames:  999  epsilon:  0.48
episode:  731  score:  -59.32  avg_score:  -37.07  frames:  163  epsilon:  0.48
episode:  732  score:  -81.69  avg_score:  -36.62  frames:  121  epsilon:  0.48
episode:  733  score:  -56.74  avg_score:  -37.25  frames:  134  epsilon:  0.48
episode:  734  score:  23.47  avg_score:  -37.11  frames:  999  epsilon:  0.48
episode:  735  score:  30.53  avg_score:  -36.63  frames:  179  epsilon:  0.48
episode:  736  score:  -51.72  avg_score:  -36.85  frames:  133  epsilon:  0.48
episode:  737  score:  1.35  avg_score:  -36.57  frames:  127  epsilon:  0.48
episode:  738  score:  -45.89  avg_score:  -36.71  frames:  107  epsilon:  0.48
episode:  739  score:  101.58  avg_score:  -34.70  frames:  999  epsilon:  0.48
episode:  740  score:  -54.74  avg_score:  -34.66  frames:  194  epsilon:  0.48
episode:  741  score:  19.35  avg_score:  -34.13  frames:  129  epsilon:  0.48
episode:  742  score:  6.17  avg_score:  -34.10  frames:  151  epsilon:  0.48
episode:  743  score:  -52.75  avg_score:  -34.17  frames:  152  epsilon:  0.48
episode:  744  score:  -96.65  avg_score:  -35.02  frames:  94  epsilon:  0.48
episode:  745  score:  12.37  avg_score:  -34.77  frames:  99  epsilon:  0.47
episode:  746  score:  -39.61  avg_score:  -35.00  frames:  174  epsilon:  0.47
episode:  747  score:  -14.03  avg_score:  -34.53  frames:  197  epsilon:  0.47
episode:  748  score:  107.39  avg_score:  -32.73  frames:  999  epsilon:  0.47
episode:  749  score:  -32.65  avg_score:  -32.87  frames:  187  epsilon:  0.47
episode:  750  score:  -104.20  avg_score:  -33.40  frames:  181  epsilon:  0.47
episode:  751  score:  -7.12  avg_score:  -33.12  frames:  171  epsilon:  0.47
episode:  752  score:  -73.34  avg_score:  -33.76  frames:  138  epsilon:  0.47
episode:  753  score:  -167.86  avg_score:  -34.87  frames:  174  epsilon:  0.47
episode:  754  score:  108.33  avg_score:  -32.93  frames:  999  epsilon:  0.47
episode:  755  score:  2.89  avg_score:  -33.23  frames:  242  epsilon:  0.47
episode:  756  score:  -90.86  avg_score:  -34.05  frames:  140  epsilon:  0.47
episode:  757  score:  -14.58  avg_score:  -34.76  frames:  169  epsilon:  0.47
episode:  758  score:  84.64  avg_score:  -33.39  frames:  999  epsilon:  0.47
episode:  759  score:  -38.42  avg_score:  -34.09  frames:  188  epsilon:  0.47
episode:  760  score:  74.50  avg_score:  -32.91  frames:  999  epsilon:  0.47
episode:  761  score:  -21.81  avg_score:  -32.95  frames:  112  epsilon:  0.47
episode:  762  score:  125.31  avg_score:  -31.66  frames:  999  epsilon:  0.47
episode:  763  score:  -135.48  avg_score:  -33.18  frames:  185  epsilon:  0.47
episode:  764  score:  -194.40  avg_score:  -34.54  frames:  177  epsilon:  0.47
episode:  765  score:  9.47  avg_score:  -34.51  frames:  999  epsilon:  0.47
episode:  766  score:  45.37  avg_score:  -32.57  frames:  143  epsilon:  0.46
episode:  767  score:  -108.64  avg_score:  -32.71  frames:  284  epsilon:  0.46
episode:  768  score:  -57.89  avg_score:  -32.94  frames:  190  epsilon:  0.46
episode:  769  score:  76.95  avg_score:  -31.97  frames:  999  epsilon:  0.46
episode:  770  score:  12.16  avg_score:  -31.31  frames:  166  epsilon:  0.46
episode:  771  score:  -11.83  avg_score:  -31.38  frames:  135  epsilon:  0.46
episode:  772  score:  146.93  avg_score:  -29.64  frames:  999  epsilon:  0.46
episode:  773  score:  -20.82  avg_score:  -29.09  frames:  119  epsilon:  0.46
episode:  774  score:  -47.00  avg_score:  -29.20  frames:  205  epsilon:  0.46
episode:  775  score:  80.85  avg_score:  -28.04  frames:  999  epsilon:  0.46
episode:  776  score:  -24.13  avg_score:  -27.68  frames:  999  epsilon:  0.46
episode:  777  score:  62.84  avg_score:  -25.05  frames:  999  epsilon:  0.46
episode:  778  score:  -44.94  avg_score:  -23.89  frames:  148  epsilon:  0.46
episode:  779  score:  -76.43  avg_score:  -23.00  frames:  116  epsilon:  0.46
episode:  780  score:  -4.20  avg_score:  -22.66  frames:  222  epsilon:  0.46
episode:  781  score:  106.51  avg_score:  -20.43  frames:  999  epsilon:  0.46
episode:  782  score:  153.17  avg_score:  -18.65  frames:  999  epsilon:  0.46
episode:  783  score:  82.30  avg_score:  -15.94  frames:  999  epsilon:  0.46
episode:  784  score:  -67.93  avg_score:  -15.61  frames:  171  epsilon:  0.46
episode:  785  score:  -12.25  avg_score:  -14.98  frames:  121  epsilon:  0.46
episode:  786  score:  -15.04  avg_score:  -14.18  frames:  204  epsilon:  0.46
episode:  787  score:  -39.08  avg_score:  -14.18  frames:  163  epsilon:  0.46
episode:  788  score:  28.14  avg_score:  -13.82  frames:  166  epsilon:  0.45
episode:  789  score:  103.96  avg_score:  -13.08  frames:  999  epsilon:  0.45
episode:  790  score:  -83.16  avg_score:  -14.60  frames:  120  epsilon:  0.45
episode:  791  score:  110.56  avg_score:  -13.22  frames:  999  epsilon:  0.45
episode:  792  score:  -5.79  avg_score:  -12.62  frames:  89  epsilon:  0.45
episode:  793  score:  -37.21  avg_score:  -12.24  frames:  178  epsilon:  0.45
episode:  794  score:  -54.88  avg_score:  -11.40  frames:  114  epsilon:  0.45
episode:  795  score:  -59.62  avg_score:  -11.77  frames:  154  epsilon:  0.45
episode:  796  score:  -65.25  avg_score:  -12.46  frames:  141  epsilon:  0.45
episode:  797  score:  38.85  avg_score:  -11.48  frames:  999  epsilon:  0.45
episode:  798  score:  -16.79  avg_score:  -10.95  frames:  195  epsilon:  0.45
episode:  799  score:  16.05  avg_score:  -9.90  frames:  198  epsilon:  0.45
episode:  800  score:  -84.11  avg_score:  -10.57  frames:  712  epsilon:  0.45
episode:  801  score:  98.93  avg_score:  -7.70  frames:  999  epsilon:  0.45
episode:  802  score:  -0.76  avg_score:  -6.88  frames:  158  epsilon:  0.45
episode:  803  score:  -9.15  avg_score:  -6.94  frames:  108  epsilon:  0.45
episode:  804  score:  135.67  avg_score:  -5.54  frames:  999  epsilon:  0.45
episode:  805  score:  12.53  avg_score:  -4.93  frames:  161  epsilon:  0.45
episode:  806  score:  -93.78  avg_score:  -5.68  frames:  390  epsilon:  0.45
episode:  807  score:  -7.54  avg_score:  -5.74  frames:  139  epsilon:  0.45
episode:  808  score:  -113.72  avg_score:  -7.54  frames:  365  epsilon:  0.45
episode:  809  score:  -68.17  avg_score:  -7.86  frames:  62  epsilon:  0.45
episode:  810  score:  -16.70  avg_score:  -8.09  frames:  155  epsilon:  0.44
episode:  811  score:  -66.28  avg_score:  -8.85  frames:  151  epsilon:  0.44
episode:  812  score:  -8.56  avg_score:  -9.36  frames:  107  epsilon:  0.44
episode:  813  score:  -29.41  avg_score:  -9.84  frames:  191  epsilon:  0.44
episode:  814  score:  80.89  avg_score:  -8.89  frames:  999  epsilon:  0.44
episode:  815  score:  -148.64  avg_score:  -10.97  frames:  278  epsilon:  0.44
episode:  816  score:  126.49  avg_score:  -9.66  frames:  999  epsilon:  0.44
episode:  817  score:  -72.33  avg_score:  -10.01  frames:  148  epsilon:  0.44
episode:  818  score:  77.25  avg_score:  -8.38  frames:  999  epsilon:  0.44
episode:  819  score:  11.17  avg_score:  -8.31  frames:  139  epsilon:  0.44
episode:  820  score:  -111.39  avg_score:  -8.87  frames:  117  epsilon:  0.44
episode:  821  score:  -18.69  avg_score:  -8.88  frames:  85  epsilon:  0.44
episode:  822  score:  118.14  avg_score:  -8.13  frames:  999  epsilon:  0.44
episode:  823  score:  -135.44  avg_score:  -9.43  frames:  760  epsilon:  0.44
episode:  824  score:  83.28  avg_score:  -9.49  frames:  999  epsilon:  0.44
episode:  825  score:  69.99  avg_score:  -7.62  frames:  999  epsilon:  0.44
episode:  826  score:  8.23  avg_score:  -7.53  frames:  116  epsilon:  0.44
episode:  827  score:  8.42  avg_score:  -6.96  frames:  135  epsilon:  0.44
episode:  828  score:  12.23  avg_score:  -6.70  frames:  999  epsilon:  0.44
episode:  829  score:  70.90  avg_score:  -5.64  frames:  999  epsilon:  0.44
episode:  830  score:  -16.40  avg_score:  -6.02  frames:  146  epsilon:  0.44
episode:  831  score:  83.28  avg_score:  -4.59  frames:  999  epsilon:  0.44
episode:  832  score:  -87.09  avg_score:  -4.64  frames:  158  epsilon:  0.44
episode:  833  score:  -213.60  avg_score:  -6.21  frames:  103  epsilon:  0.43
episode:  834  score:  8.32  avg_score:  -6.36  frames:  205  epsilon:  0.43
episode:  835  score:  7.14  avg_score:  -6.60  frames:  115  epsilon:  0.43
episode:  836  score:  -59.15  avg_score:  -6.67  frames:  119  epsilon:  0.43
episode:  837  score:  36.33  avg_score:  -6.32  frames:  999  epsilon:  0.43
episode:  838  score:  -26.59  avg_score:  -6.13  frames:  207  epsilon:  0.43
episode:  839  score:  -72.19  avg_score:  -7.87  frames:  144  epsilon:  0.43
episode:  840  score:  80.77  avg_score:  -6.51  frames:  999  epsilon:  0.43
episode:  841  score:  27.16  avg_score:  -6.43  frames:  234  epsilon:  0.43
episode:  842  score:  133.57  avg_score:  -5.16  frames:  999  epsilon:  0.43
episode:  843  score:  22.29  avg_score:  -4.41  frames:  148  epsilon:  0.43
episode:  844  score:  7.93  avg_score:  -3.36  frames:  102  epsilon:  0.43
episode:  845  score:  139.61  avg_score:  -2.09  frames:  999  epsilon:  0.43
episode:  846  score:  9.77  avg_score:  -1.60  frames:  128  epsilon:  0.43
episode:  847  score:  -6.36  avg_score:  -1.52  frames:  171  epsilon:  0.43
episode:  848  score:  0.82  avg_score:  -2.59  frames:  146  epsilon:  0.43
episode:  849  score:  107.29  avg_score:  -1.19  frames:  999  epsilon:  0.43
episode:  850  score:  -44.04  avg_score:  -0.59  frames:  148  epsilon:  0.43
episode:  851  score:  39.59  avg_score:  -0.12  frames:  197  epsilon:  0.43
episode:  852  score:  43.01  avg_score:  1.04  frames:  215  epsilon:  0.43
episode:  853  score:  11.38  avg_score:  2.84  frames:  224  epsilon:  0.43
episode:  854  score:  32.52  avg_score:  2.08  frames:  156  epsilon:  0.43
episode:  855  score:  115.86  avg_score:  3.21  frames:  999  epsilon:  0.43
episode:  856  score:  -18.75  avg_score:  3.93  frames:  194  epsilon:  0.42
episode:  857  score:  133.56  avg_score:  5.41  frames:  999  epsilon:  0.42
episode:  858  score:  138.20  avg_score:  5.95  frames:  999  epsilon:  0.42
episode:  859  score:  58.18  avg_score:  6.91  frames:  999  epsilon:  0.42
episode:  860  score:  -33.13  avg_score:  5.84  frames:  218  epsilon:  0.42
episode:  861  score:  84.27  avg_score:  6.90  frames:  999  epsilon:  0.42
episode:  862  score:  -51.74  avg_score:  5.13  frames:  174  epsilon:  0.42
episode:  863  score:  122.76  avg_score:  7.71  frames:  999  epsilon:  0.42
episode:  864  score:  -37.53  avg_score:  9.28  frames:  76  epsilon:  0.42
episode:  865  score:  -53.22  avg_score:  8.65  frames:  96  epsilon:  0.42
episode:  866  score:  16.33  avg_score:  8.36  frames:  112  epsilon:  0.42
episode:  867  score:  131.36  avg_score:  10.76  frames:  999  epsilon:  0.42
episode:  868  score:  -10.73  avg_score:  11.23  frames:  176  epsilon:  0.42
episode:  869  score:  4.21  avg_score:  10.50  frames:  153  epsilon:  0.42
episode:  870  score:  40.33  avg_score:  10.79  frames:  999  epsilon:  0.42
episode:  871  score:  -17.70  avg_score:  10.73  frames:  154  epsilon:  0.42
episode:  872  score:  -48.77  avg_score:  8.77  frames:  171  epsilon:  0.42
episode:  873  score:  117.79  avg_score:  10.16  frames:  999  epsilon:  0.42
episode:  874  score:  -5.83  avg_score:  10.57  frames:  170  epsilon:  0.42
episode:  875  score:  -10.26  avg_score:  9.66  frames:  141  epsilon:  0.42
episode:  876  score:  66.43  avg_score:  10.56  frames:  999  epsilon:  0.42
episode:  877  score:  91.61  avg_score:  10.85  frames:  999  epsilon:  0.42
episode:  878  score:  -166.15  avg_score:  9.64  frames:  249  epsilon:  0.42
episode:  879  score:  10.37  avg_score:  10.51  frames:  288  epsilon:  0.42
episode:  880  score:  78.44  avg_score:  11.33  frames:  999  epsilon:  0.41
episode:  881  score:  -2.28  avg_score:  10.24  frames:  153  epsilon:  0.41
episode:  882  score:  -341.89  avg_score:  5.29  frames:  946  epsilon:  0.41
episode:  883  score:  -55.69  avg_score:  3.91  frames:  202  epsilon:  0.41
episode:  884  score:  136.59  avg_score:  5.96  frames:  999  epsilon:  0.41
episode:  885  score:  -20.20  avg_score:  5.88  frames:  256  epsilon:  0.41
episode:  886  score:  88.96  avg_score:  6.92  frames:  999  epsilon:  0.41
episode:  887  score:  102.57  avg_score:  8.34  frames:  999  epsilon:  0.41
episode:  888  score:  51.91  avg_score:  8.57  frames:  999  epsilon:  0.41
episode:  889  score:  -13.91  avg_score:  7.40  frames:  309  epsilon:  0.41
episode:  890  score:  -58.51  avg_score:  7.64  frames:  117  epsilon:  0.41
episode:  891  score:  -30.21  avg_score:  6.23  frames:  74  epsilon:  0.41
episode:  892  score:  9.85  avg_score:  6.39  frames:  90  epsilon:  0.41
episode:  893  score:  140.34  avg_score:  8.17  frames:  999  epsilon:  0.41
episode:  894  score:  92.88  avg_score:  9.64  frames:  999  epsilon:  0.41
episode:  895  score:  -55.97  avg_score:  9.68  frames:  237  epsilon:  0.41
episode:  896  score:  112.37  avg_score:  11.46  frames:  999  epsilon:  0.41
episode:  897  score:  69.31  avg_score:  11.76  frames:  999  epsilon:  0.41
episode:  898  score:  -1.53  avg_score:  11.91  frames:  140  epsilon:  0.41
episode:  899  score:  86.93  avg_score:  12.62  frames:  999  epsilon:  0.41
episode:  900  score:  8.42  avg_score:  13.55  frames:  198  epsilon:  0.41
episode:  901  score:  112.22  avg_score:  13.68  frames:  999  epsilon:  0.41
episode:  902  score:  -28.27  avg_score:  13.41  frames:  147  epsilon:  0.41
episode:  903  score:  8.48  avg_score:  13.58  frames:  193  epsilon:  0.41
episode:  904  score:  138.72  avg_score:  13.61  frames:  999  epsilon:  0.41
episode:  905  score:  134.25  avg_score:  14.83  frames:  999  epsilon:  0.40
episode:  906  score:  -40.71  avg_score:  15.36  frames:  195  epsilon:  0.40
episode:  907  score:  36.33  avg_score:  15.80  frames:  171  epsilon:  0.40
episode:  908  score:  -55.20  avg_score:  16.38  frames:  127  epsilon:  0.40
episode:  909  score:  136.94  avg_score:  18.44  frames:  999  epsilon:  0.40
episode:  910  score:  -3.05  avg_score:  18.57  frames:  150  epsilon:  0.40
episode:  911  score:  -19.69  avg_score:  19.04  frames:  167  epsilon:  0.40
episode:  912  score:  -35.49  avg_score:  18.77  frames:  130  epsilon:  0.40
episode:  913  score:  104.37  avg_score:  20.11  frames:  999  epsilon:  0.40
episode:  914  score:  22.73  avg_score:  19.52  frames:  999  epsilon:  0.40
episode:  915  score:  -48.58  avg_score:  20.52  frames:  93  epsilon:  0.40
episode:  916  score:  -15.62  avg_score:  19.10  frames:  205  epsilon:  0.40
episode:  917  score:  131.43  avg_score:  21.14  frames:  999  epsilon:  0.40
episode:  918  score:  33.95  avg_score:  20.71  frames:  115  epsilon:  0.40
episode:  919  score:  -2.42  avg_score:  20.57  frames:  139  epsilon:  0.40
episode:  920  score:  -6.64  avg_score:  21.62  frames:  123  epsilon:  0.40
episode:  921  score:  -7.79  avg_score:  21.73  frames:  232  epsilon:  0.40
episode:  922  score:  -20.33  avg_score:  20.34  frames:  98  epsilon:  0.40
episode:  923  score:  109.67  avg_score:  22.80  frames:  999  epsilon:  0.40
episode:  924  score:  -0.21  avg_score:  21.96  frames:  114  epsilon:  0.40
episode:  925  score:  -12.26  avg_score:  21.14  frames:  127  epsilon:  0.40
episode:  926  score:  -377.77  avg_score:  17.28  frames:  295  epsilon:  0.40
episode:  927  score:  3.75  avg_score:  17.23  frames:  208  epsilon:  0.40
episode:  928  score:  -6.33  avg_score:  17.05  frames:  137  epsilon:  0.40
episode:  929  score:  28.27  avg_score:  16.62  frames:  999  epsilon:  0.40
episode:  930  score:  24.62  avg_score:  17.03  frames:  146  epsilon:  0.39
episode:  931  score:  99.96  avg_score:  17.20  frames:  999  epsilon:  0.39
episode:  932  score:  -11.61  avg_score:  17.95  frames:  179  epsilon:  0.39
episode:  933  score:  -44.52  avg_score:  19.64  frames:  209  epsilon:  0.39
episode:  934  score:  -59.54  avg_score:  18.96  frames:  190  epsilon:  0.39
episode:  935  score:  32.37  avg_score:  19.22  frames:  132  epsilon:  0.39
episode:  936  score:  -31.29  avg_score:  19.49  frames:  88  epsilon:  0.39
episode:  937  score:  -20.47  avg_score:  18.93  frames:  108  epsilon:  0.39
episode:  938  score:  5.62  avg_score:  19.25  frames:  129  epsilon:  0.39
episode:  939  score:  -13.05  avg_score:  19.84  frames:  161  epsilon:  0.39
episode:  940  score:  101.46  avg_score:  20.05  frames:  999  epsilon:  0.39
episode:  941  score:  -77.88  avg_score:  19.00  frames:  117  epsilon:  0.39
episode:  942  score:  43.91  avg_score:  18.10  frames:  245  epsilon:  0.39
episode:  943  score:  -29.58  avg_score:  17.58  frames:  146  epsilon:  0.39
episode:  944  score:  -0.20  avg_score:  17.50  frames:  198  epsilon:  0.39
episode:  945  score:  47.22  avg_score:  16.58  frames:  172  epsilon:  0.39
episode:  946  score:  122.83  avg_score:  17.71  frames:  999  epsilon:  0.39
episode:  947  score:  51.30  avg_score:  18.28  frames:  999  epsilon:  0.39
episode:  948  score:  113.15  avg_score:  19.41  frames:  999  epsilon:  0.39
episode:  949  score:  -30.34  avg_score:  18.03  frames:  167  epsilon:  0.39
episode:  950  score:  -85.36  avg_score:  17.62  frames:  221  epsilon:  0.39
episode:  951  score:  144.54  avg_score:  18.67  frames:  999  epsilon:  0.39
episode:  952  score:  -139.02  avg_score:  16.85  frames:  448  epsilon:  0.39
episode:  953  score:  107.66  avg_score:  17.81  frames:  999  epsilon:  0.39
episode:  954  score:  -35.06  avg_score:  17.13  frames:  391  epsilon:  0.39
episode:  955  score:  76.63  avg_score:  16.74  frames:  999  epsilon:  0.38
episode:  956  score:  76.52  avg_score:  17.69  frames:  999  epsilon:  0.38
episode:  957  score:  5.24  avg_score:  16.41  frames:  309  epsilon:  0.38
episode:  958  score:  123.11  avg_score:  16.26  frames:  999  epsilon:  0.38
episode:  959  score:  -193.93  avg_score:  13.74  frames:  916  epsilon:  0.38
episode:  960  score:  59.38  avg_score:  14.66  frames:  999  epsilon:  0.38
episode:  961  score:  -233.32  avg_score:  11.49  frames:  316  epsilon:  0.38
episode:  962  score:  -432.75  avg_score:  7.68  frames:  562  epsilon:  0.38
episode:  963  score:  -21.75  avg_score:  6.23  frames:  999  epsilon:  0.38
episode:  964  score:  104.26  avg_score:  7.65  frames:  999  epsilon:  0.38
episode:  965  score:  -96.52  avg_score:  7.22  frames:  263  epsilon:  0.38
episode:  966  score:  139.98  avg_score:  8.45  frames:  999  epsilon:  0.38
episode:  967  score:  146.98  avg_score:  8.61  frames:  999  epsilon:  0.38
episode:  968  score:  94.36  avg_score:  9.66  frames:  999  epsilon:  0.38
episode:  969  score:  -3.30  avg_score:  9.59  frames:  125  epsilon:  0.38
episode:  970  score:  -29.77  avg_score:  8.88  frames:  111  epsilon:  0.38
episode:  971  score:  -202.35  avg_score:  7.04  frames:  248  epsilon:  0.38
episode:  972  score:  -40.13  avg_score:  7.12  frames:  86  epsilon:  0.38
episode:  973  score:  -46.80  avg_score:  5.48  frames:  999  epsilon:  0.38
episode:  974  score:  139.08  avg_score:  6.93  frames:  999  epsilon:  0.38
episode:  975  score:  117.08  avg_score:  8.20  frames:  999  epsilon:  0.38
episode:  976  score:  34.77  avg_score:  7.88  frames:  98  epsilon:  0.38
episode:  977  score:  31.92  avg_score:  7.29  frames:  999  epsilon:  0.38
episode:  978  score:  70.20  avg_score:  9.65  frames:  999  epsilon:  0.38
episode:  979  score:  -100.33  avg_score:  8.54  frames:  999  epsilon:  0.38
episode:  980  score:  -2.34  avg_score:  7.74  frames:  119  epsilon:  0.38
episode:  981  score:  10.99  avg_score:  7.87  frames:  143  epsilon:  0.37
episode:  982  score:  126.74  avg_score:  12.56  frames:  999  epsilon:  0.37
episode:  983  score:  -17.31  avg_score:  12.94  frames:  156  epsilon:  0.37
episode:  984  score:  21.43  avg_score:  11.79  frames:  130  epsilon:  0.37
episode:  985  score:  -5.88  avg_score:  11.93  frames:  162  epsilon:  0.37
episode:  986  score:  13.22  avg_score:  11.17  frames:  142  epsilon:  0.37
episode:  987  score:  -39.33  avg_score:  9.75  frames:  97  epsilon:  0.37
episode:  988  score:  -3.06  avg_score:  9.20  frames:  127  epsilon:  0.37
episode:  989  score:  0.69  avg_score:  9.35  frames:  159  epsilon:  0.37
episode:  990  score:  11.77  avg_score:  10.05  frames:  152  epsilon:  0.37
episode:  991  score:  154.55  avg_score:  11.90  frames:  999  epsilon:  0.37
episode:  992  score:  9.99  avg_score:  11.90  frames:  122  epsilon:  0.37
episode:  993  score:  125.34  avg_score:  11.75  frames:  999  epsilon:  0.37
episode:  994  score:  89.57  avg_score:  11.72  frames:  999  epsilon:  0.37
episode:  995  score:  63.16  avg_score:  12.91  frames:  999  epsilon:  0.37
episode:  996  score:  0.59  avg_score:  11.79  frames:  999  epsilon:  0.37
episode:  997  score:  86.66  avg_score:  11.97  frames:  999  epsilon:  0.37
episode:  998  score:  3.26  avg_score:  12.01  frames:  134  epsilon:  0.37
episode:  999  score:  121.86  avg_score:  12.36  frames:  999  epsilon:  0.37
episode:  1000  score:  120.62  avg_score:  13.49  frames:  999  epsilon:  0.37
episode:  1001  score:  -39.48  avg_score:  11.97  frames:  156  epsilon:  0.37
episode:  1002  score:  112.72  avg_score:  13.38  frames:  999  epsilon:  0.37
episode:  1003  score:  100.91  avg_score:  14.30  frames:  999  epsilon:  0.37
episode:  1004  score:  7.03  avg_score:  12.99  frames:  127  epsilon:  0.37
episode:  1005  score:  96.39  avg_score:  12.61  frames:  999  epsilon:  0.37
episode:  1006  score:  -215.79  avg_score:  10.86  frames:  188  epsilon:  0.37
episode:  1007  score:  151.96  avg_score:  12.01  frames:  881  epsilon:  0.37
episode:  1008  score:  -16.65  avg_score:  12.40  frames:  124  epsilon:  0.37
episode:  1009  score:  140.66  avg_score:  12.44  frames:  999  epsilon:  0.36
episode:  1010  score:  84.83  avg_score:  13.31  frames:  999  epsilon:  0.36
episode:  1011  score:  23.03  avg_score:  13.74  frames:  139  epsilon:  0.36
episode:  1012  score:  120.98  avg_score:  15.31  frames:  999  epsilon:  0.36
episode:  1013  score:  71.03  avg_score:  14.97  frames:  999  epsilon:  0.36
episode:  1014  score:  106.17  avg_score:  15.81  frames:  999  epsilon:  0.36
episode:  1015  score:  -76.23  avg_score:  15.53  frames:  72  epsilon:  0.36
episode:  1016  score:  82.02  avg_score:  16.51  frames:  999  epsilon:  0.36
episode:  1017  score:  99.05  avg_score:  16.18  frames:  999  epsilon:  0.36
episode:  1018  score:  105.27  avg_score:  16.90  frames:  999  epsilon:  0.36
episode:  1019  score:  42.31  avg_score:  17.34  frames:  999  epsilon:  0.36
episode:  1020  score:  87.46  avg_score:  18.28  frames:  999  epsilon:  0.36
episode:  1021  score:  -48.19  avg_score:  17.88  frames:  138  epsilon:  0.36
episode:  1022  score:  255.67  avg_score:  20.64  frames:  296  epsilon:  0.36
episode:  1023  score:  13.97  avg_score:  19.68  frames:  115  epsilon:  0.36
episode:  1024  score:  11.45  avg_score:  19.80  frames:  999  epsilon:  0.36
episode:  1025  score:  113.81  avg_score:  21.06  frames:  999  epsilon:  0.36
episode:  1026  score:  103.08  avg_score:  25.87  frames:  999  epsilon:  0.36
episode:  1027  score:  110.12  avg_score:  26.93  frames:  999  epsilon:  0.36
episode:  1028  score:  93.59  avg_score:  27.93  frames:  999  epsilon:  0.36
episode:  1029  score:  144.03  avg_score:  29.09  frames:  956  epsilon:  0.36
episode:  1030  score:  95.18  avg_score:  29.80  frames:  999  epsilon:  0.36
episode:  1031  score:  -132.87  avg_score:  27.47  frames:  999  epsilon:  0.36
episode:  1032  score:  -7.45  avg_score:  27.51  frames:  144  epsilon:  0.36
episode:  1033  score:  86.42  avg_score:  28.82  frames:  999  epsilon:  0.36
episode:  1034  score:  -75.14  avg_score:  28.66  frames:  125  epsilon:  0.36
episode:  1035  score:  -244.63  avg_score:  25.89  frames:  308  epsilon:  0.36
episode:  1036  score:  82.22  avg_score:  27.03  frames:  999  epsilon:  0.35
episode:  1037  score:  -280.53  avg_score:  24.43  frames:  650  epsilon:  0.35
episode:  1038  score:  -243.89  avg_score:  21.93  frames:  388  epsilon:  0.35
episode:  1039  score:  -1.17  avg_score:  22.05  frames:  151  epsilon:  0.35
episode:  1040  score:  49.69  avg_score:  21.53  frames:  999  epsilon:  0.35
episode:  1041  score:  115.45  avg_score:  23.47  frames:  999  epsilon:  0.35
episode:  1042  score:  -8.38  avg_score:  22.94  frames:  114  epsilon:  0.35
episode:  1043  score:  84.97  avg_score:  24.09  frames:  999  epsilon:  0.35
episode:  1044  score:  -141.94  avg_score:  22.67  frames:  148  epsilon:  0.35
episode:  1045  score:  99.90  avg_score:  23.20  frames:  999  epsilon:  0.35
episode:  1046  score:  -88.99  avg_score:  21.08  frames:  224  epsilon:  0.35
episode:  1047  score:  107.89  avg_score:  21.65  frames:  999  epsilon:  0.35
episode:  1048  score:  10.73  avg_score:  20.62  frames:  144  epsilon:  0.35
episode:  1049  score:  -32.76  avg_score:  20.60  frames:  97  epsilon:  0.35
episode:  1050  score:  32.51  avg_score:  21.78  frames:  166  epsilon:  0.35
episode:  1051  score:  107.85  avg_score:  21.41  frames:  999  epsilon:  0.35
episode:  1052  score:  -34.47  avg_score:  22.45  frames:  119  epsilon:  0.35
episode:  1053  score:  82.57  avg_score:  22.20  frames:  999  epsilon:  0.35
episode:  1054  score:  124.84  avg_score:  23.80  frames:  999  epsilon:  0.35
episode:  1055  score:  3.11  avg_score:  23.07  frames:  158  epsilon:  0.35
episode:  1056  score:  103.43  avg_score:  23.34  frames:  999  epsilon:  0.35
episode:  1057  score:  126.18  avg_score:  24.55  frames:  999  epsilon:  0.35
episode:  1058  score:  -52.38  avg_score:  22.79  frames:  130  epsilon:  0.35
episode:  1059  score:  123.75  avg_score:  25.97  frames:  999  epsilon:  0.35
episode:  1060  score:  111.79  avg_score:  26.49  frames:  999  epsilon:  0.35
episode:  1061  score:  112.02  avg_score:  29.95  frames:  999  epsilon:  0.35
episode:  1062  score:  122.57  avg_score:  35.50  frames:  999  epsilon:  0.35
episode:  1063  score:  118.45  avg_score:  36.90  frames:  999  epsilon:  0.35
episode:  1064  score:  41.19  avg_score:  36.27  frames:  999  epsilon:  0.35
episode:  1065  score:  137.97  avg_score:  38.61  frames:  999  epsilon:  0.34
episode:  1066  score:  -6.56  avg_score:  37.15  frames:  113  epsilon:  0.34
episode:  1067  score:  -20.85  avg_score:  35.47  frames:  107  epsilon:  0.34
episode:  1068  score:  -25.04  avg_score:  34.28  frames:  149  epsilon:  0.34
episode:  1069  score:  121.85  avg_score:  35.53  frames:  999  epsilon:  0.34
episode:  1070  score:  155.90  avg_score:  37.39  frames:  826  epsilon:  0.34
episode:  1071  score:  114.64  avg_score:  40.56  frames:  999  epsilon:  0.34
episode:  1072  score:  2.86  avg_score:  40.99  frames:  122  epsilon:  0.34
episode:  1073  score:  48.14  avg_score:  41.93  frames:  156  epsilon:  0.34
episode:  1074  score:  23.23  avg_score:  40.78  frames:  137  epsilon:  0.34
episode:  1075  score:  10.95  avg_score:  39.71  frames:  123  epsilon:  0.34
episode:  1076  score:  -10.78  avg_score:  39.26  frames:  81  epsilon:  0.34
episode:  1077  score:  115.76  avg_score:  40.10  frames:  999  epsilon:  0.34
episode:  1078  score:  -9.71  avg_score:  39.30  frames:  128  epsilon:  0.34
episode:  1079  score:  117.64  avg_score:  41.48  frames:  999  epsilon:  0.34
episode:  1080  score:  -125.60  avg_score:  40.25  frames:  226  epsilon:  0.34
episode:  1081  score:  -7.75  avg_score:  40.06  frames:  146  epsilon:  0.34
episode:  1082  score:  108.12  avg_score:  39.87  frames:  999  epsilon:  0.34
episode:  1083  score:  92.10  avg_score:  40.97  frames:  999  epsilon:  0.34
episode:  1084  score:  81.43  avg_score:  41.57  frames:  999  epsilon:  0.34
episode:  1085  score:  52.56  avg_score:  42.15  frames:  999  epsilon:  0.34
episode:  1086  score:  -16.21  avg_score:  41.86  frames:  149  epsilon:  0.34
episode:  1087  score:  20.49  avg_score:  42.45  frames:  141  epsilon:  0.34
episode:  1088  score:  86.88  avg_score:  43.35  frames:  999  epsilon:  0.34
episode:  1089  score:  -113.58  avg_score:  42.21  frames:  180  epsilon:  0.34
episode:  1090  score:  -100.30  avg_score:  41.09  frames:  196  epsilon:  0.34
episode:  1091  score:  -38.00  avg_score:  39.16  frames:  103  epsilon:  0.34
episode:  1092  score:  82.67  avg_score:  39.89  frames:  999  epsilon:  0.34
episode:  1093  score:  -64.40  avg_score:  37.99  frames:  259  epsilon:  0.34
episode:  1094  score:  91.17  avg_score:  38.01  frames:  999  epsilon:  0.33
episode:  1095  score:  95.50  avg_score:  38.33  frames:  999  epsilon:  0.33
episode:  1096  score:  92.13  avg_score:  39.25  frames:  999  epsilon:  0.33
episode:  1097  score:  117.43  avg_score:  39.56  frames:  999  epsilon:  0.33
episode:  1098  score:  -63.96  avg_score:  38.88  frames:  97  epsilon:  0.33
episode:  1099  score:  -163.86  avg_score:  36.03  frames:  255  epsilon:  0.33
episode:  1100  score:  -19.13  avg_score:  34.63  frames:  109  epsilon:  0.33
episode:  1101  score:  -180.99  avg_score:  33.21  frames:  740  epsilon:  0.33
episode:  1102  score:  -0.74  avg_score:  32.08  frames:  999  epsilon:  0.33
episode:  1103  score:  -23.65  avg_score:  30.83  frames:  92  epsilon:  0.33
episode:  1104  score:  99.80  avg_score:  31.76  frames:  999  epsilon:  0.33
episode:  1105  score:  198.08  avg_score:  32.78  frames:  398  epsilon:  0.33
episode:  1106  score:  -153.84  avg_score:  33.40  frames:  363  epsilon:  0.33
episode:  1107  score:  74.48  avg_score:  32.62  frames:  999  epsilon:  0.33
episode:  1108  score:  -38.58  avg_score:  32.40  frames:  174  epsilon:  0.33
episode:  1109  score:  121.13  avg_score:  32.21  frames:  999  epsilon:  0.33
episode:  1110  score:  36.86  avg_score:  31.73  frames:  999  epsilon:  0.33
episode:  1111  score:  19.83  avg_score:  31.70  frames:  138  epsilon:  0.33
episode:  1112  score:  119.78  avg_score:  31.69  frames:  999  epsilon:  0.33
episode:  1113  score:  104.52  avg_score:  32.02  frames:  999  epsilon:  0.33
episode:  1114  score:  -327.97  avg_score:  27.68  frames:  900  epsilon:  0.33
episode:  1115  score:  -145.16  avg_score:  26.99  frames:  213  epsilon:  0.33
episode:  1116  score:  -2.55  avg_score:  26.14  frames:  139  epsilon:  0.33
episode:  1117  score:  -80.04  avg_score:  24.35  frames:  999  epsilon:  0.33
episode:  1118  score:  152.60  avg_score:  24.83  frames:  999  epsilon:  0.33
episode:  1119  score:  51.65  avg_score:  24.92  frames:  999  epsilon:  0.33
episode:  1120  score:  124.54  avg_score:  25.29  frames:  999  epsilon:  0.33
episode:  1121  score:  -6.08  avg_score:  25.71  frames:  84  epsilon:  0.33
episode:  1122  score:  102.02  avg_score:  24.18  frames:  999  epsilon:  0.33
episode:  1123  score:  96.37  avg_score:  25.00  frames:  999  epsilon:  0.33
episode:  1124  score:  -267.84  avg_score:  22.21  frames:  228  epsilon:  0.33
episode:  1125  score:  162.99  avg_score:  22.70  frames:  999  epsilon:  0.32
episode:  1126  score:  110.61  avg_score:  22.77  frames:  999  epsilon:  0.32
episode:  1127  score:  100.78  avg_score:  22.68  frames:  999  epsilon:  0.32
episode:  1128  score:  118.28  avg_score:  22.93  frames:  999  epsilon:  0.32
episode:  1129  score:  -54.44  avg_score:  20.94  frames:  116  epsilon:  0.32
episode:  1130  score:  -191.50  avg_score:  18.08  frames:  784  epsilon:  0.32
episode:  1131  score:  -334.20  avg_score:  16.06  frames:  683  epsilon:  0.32
episode:  1132  score:  -7.09  avg_score:  16.07  frames:  137  epsilon:  0.32
episode:  1133  score:  149.20  avg_score:  16.69  frames:  999  epsilon:  0.32
episode:  1134  score:  84.20  avg_score:  18.29  frames:  999  epsilon:  0.32
episode:  1135  score:  6.63  avg_score:  20.80  frames:  149  epsilon:  0.32
episode:  1136  score:  54.07  avg_score:  20.52  frames:  999  epsilon:  0.32
episode:  1137  score:  38.60  avg_score:  23.71  frames:  112  epsilon:  0.32
episode:  1138  score:  21.82  avg_score:  26.37  frames:  999  epsilon:  0.32
episode:  1139  score:  -117.22  avg_score:  25.21  frames:  89  epsilon:  0.32
episode:  1140  score:  111.30  avg_score:  25.82  frames:  999  epsilon:  0.32
episode:  1141  score:  35.26  avg_score:  25.02  frames:  999  epsilon:  0.32
episode:  1142  score:  10.05  avg_score:  25.20  frames:  125  epsilon:  0.32
episode:  1143  score:  32.81  avg_score:  24.68  frames:  999  epsilon:  0.32
episode:  1144  score:  122.49  avg_score:  27.33  frames:  999  epsilon:  0.32
episode:  1145  score:  107.65  avg_score:  27.40  frames:  999  epsilon:  0.32
episode:  1146  score:  87.98  avg_score:  29.17  frames:  999  epsilon:  0.32
episode:  1147  score:  -13.51  avg_score:  27.96  frames:  148  epsilon:  0.32
episode:  1148  score:  42.44  avg_score:  28.28  frames:  999  epsilon:  0.32
episode:  1149  score:  -75.82  avg_score:  27.85  frames:  999  epsilon:  0.32
episode:  1150  score:  24.04  avg_score:  27.76  frames:  999  epsilon:  0.32
episode:  1151  score:  13.50  avg_score:  26.82  frames:  999  epsilon:  0.32
episode:  1152  score:  246.30  avg_score:  29.63  frames:  794  epsilon:  0.32
episode:  1153  score:  106.98  avg_score:  29.87  frames:  999  epsilon:  0.32
episode:  1154  score:  185.14  avg_score:  30.47  frames:  947  epsilon:  0.32
episode:  1155  score:  -70.17  avg_score:  29.74  frames:  73  epsilon:  0.32
episode:  1156  score:  -11.97  avg_score:  28.59  frames:  104  epsilon:  0.31
episode:  1157  score:  127.32  avg_score:  28.60  frames:  999  epsilon:  0.31
episode:  1158  score:  71.59  avg_score:  29.84  frames:  999  epsilon:  0.31
episode:  1159  score:  258.12  avg_score:  31.18  frames:  469  epsilon:  0.31
episode:  1160  score:  87.01  avg_score:  30.93  frames:  999  epsilon:  0.31
episode:  1161  score:  57.47  avg_score:  30.39  frames:  999  epsilon:  0.31
episode:  1162  score:  41.44  avg_score:  29.58  frames:  103  epsilon:  0.31
episode:  1163  score:  117.40  avg_score:  29.57  frames:  999  epsilon:  0.31
episode:  1164  score:  35.58  avg_score:  29.51  frames:  999  epsilon:  0.31
episode:  1165  score:  -40.37  avg_score:  27.73  frames:  110  epsilon:  0.31
episode:  1166  score:  232.35  avg_score:  30.12  frames:  751  epsilon:  0.31
episode:  1167  score:  89.16  avg_score:  31.22  frames:  999  epsilon:  0.31
episode:  1168  score:  24.17  avg_score:  31.71  frames:  999  epsilon:  0.31
episode:  1169  score:  62.87  avg_score:  31.12  frames:  999  epsilon:  0.31
episode:  1170  score:  -118.02  avg_score:  28.38  frames:  174  epsilon:  0.31
episode:  1171  score:  -12.56  avg_score:  27.11  frames:  202  epsilon:  0.31
episode:  1172  score:  6.48  avg_score:  27.14  frames:  151  epsilon:  0.31
episode:  1173  score:  79.06  avg_score:  27.45  frames:  999  epsilon:  0.31
episode:  1174  score:  -7.92  avg_score:  27.14  frames:  138  epsilon:  0.31
episode:  1175  score:  86.32  avg_score:  27.89  frames:  999  epsilon:  0.31
episode:  1176  score:  -13.43  avg_score:  27.87  frames:  120  epsilon:  0.31
episode:  1177  score:  96.57  avg_score:  27.68  frames:  999  epsilon:  0.31
episode:  1178  score:  120.29  avg_score:  28.98  frames:  999  epsilon:  0.31
episode:  1179  score:  -142.90  avg_score:  26.37  frames:  999  epsilon:  0.31
episode:  1180  score:  -44.15  avg_score:  27.19  frames:  120  epsilon:  0.31
episode:  1181  score:  -75.22  avg_score:  26.51  frames:  154  epsilon:  0.31
episode:  1182  score:  -7.37  avg_score:  25.36  frames:  127  epsilon:  0.31
episode:  1183  score:  -45.90  avg_score:  23.98  frames:  141  epsilon:  0.31
episode:  1184  score:  44.17  avg_score:  23.60  frames:  105  epsilon:  0.31
episode:  1185  score:  101.37  avg_score:  24.09  frames:  999  epsilon:  0.31
episode:  1186  score:  -2.45  avg_score:  24.23  frames:  99  epsilon:  0.31
episode:  1187  score:  -42.31  avg_score:  23.60  frames:  89  epsilon:  0.31
episode:  1188  score:  25.78  avg_score:  22.99  frames:  999  epsilon:  0.30
episode:  1189  score:  -278.07  avg_score:  21.34  frames:  555  epsilon:  0.30
episode:  1190  score:  -7.56  avg_score:  22.27  frames:  999  epsilon:  0.30
episode:  1191  score:  -21.28  avg_score:  22.44  frames:  999  epsilon:  0.30
episode:  1192  score:  -54.15  avg_score:  21.07  frames:  183  epsilon:  0.30
episode:  1193  score:  79.49  avg_score:  22.51  frames:  999  epsilon:  0.30
episode:  1194  score:  -357.04  avg_score:  18.03  frames:  612  epsilon:  0.30
episode:  1195  score:  15.50  avg_score:  17.23  frames:  999  epsilon:  0.30
episode:  1196  score:  -125.20  avg_score:  15.05  frames:  510  epsilon:  0.30
episode:  1197  score:  -57.93  avg_score:  13.30  frames:  207  epsilon:  0.30
episode:  1198  score:  -170.00  avg_score:  12.24  frames:  733  epsilon:  0.30
episode:  1199  score:  -64.33  avg_score:  13.24  frames:  999  epsilon:  0.30
episode:  1200  score:  133.62  avg_score:  14.76  frames:  999  epsilon:  0.30
episode:  1201  score:  36.65  avg_score:  16.94  frames:  999  epsilon:  0.30
episode:  1202  score:  26.87  avg_score:  17.22  frames:  999  epsilon:  0.30
episode:  1203  score:  82.02  avg_score:  18.27  frames:  999  epsilon:  0.30
episode:  1204  score:  80.45  avg_score:  18.08  frames:  999  epsilon:  0.30
episode:  1205  score:  89.60  avg_score:  16.99  frames:  999  epsilon:  0.30
episode:  1206  score:  87.08  avg_score:  19.40  frames:  999  epsilon:  0.30
episode:  1207  score:  84.05  avg_score:  19.50  frames:  999  epsilon:  0.30
episode:  1208  score:  -93.96  avg_score:  18.95  frames:  427  epsilon:  0.30
episode:  1209  score:  95.59  avg_score:  18.69  frames:  999  epsilon:  0.30
episode:  1210  score:  66.71  avg_score:  18.99  frames:  999  epsilon:  0.30
episode:  1211  score:  173.32  avg_score:  20.52  frames:  463  epsilon:  0.30
episode:  1212  score:  210.02  avg_score:  21.43  frames:  664  epsilon:  0.30
episode:  1213  score:  68.86  avg_score:  21.07  frames:  999  epsilon:  0.30
episode:  1214  score:  30.12  avg_score:  24.65  frames:  999  epsilon:  0.30
episode:  1215  score:  -13.61  avg_score:  25.97  frames:  113  epsilon:  0.30
episode:  1216  score:  162.93  avg_score:  27.62  frames:  988  epsilon:  0.30
episode:  1217  score:  70.81  avg_score:  29.13  frames:  999  epsilon:  0.30
episode:  1218  score:  195.12  avg_score:  29.55  frames:  571  epsilon:  0.30
episode:  1219  score:  49.47  avg_score:  29.53  frames:  999  epsilon:  0.30
episode:  1220  score:  -13.84  avg_score:  28.15  frames:  177  epsilon:  0.30
episode:  1221  score:  3.73  avg_score:  28.25  frames:  96  epsilon:  0.30
episode:  1222  score:  13.00  avg_score:  27.36  frames:  195  epsilon:  0.29
episode:  1223  score:  132.07  avg_score:  27.71  frames:  999  epsilon:  0.29
episode:  1224  score:  57.65  avg_score:  30.97  frames:  161  epsilon:  0.29
episode:  1225  score:  82.34  avg_score:  30.16  frames:  999  epsilon:  0.29
episode:  1226  score:  166.01  avg_score:  30.72  frames:  985  epsilon:  0.29
episode:  1227  score:  -41.63  avg_score:  29.29  frames:  167  epsilon:  0.29
episode:  1228  score:  36.88  avg_score:  28.48  frames:  999  epsilon:  0.29
episode:  1229  score:  -136.99  avg_score:  27.65  frames:  86  epsilon:  0.29
episode:  1230  score:  -74.58  avg_score:  28.82  frames:  431  epsilon:  0.29
episode:  1231  score:  125.39  avg_score:  33.42  frames:  999  epsilon:  0.29
episode:  1232  score:  -117.43  avg_score:  32.31  frames:  95  epsilon:  0.29
episode:  1233  score:  -141.83  avg_score:  29.40  frames:  84  epsilon:  0.29
episode:  1234  score:  6.65  avg_score:  28.63  frames:  999  epsilon:  0.29
episode:  1235  score:  65.51  avg_score:  29.22  frames:  999  epsilon:  0.29
episode:  1236  score:  103.48  avg_score:  29.71  frames:  999  epsilon:  0.29
episode:  1237  score:  79.43  avg_score:  30.12  frames:  999  epsilon:  0.29
episode:  1238  score:  92.80  avg_score:  30.83  frames:  999  epsilon:  0.29
episode:  1239  score:  44.63  avg_score:  32.45  frames:  999  epsilon:  0.29
episode:  1240  score:  31.08  avg_score:  31.65  frames:  999  epsilon:  0.29
episode:  1241  score:  57.08  avg_score:  31.86  frames:  999  epsilon:  0.29
episode:  1242  score:  -19.30  avg_score:  31.57  frames:  999  epsilon:  0.29
episode:  1243  score:  -244.10  avg_score:  28.80  frames:  957  epsilon:  0.29
episode:  1244  score:  89.34  avg_score:  28.47  frames:  999  epsilon:  0.29
episode:  1245  score:  17.06  avg_score:  27.56  frames:  136  epsilon:  0.29
episode:  1246  score:  129.69  avg_score:  27.98  frames:  999  epsilon:  0.29
episode:  1247  score:  88.23  avg_score:  29.00  frames:  999  epsilon:  0.29
episode:  1248  score:  11.64  avg_score:  28.69  frames:  138  epsilon:  0.29
episode:  1249  score:  -98.45  avg_score:  28.46  frames:  396  epsilon:  0.29
episode:  1250  score:  -283.97  avg_score:  25.38  frames:  218  epsilon:  0.29
episode:  1251  score:  -4.66  avg_score:  25.20  frames:  667  epsilon:  0.29
episode:  1252  score:  93.18  avg_score:  23.67  frames:  999  epsilon:  0.29
episode:  1253  score:  114.63  avg_score:  23.75  frames:  999  epsilon:  0.29
episode:  1254  score:  263.03  avg_score:  24.53  frames:  223  epsilon:  0.29
episode:  1255  score:  258.45  avg_score:  27.81  frames:  621  epsilon:  0.29
episode:  1256  score:  188.67  avg_score:  29.82  frames:  711  epsilon:  0.28
episode:  1257  score:  103.88  avg_score:  29.58  frames:  999  epsilon:  0.28
episode:  1258  score:  -32.00  avg_score:  28.55  frames:  119  epsilon:  0.28
episode:  1259  score:  152.21  avg_score:  27.49  frames:  999  epsilon:  0.28
episode:  1260  score:  -3.44  avg_score:  26.58  frames:  120  epsilon:  0.28
episode:  1261  score:  111.34  avg_score:  27.12  frames:  999  epsilon:  0.28
episode:  1262  score:  16.86  avg_score:  26.88  frames:  165  epsilon:  0.28
episode:  1263  score:  212.63  avg_score:  27.83  frames:  453  epsilon:  0.28
episode:  1264  score:  -10.02  avg_score:  27.37  frames:  112  epsilon:  0.28
episode:  1265  score:  27.52  avg_score:  28.05  frames:  999  epsilon:  0.28
episode:  1266  score:  79.54  avg_score:  26.52  frames:  999  epsilon:  0.28
episode:  1267  score:  49.36  avg_score:  26.13  frames:  999  epsilon:  0.28
episode:  1268  score:  278.67  avg_score:  28.67  frames:  263  epsilon:  0.28
episode:  1269  score:  -6.92  avg_score:  27.97  frames:  118  epsilon:  0.28
episode:  1270  score:  68.72  avg_score:  29.84  frames:  999  epsilon:  0.28
episode:  1271  score:  5.06  avg_score:  30.02  frames:  127  epsilon:  0.28
episode:  1272  score:  75.82  avg_score:  30.71  frames:  999  epsilon:  0.28
episode:  1273  score:  109.40  avg_score:  31.01  frames:  999  epsilon:  0.28
episode:  1274  score:  -5.39  avg_score:  31.04  frames:  100  epsilon:  0.28
episode:  1275  score:  73.04  avg_score:  30.91  frames:  999  epsilon:  0.28
episode:  1276  score:  88.29  avg_score:  31.92  frames:  999  epsilon:  0.28
episode:  1277  score:  94.60  avg_score:  31.90  frames:  999  epsilon:  0.28
episode:  1278  score:  118.07  avg_score:  31.88  frames:  999  epsilon:  0.28
episode:  1279  score:  11.79  avg_score:  33.43  frames:  97  epsilon:  0.28
episode:  1280  score:  142.49  avg_score:  35.30  frames:  999  epsilon:  0.28
episode:  1281  score:  78.51  avg_score:  36.83  frames:  999  epsilon:  0.28
episode:  1282  score:  178.88  avg_score:  38.70  frames:  616  epsilon:  0.28
episode:  1283  score:  26.94  avg_score:  39.42  frames:  116  epsilon:  0.28
episode:  1284  score:  17.35  avg_score:  39.16  frames:  92  epsilon:  0.28
episode:  1285  score:  135.60  avg_score:  39.50  frames:  999  epsilon:  0.28
episode:  1286  score:  -34.62  avg_score:  39.18  frames:  199  epsilon:  0.28
episode:  1287  score:  78.34  avg_score:  40.38  frames:  999  epsilon:  0.28
episode:  1288  score:  73.79  avg_score:  40.86  frames:  999  epsilon:  0.28
episode:  1289  score:  50.37  avg_score:  44.15  frames:  999  epsilon:  0.28
episode:  1290  score:  0.81  avg_score:  44.23  frames:  114  epsilon:  0.28
episode:  1291  score:  130.75  avg_score:  45.75  frames:  999  epsilon:  0.28
episode:  1292  score:  130.27  avg_score:  47.60  frames:  999  epsilon:  0.27
episode:  1293  score:  86.81  avg_score:  47.67  frames:  999  epsilon:  0.27
episode:  1294  score:  93.46  avg_score:  52.17  frames:  999  epsilon:  0.27
episode:  1295  score:  -17.16  avg_score:  51.85  frames:  110  epsilon:  0.27
episode:  1296  score:  221.97  avg_score:  55.32  frames:  458  epsilon:  0.27
episode:  1297  score:  120.05  avg_score:  57.10  frames:  999  epsilon:  0.27
episode:  1298  score:  -183.41  avg_score:  56.96  frames:  273  epsilon:  0.27
episode:  1299  score:  43.62  avg_score:  58.04  frames:  999  epsilon:  0.27
episode:  1300  score:  117.19  avg_score:  57.88  frames:  999  epsilon:  0.27
episode:  1301  score:  253.06  avg_score:  60.04  frames:  587  epsilon:  0.27
episode:  1302  score:  -220.60  avg_score:  57.57  frames:  311  epsilon:  0.27
episode:  1303  score:  70.14  avg_score:  57.45  frames:  999  epsilon:  0.27
episode:  1304  score:  29.47  avg_score:  56.94  frames:  117  epsilon:  0.27
episode:  1305  score:  -10.77  avg_score:  55.94  frames:  196  epsilon:  0.27
episode:  1306  score:  79.95  avg_score:  55.87  frames:  999  epsilon:  0.27
episode:  1307  score:  40.99  avg_score:  55.43  frames:  999  epsilon:  0.27
episode:  1308  score:  122.49  avg_score:  57.60  frames:  999  epsilon:  0.27
episode:  1309  score:  -142.90  avg_score:  55.21  frames:  504  epsilon:  0.27
episode:  1310  score:  206.86  avg_score:  56.62  frames:  649  epsilon:  0.27
episode:  1311  score:  89.98  avg_score:  55.78  frames:  999  epsilon:  0.27
episode:  1312  score:  260.43  avg_score:  56.29  frames:  495  epsilon:  0.27
episode:  1313  score:  -262.49  avg_score:  52.97  frames:  385  epsilon:  0.27
episode:  1314  score:  236.47  avg_score:  55.04  frames:  377  epsilon:  0.27
episode:  1315  score:  9.38  avg_score:  55.27  frames:  201  epsilon:  0.27
episode:  1316  score:  63.54  avg_score:  54.27  frames:  999  epsilon:  0.27
episode:  1317  score:  107.93  avg_score:  54.64  frames:  999  epsilon:  0.27
episode:  1318  score:  -407.22  avg_score:  48.62  frames:  241  epsilon:  0.27
episode:  1319  score:  121.40  avg_score:  49.34  frames:  999  epsilon:  0.27
episode:  1320  score:  230.32  avg_score:  51.78  frames:  329  epsilon:  0.27
episode:  1321  score:  114.06  avg_score:  52.88  frames:  999  epsilon:  0.27
episode:  1322  score:  115.14  avg_score:  53.91  frames:  999  epsilon:  0.27
episode:  1323  score:  204.95  avg_score:  54.63  frames:  822  epsilon:  0.27
episode:  1324  score:  68.81  avg_score:  54.75  frames:  999  epsilon:  0.27
episode:  1325  score:  -114.40  avg_score:  52.78  frames:  361  epsilon:  0.27
episode:  1326  score:  42.03  avg_score:  51.54  frames:  176  epsilon:  0.27
episode:  1327  score:  102.45  avg_score:  52.98  frames:  999  epsilon:  0.27
episode:  1328  score:  15.42  avg_score:  52.77  frames:  83  epsilon:  0.27
episode:  1329  score:  210.37  avg_score:  56.24  frames:  791  epsilon:  0.26
episode:  1330  score:  37.65  avg_score:  57.36  frames:  999  epsilon:  0.26
episode:  1331  score:  28.15  avg_score:  56.39  frames:  141  epsilon:  0.26
episode:  1332  score:  15.73  avg_score:  57.72  frames:  123  epsilon:  0.26
episode:  1333  score:  119.86  avg_score:  60.34  frames:  999  epsilon:  0.26
episode:  1334  score:  157.80  avg_score:  61.85  frames:  999  epsilon:  0.26
episode:  1335  score:  -15.51  avg_score:  61.04  frames:  136  epsilon:  0.26
episode:  1336  score:  74.61  avg_score:  60.75  frames:  999  epsilon:  0.26
episode:  1337  score:  111.73  avg_score:  61.07  frames:  999  epsilon:  0.26
episode:  1338  score:  268.45  avg_score:  62.83  frames:  431  epsilon:  0.26
episode:  1339  score:  143.67  avg_score:  63.82  frames:  999  epsilon:  0.26
episode:  1340  score:  131.88  avg_score:  64.83  frames:  999  epsilon:  0.26
episode:  1341  score:  53.37  avg_score:  64.79  frames:  999  epsilon:  0.26
episode:  1342  score:  140.33  avg_score:  66.39  frames:  999  epsilon:  0.26
episode:  1343  score:  94.75  avg_score:  69.78  frames:  999  epsilon:  0.26
episode:  1344  score:  124.14  avg_score:  70.12  frames:  999  epsilon:  0.26
episode:  1345  score:  107.78  avg_score:  71.03  frames:  999  epsilon:  0.26
episode:  1346  score:  279.35  avg_score:  72.53  frames:  244  epsilon:  0.26
episode:  1347  score:  157.18  avg_score:  73.22  frames:  859  epsilon:  0.26
episode:  1348  score:  -1.69  avg_score:  73.08  frames:  140  epsilon:  0.26
episode:  1349  score:  77.55  avg_score:  74.84  frames:  999  epsilon:  0.26
episode:  1350  score:  -23.53  avg_score:  77.45  frames:  999  epsilon:  0.26
episode:  1351  score:  14.25  avg_score:  77.64  frames:  179  epsilon:  0.26
episode:  1352  score:  -186.91  avg_score:  74.84  frames:  460  epsilon:  0.26
episode:  1353  score:  229.49  avg_score:  75.99  frames:  889  epsilon:  0.26
episode:  1354  score:  104.83  avg_score:  74.40  frames:  999  epsilon:  0.26
episode:  1355  score:  236.58  avg_score:  74.18  frames:  654  epsilon:  0.26
episode:  1356  score:  59.91  avg_score:  72.90  frames:  999  epsilon:  0.26
episode:  1357  score:  61.39  avg_score:  72.47  frames:  999  epsilon:  0.26
episode:  1358  score:  -50.94  avg_score:  72.28  frames:  105  epsilon:  0.26
episode:  1359  score:  111.29  avg_score:  71.87  frames:  999  epsilon:  0.26
episode:  1360  score:  12.39  avg_score:  72.03  frames:  110  epsilon:  0.26
episode:  1361  score:  -19.66  avg_score:  70.72  frames:  108  epsilon:  0.26
episode:  1362  score:  237.71  avg_score:  72.93  frames:  882  epsilon:  0.26
episode:  1363  score:  0.83  avg_score:  70.81  frames:  163  epsilon:  0.26
episode:  1364  score:  235.28  avg_score:  73.27  frames:  694  epsilon:  0.26
episode:  1365  score:  233.96  avg_score:  75.33  frames:  898  epsilon:  0.26
episode:  1366  score:  -16.21  avg_score:  74.37  frames:  134  epsilon:  0.26
episode:  1367  score:  253.61  avg_score:  76.41  frames:  283  epsilon:  0.25
episode:  1368  score:  132.85  avg_score:  74.96  frames:  999  epsilon:  0.25
episode:  1369  score:  100.29  avg_score:  76.03  frames:  999  epsilon:  0.25
episode:  1370  score:  96.99  avg_score:  76.31  frames:  999  epsilon:  0.25
episode:  1371  score:  -0.67  avg_score:  76.25  frames:  104  epsilon:  0.25
episode:  1372  score:  97.91  avg_score:  76.47  frames:  999  epsilon:  0.25
episode:  1373  score:  45.91  avg_score:  75.84  frames:  96  epsilon:  0.25
episode:  1374  score:  -13.89  avg_score:  75.75  frames:  99  epsilon:  0.25
episode:  1375  score:  143.84  avg_score:  76.46  frames:  999  epsilon:  0.25
episode:  1376  score:  99.39  avg_score:  76.57  frames:  999  epsilon:  0.25
episode:  1377  score:  77.34  avg_score:  76.40  frames:  999  epsilon:  0.25
episode:  1378  score:  119.86  avg_score:  76.42  frames:  999  epsilon:  0.25
episode:  1379  score:  -165.31  avg_score:  74.65  frames:  788  epsilon:  0.25
episode:  1380  score:  -308.09  avg_score:  70.14  frames:  547  epsilon:  0.25
episode:  1381  score:  15.20  avg_score:  69.51  frames:  107  epsilon:  0.25
episode:  1382  score:  21.19  avg_score:  67.93  frames:  999  epsilon:  0.25
episode:  1383  score:  56.70  avg_score:  68.23  frames:  999  epsilon:  0.25
episode:  1384  score:  78.94  avg_score:  68.85  frames:  999  epsilon:  0.25
episode:  1385  score:  15.05  avg_score:  67.64  frames:  108  epsilon:  0.25
episode:  1386  score:  -23.41  avg_score:  67.75  frames:  120  epsilon:  0.25
episode:  1387  score:  245.04  avg_score:  69.42  frames:  509  epsilon:  0.25
episode:  1388  score:  183.28  avg_score:  70.51  frames:  295  epsilon:  0.25
episode:  1389  score:  108.85  avg_score:  71.10  frames:  999  epsilon:  0.25
episode:  1390  score:  34.04  avg_score:  71.43  frames:  131  epsilon:  0.25
episode:  1391  score:  -226.80  avg_score:  67.86  frames:  696  epsilon:  0.25
episode:  1392  score:  47.22  avg_score:  67.03  frames:  104  epsilon:  0.25
episode:  1393  score:  7.04  avg_score:  66.23  frames:  107  epsilon:  0.25
episode:  1394  score:  86.47  avg_score:  66.16  frames:  999  epsilon:  0.25
episode:  1395  score:  49.76  avg_score:  66.83  frames:  999  epsilon:  0.25
episode:  1396  score:  113.46  avg_score:  65.74  frames:  999  epsilon:  0.25
episode:  1397  score:  31.89  avg_score:  64.86  frames:  122  epsilon:  0.25
episode:  1398  score:  41.24  avg_score:  67.11  frames:  107  epsilon:  0.25
episode:  1399  score:  120.12  avg_score:  67.87  frames:  999  epsilon:  0.25
episode:  1400  score:  44.81  avg_score:  67.15  frames:  108  epsilon:  0.25
episode:  1401  score:  159.34  avg_score:  66.21  frames:  999  epsilon:  0.25
episode:  1402  score:  273.36  avg_score:  71.15  frames:  311  epsilon:  0.25
episode:  1403  score:  -21.18  avg_score:  70.24  frames:  72  epsilon:  0.25
episode:  1404  score:  86.84  avg_score:  70.81  frames:  999  epsilon:  0.25
episode:  1405  score:  18.14  avg_score:  71.10  frames:  125  epsilon:  0.25
episode:  1406  score:  28.92  avg_score:  70.59  frames:  106  epsilon:  0.25
episode:  1407  score:  260.73  avg_score:  72.79  frames:  262  epsilon:  0.24
episode:  1408  score:  178.10  avg_score:  73.34  frames:  781  epsilon:  0.24
episode:  1409  score:  92.19  avg_score:  75.69  frames:  999  epsilon:  0.24
episode:  1410  score:  123.14  avg_score:  74.86  frames:  999  epsilon:  0.24
episode:  1411  score:  95.62  avg_score:  74.91  frames:  999  epsilon:  0.24
episode:  1412  score:  28.45  avg_score:  72.59  frames:  999  epsilon:  0.24
episode:  1413  score:  138.72  avg_score:  76.61  frames:  999  epsilon:  0.24
episode:  1414  score:  14.42  avg_score:  74.39  frames:  139  epsilon:  0.24
episode:  1415  score:  166.06  avg_score:  75.95  frames:  798  epsilon:  0.24
episode:  1416  score:  28.59  avg_score:  75.60  frames:  93  epsilon:  0.24
episode:  1417  score:  47.38  avg_score:  75.00  frames:  999  epsilon:  0.24
episode:  1418  score:  231.44  avg_score:  81.38  frames:  626  epsilon:  0.24
episode:  1419  score:  124.19  avg_score:  81.41  frames:  999  epsilon:  0.24
episode:  1420  score:  262.44  avg_score:  81.73  frames:  472  epsilon:  0.24
episode:  1421  score:  120.19  avg_score:  81.79  frames:  999  epsilon:  0.24
episode:  1422  score:  246.87  avg_score:  83.11  frames:  514  epsilon:  0.24
episode:  1423  score:  -148.52  avg_score:  79.58  frames:  211  epsilon:  0.24
episode:  1424  score:  217.58  avg_score:  81.06  frames:  779  epsilon:  0.24
episode:  1425  score:  216.25  avg_score:  84.37  frames:  534  epsilon:  0.24
episode:  1426  score:  83.90  avg_score:  84.79  frames:  999  epsilon:  0.24
episode:  1427  score:  -16.91  avg_score:  83.60  frames:  98  epsilon:  0.24
episode:  1428  score:  111.35  avg_score:  84.56  frames:  999  epsilon:  0.24
episode:  1429  score:  202.96  avg_score:  84.48  frames:  776  epsilon:  0.24
episode:  1430  score:  214.22  avg_score:  86.25  frames:  603  epsilon:  0.24
episode:  1431  score:  34.11  avg_score:  86.31  frames:  999  epsilon:  0.24
episode:  1432  score:  -62.79  avg_score:  85.52  frames:  82  epsilon:  0.24
episode:  1433  score:  106.57  avg_score:  85.39  frames:  999  epsilon:  0.24
episode:  1434  score:  79.76  avg_score:  84.61  frames:  999  epsilon:  0.24
episode:  1435  score:  -171.49  avg_score:  83.05  frames:  495  epsilon:  0.24
episode:  1436  score:  79.66  avg_score:  83.10  frames:  999  epsilon:  0.24
episode:  1437  score:  71.46  avg_score:  82.70  frames:  999  epsilon:  0.24
episode:  1438  score:  0.02  avg_score:  80.01  frames:  92  epsilon:  0.24
episode:  1439  score:  76.33  avg_score:  79.34  frames:  999  epsilon:  0.24
episode:  1440  score:  110.25  avg_score:  79.12  frames:  772  epsilon:  0.24
episode:  1441  score:  28.29  avg_score:  78.87  frames:  999  epsilon:  0.24
episode:  1442  score:  -42.57  avg_score:  77.04  frames:  999  epsilon:  0.24
episode:  1443  score:  58.59  avg_score:  76.68  frames:  999  epsilon:  0.24
episode:  1444  score:  108.42  avg_score:  76.52  frames:  999  epsilon:  0.24
episode:  1445  score:  91.77  avg_score:  76.36  frames:  999  epsilon:  0.24
episode:  1446  score:  89.97  avg_score:  74.47  frames:  999  epsilon:  0.24
episode:  1447  score:  124.53  avg_score:  74.14  frames:  949  epsilon:  0.24
episode:  1448  score:  -5.61  avg_score:  74.10  frames:  999  epsilon:  0.24
episode:  1449  score:  111.32  avg_score:  74.44  frames:  999  epsilon:  0.23
episode:  1450  score:  27.58  avg_score:  74.95  frames:  154  epsilon:  0.23
episode:  1451  score:  -4.65  avg_score:  74.76  frames:  208  epsilon:  0.23
episode:  1452  score:  160.26  avg_score:  78.24  frames:  715  epsilon:  0.23
episode:  1453  score:  96.49  avg_score:  76.91  frames:  999  epsilon:  0.23
episode:  1454  score:  60.21  avg_score:  76.46  frames:  999  epsilon:  0.23
episode:  1455  score:  119.82  avg_score:  75.29  frames:  977  epsilon:  0.23
episode:  1456  score:  263.15  avg_score:  77.32  frames:  220  epsilon:  0.23
episode:  1457  score:  52.10  avg_score:  77.23  frames:  116  epsilon:  0.23
episode:  1458  score:  90.48  avg_score:  78.65  frames:  999  epsilon:  0.23
episode:  1459  score:  189.58  avg_score:  79.43  frames:  709  epsilon:  0.23
episode:  1460  score:  87.90  avg_score:  80.18  frames:  999  epsilon:  0.23
episode:  1461  score:  94.82  avg_score:  81.33  frames:  999  epsilon:  0.23
episode:  1462  score:  77.46  avg_score:  79.73  frames:  999  epsilon:  0.23
episode:  1463  score:  47.44  avg_score:  80.19  frames:  999  epsilon:  0.23
episode:  1464  score:  50.33  avg_score:  78.34  frames:  999  epsilon:  0.23
episode:  1465  score:  43.37  avg_score:  76.44  frames:  999  epsilon:  0.23
episode:  1466  score:  5.28  avg_score:  76.65  frames:  999  epsilon:  0.23
episode:  1467  score:  170.10  avg_score:  75.82  frames:  527  epsilon:  0.23
episode:  1468  score:  183.92  avg_score:  76.33  frames:  479  epsilon:  0.23
episode:  1469  score:  74.38  avg_score:  76.07  frames:  999  epsilon:  0.23
episode:  1470  score:  43.18  avg_score:  75.53  frames:  999  epsilon:  0.23
episode:  1471  score:  -0.61  avg_score:  75.53  frames:  191  epsilon:  0.23
episode:  1472  score:  37.26  avg_score:  74.92  frames:  999  epsilon:  0.23
episode:  1473  score:  63.93  avg_score:  75.10  frames:  999  epsilon:  0.23
episode:  1474  score:  87.75  avg_score:  76.12  frames:  999  epsilon:  0.23
episode:  1475  score:  147.81  avg_score:  76.16  frames:  999  epsilon:  0.23
episode:  1476  score:  87.41  avg_score:  76.04  frames:  999  epsilon:  0.23
episode:  1477  score:  47.24  avg_score:  75.74  frames:  999  epsilon:  0.23
episode:  1478  score:  151.89  avg_score:  76.06  frames:  827  epsilon:  0.23
episode:  1479  score:  56.98  avg_score:  78.28  frames:  999  epsilon:  0.23
episode:  1480  score:  138.81  avg_score:  82.75  frames:  888  epsilon:  0.23
episode:  1481  score:  185.70  avg_score:  84.46  frames:  973  epsilon:  0.23
episode:  1482  score:  -36.32  avg_score:  83.88  frames:  259  epsilon:  0.23
episode:  1483  score:  49.89  avg_score:  83.81  frames:  999  epsilon:  0.23
episode:  1484  score:  109.38  avg_score:  84.12  frames:  999  epsilon:  0.23
episode:  1485  score:  149.69  avg_score:  85.46  frames:  999  epsilon:  0.23
episode:  1486  score:  215.16  avg_score:  87.85  frames:  915  epsilon:  0.23
episode:  1487  score:  223.75  avg_score:  87.64  frames:  431  epsilon:  0.23
episode:  1488  score:  -41.41  avg_score:  85.39  frames:  129  epsilon:  0.23
episode:  1489  score:  252.33  avg_score:  86.82  frames:  797  epsilon:  0.23
episode:  1490  score:  -17.07  avg_score:  86.31  frames:  187  epsilon:  0.23
episode:  1491  score:  68.10  avg_score:  89.26  frames:  999  epsilon:  0.23
episode:  1492  score:  195.27  avg_score:  90.74  frames:  350  epsilon:  0.22
episode:  1493  score:  107.92  avg_score:  91.75  frames:  999  epsilon:  0.22
episode:  1494  score:  237.29  avg_score:  93.26  frames:  925  epsilon:  0.22
episode:  1495  score:  206.20  avg_score:  94.82  frames:  352  epsilon:  0.22
episode:  1496  score:  261.07  avg_score:  96.30  frames:  536  epsilon:  0.22
episode:  1497  score:  5.59  avg_score:  96.04  frames:  90  epsilon:  0.22
episode:  1498  score:  -152.01  avg_score:  94.10  frames:  492  epsilon:  0.22
episode:  1499  score:  84.35  avg_score:  93.75  frames:  999  epsilon:  0.22
episode:  1500  score:  45.50  avg_score:  93.75  frames:  999  epsilon:  0.22
episode:  1501  score:  5.71  avg_score:  92.22  frames:  105  epsilon:  0.22
episode:  1502  score:  191.05  avg_score:  91.39  frames:  994  epsilon:  0.22
episode:  1503  score:  93.57  avg_score:  92.54  frames:  999  epsilon:  0.22
episode:  1504  score:  23.85  avg_score:  91.91  frames:  115  epsilon:  0.22
episode:  1505  score:  40.23  avg_score:  92.13  frames:  86  epsilon:  0.22
episode:  1506  score:  -47.47  avg_score:  91.37  frames:  124  epsilon:  0.22
episode:  1507  score:  238.80  avg_score:  91.15  frames:  745  epsilon:  0.22
episode:  1508  score:  -15.58  avg_score:  89.21  frames:  94  epsilon:  0.22
episode:  1509  score:  86.18  avg_score:  89.15  frames:  999  epsilon:  0.22
episode:  1510  score:  28.68  avg_score:  88.21  frames:  999  epsilon:  0.22
episode:  1511  score:  160.52  avg_score:  88.86  frames:  999  epsilon:  0.22
episode:  1512  score:  97.00  avg_score:  89.54  frames:  999  epsilon:  0.22
episode:  1513  score:  28.06  avg_score:  88.44  frames:  84  epsilon:  0.22
episode:  1514  score:  118.70  avg_score:  89.48  frames:  999  epsilon:  0.22
episode:  1515  score:  -45.49  avg_score:  87.36  frames:  96  epsilon:  0.22
episode:  1516  score:  238.88  avg_score:  89.47  frames:  572  epsilon:  0.22
episode:  1517  score:  208.01  avg_score:  91.07  frames:  365  epsilon:  0.22
episode:  1518  score:  -39.41  avg_score:  88.36  frames:  133  epsilon:  0.22
episode:  1519  score:  268.42  avg_score:  89.81  frames:  977  epsilon:  0.22
episode:  1520  score:  -50.47  avg_score:  86.68  frames:  105  epsilon:  0.22
episode:  1521  score:  33.95  avg_score:  85.82  frames:  999  epsilon:  0.22
episode:  1522  score:  26.44  avg_score:  83.61  frames:  89  epsilon:  0.22
episode:  1523  score:  101.69  avg_score:  86.11  frames:  999  epsilon:  0.22
episode:  1524  score:  -7.91  avg_score:  83.86  frames:  76  epsilon:  0.22
episode:  1525  score:  110.02  avg_score:  82.80  frames:  999  epsilon:  0.22
episode:  1526  score:  31.17  avg_score:  82.27  frames:  167  epsilon:  0.22
episode:  1527  score:  2.56  avg_score:  82.46  frames:  96  epsilon:  0.22
episode:  1528  score:  -5.92  avg_score:  81.29  frames:  122  epsilon:  0.22
episode:  1529  score:  -79.98  avg_score:  78.46  frames:  221  epsilon:  0.22
episode:  1530  score:  197.21  avg_score:  78.29  frames:  868  epsilon:  0.22
episode:  1531  score:  228.76  avg_score:  80.24  frames:  229  epsilon:  0.22
episode:  1532  score:  88.37  avg_score:  81.75  frames:  999  epsilon:  0.22
episode:  1533  score:  101.34  avg_score:  81.70  frames:  999  epsilon:  0.22
episode:  1534  score:  149.45  avg_score:  82.39  frames:  999  epsilon:  0.22
episode:  1535  score:  115.96  avg_score:  85.27  frames:  999  epsilon:  0.22
episode:  1536  score:  137.36  avg_score:  85.85  frames:  989  epsilon:  0.22
episode:  1537  score:  250.16  avg_score:  87.63  frames:  314  epsilon:  0.22
episode:  1538  score:  73.59  avg_score:  88.37  frames:  999  epsilon:  0.21
episode:  1539  score:  89.39  avg_score:  88.50  frames:  999  epsilon:  0.21
episode:  1540  score:  10.90  avg_score:  87.51  frames:  121  epsilon:  0.21
episode:  1541  score:  43.38  avg_score:  87.66  frames:  999  epsilon:  0.21
episode:  1542  score:  213.33  avg_score:  90.22  frames:  974  epsilon:  0.21
episode:  1543  score:  125.78  avg_score:  90.89  frames:  999  epsilon:  0.21
episode:  1544  score:  113.27  avg_score:  90.94  frames:  999  epsilon:  0.21
episode:  1545  score:  177.76  avg_score:  91.80  frames:  767  epsilon:  0.21
episode:  1546  score:  83.36  avg_score:  91.73  frames:  999  epsilon:  0.21
episode:  1547  score:  226.42  avg_score:  92.75  frames:  282  epsilon:  0.21
episode:  1548  score:  244.55  avg_score:  95.25  frames:  575  epsilon:  0.21
episode:  1549  score:  105.36  avg_score:  95.19  frames:  999  epsilon:  0.21
episode:  1550  score:  43.58  avg_score:  95.35  frames:  107  epsilon:  0.21
episode:  1551  score:  281.40  avg_score:  98.21  frames:  151  epsilon:  0.21
episode:  1552  score:  219.03  avg_score:  98.80  frames:  333  epsilon:  0.21
episode:  1553  score:  204.90  avg_score:  99.88  frames:  342  epsilon:  0.21
episode:  1554  score:  204.62  avg_score:  101.33  frames:  306  epsilon:  0.21
episode:  1555  score:  101.29  avg_score:  101.14  frames:  999  epsilon:  0.21
episode:  1556  score:  123.23  avg_score:  99.74  frames:  999  epsilon:  0.21
episode:  1557  score:  229.69  avg_score:  101.52  frames:  495  epsilon:  0.21
episode:  1558  score:  -29.25  avg_score:  100.32  frames:  84  epsilon:  0.21
episode:  1559  score:  -33.80  avg_score:  98.09  frames:  72  epsilon:  0.21
episode:  1560  score:  255.38  avg_score:  99.76  frames:  181  epsilon:  0.21
episode:  1561  score:  213.91  avg_score:  100.95  frames:  849  epsilon:  0.21
episode:  1562  score:  74.01  avg_score:  100.92  frames:  999  epsilon:  0.21
episode:  1563  score:  -78.04  avg_score:  99.66  frames:  111  epsilon:  0.21
episode:  1564  score:  40.41  avg_score:  99.56  frames:  999  epsilon:  0.21
episode:  1565  score:  253.98  avg_score:  101.67  frames:  216  epsilon:  0.21
episode:  1566  score:  73.08  avg_score:  102.35  frames:  999  epsilon:  0.21
episode:  1567  score:  15.66  avg_score:  100.80  frames:  156  epsilon:  0.21
episode:  1568  score:  13.15  avg_score:  99.10  frames:  106  epsilon:  0.21
episode:  1569  score:  18.70  avg_score:  98.54  frames:  999  epsilon:  0.21
episode:  1570  score:  33.15  avg_score:  98.44  frames:  166  epsilon:  0.21
episode:  1571  score:  -23.05  avg_score:  98.21  frames:  101  epsilon:  0.21
episode:  1572  score:  212.07  avg_score:  99.96  frames:  586  epsilon:  0.21
episode:  1573  score:  214.56  avg_score:  101.47  frames:  685  epsilon:  0.21
episode:  1574  score:  86.10  avg_score:  101.45  frames:  999  epsilon:  0.21
episode:  1575  score:  199.42  avg_score:  101.97  frames:  979  epsilon:  0.21
episode:  1576  score:  219.11  avg_score:  103.29  frames:  218  epsilon:  0.21
episode:  1577  score:  157.61  avg_score:  104.39  frames:  999  epsilon:  0.21
episode:  1578  score:  251.63  avg_score:  105.39  frames:  477  epsilon:  0.21
episode:  1579  score:  113.27  avg_score:  105.95  frames:  999  epsilon:  0.21
episode:  1580  score:  0.15  avg_score:  104.56  frames:  424  epsilon:  0.21
episode:  1581  score:  170.99  avg_score:  104.42  frames:  608  epsilon:  0.21
episode:  1582  score:  268.31  avg_score:  107.46  frames:  554  epsilon:  0.21
episode:  1583  score:  71.62  avg_score:  107.68  frames:  999  epsilon:  0.21
episode:  1584  score:  -261.46  avg_score:  103.97  frames:  239  epsilon:  0.21
episode:  1585  score:  121.31  avg_score:  103.69  frames:  999  epsilon:  0.21
episode:  1586  score:  187.18  avg_score:  103.41  frames:  818  epsilon:  0.20
episode:  1587  score:  19.36  avg_score:  101.36  frames:  120  epsilon:  0.20
episode:  1588  score:  112.46  avg_score:  102.90  frames:  999  epsilon:  0.20
episode:  1589  score:  10.11  avg_score:  100.48  frames:  97  epsilon:  0.20
episode:  1590  score:  63.63  avg_score:  101.29  frames:  999  epsilon:  0.20
episode:  1591  score:  224.40  avg_score:  102.85  frames:  965  epsilon:  0.20
episode:  1592  score:  148.30  avg_score:  102.38  frames:  999  epsilon:  0.20
episode:  1593  score:  -20.68  avg_score:  101.09  frames:  107  epsilon:  0.20
episode:  1594  score:  114.10  avg_score:  99.86  frames:  999  epsilon:  0.20
episode:  1595  score:  104.99  avg_score:  98.85  frames:  999  epsilon:  0.20
episode:  1596  score:  151.64  avg_score:  97.76  frames:  999  epsilon:  0.20
episode:  1597  score:  -50.94  avg_score:  97.19  frames:  89  epsilon:  0.20
episode:  1598  score:  14.87  avg_score:  98.86  frames:  126  epsilon:  0.20
episode:  1599  score:  225.02  avg_score:  100.27  frames:  555  epsilon:  0.20
episode:  1600  score:  245.35  avg_score:  102.26  frames:  827  epsilon:  0.20
episode:  1601  score:  19.70  avg_score:  102.40  frames:  110  epsilon:  0.20
episode:  1602  score:  221.81  avg_score:  102.71  frames:  487  epsilon:  0.20
episode:  1603  score:  227.90  avg_score:  104.06  frames:  323  epsilon:  0.20
episode:  1604  score:  39.01  avg_score:  104.21  frames:  132  epsilon:  0.20
episode:  1605  score:  113.21  avg_score:  104.94  frames:  999  epsilon:  0.20
episode:  1606  score:  -15.69  avg_score:  105.25  frames:  87  epsilon:  0.20
episode:  1607  score:  228.47  avg_score:  105.15  frames:  530  epsilon:  0.20
episode:  1608  score:  93.96  avg_score:  106.25  frames:  999  epsilon:  0.20
episode:  1609  score:  233.98  avg_score:  107.73  frames:  664  epsilon:  0.20
episode:  1610  score:  146.76  avg_score:  108.91  frames:  999  epsilon:  0.20
episode:  1611  score:  166.50  avg_score:  108.97  frames:  967  epsilon:  0.20
episode:  1612  score:  226.87  avg_score:  110.26  frames:  447  epsilon:  0.20
episode:  1613  score:  37.03  avg_score:  110.35  frames:  101  epsilon:  0.20
episode:  1614  score:  37.30  avg_score:  109.54  frames:  75  epsilon:  0.20
episode:  1615  score:  105.25  avg_score:  111.05  frames:  999  epsilon:  0.20
episode:  1616  score:  114.18  avg_score:  109.80  frames:  999  epsilon:  0.20
episode:  1617  score:  -9.22  avg_score:  107.63  frames:  87  epsilon:  0.20
episode:  1618  score:  109.12  avg_score:  109.11  frames:  999  epsilon:  0.20
episode:  1619  score:  209.80  avg_score:  108.53  frames:  494  epsilon:  0.20
episode:  1620  score:  2.67  avg_score:  109.06  frames:  117  epsilon:  0.20
episode:  1621  score:  256.26  avg_score:  111.28  frames:  394  epsilon:  0.20
episode:  1622  score:  198.43  avg_score:  113.00  frames:  851  epsilon:  0.20
episode:  1623  score:  183.00  avg_score:  113.81  frames:  454  epsilon:  0.20
episode:  1624  score:  191.70  avg_score:  115.81  frames:  597  epsilon:  0.20
episode:  1625  score:  64.54  avg_score:  115.36  frames:  999  epsilon:  0.20
episode:  1626  score:  5.64  avg_score:  115.10  frames:  128  epsilon:  0.20
episode:  1627  score:  267.84  avg_score:  117.75  frames:  180  epsilon:  0.20
episode:  1628  score:  -13.22  avg_score:  117.68  frames:  162  epsilon:  0.20
episode:  1629  score:  -21.14  avg_score:  118.27  frames:  999  epsilon:  0.20
episode:  1630  score:  107.58  avg_score:  117.37  frames:  999  epsilon:  0.20
episode:  1631  score:  156.02  avg_score:  116.64  frames:  501  epsilon:  0.20
episode:  1632  score:  234.37  avg_score:  118.10  frames:  672  epsilon:  0.20
episode:  1633  score:  138.55  avg_score:  118.48  frames:  999  epsilon:  0.20
episode:  1634  score:  180.30  avg_score:  118.79  frames:  945  epsilon:  0.20
episode:  1635  score:  -129.89  avg_score:  116.33  frames:  177  epsilon:  0.20
episode:  1636  score:  258.40  avg_score:  117.54  frames:  386  epsilon:  0.19
episode:  1637  score:  193.87  avg_score:  116.97  frames:  798  epsilon:  0.19
episode:  1638  score:  271.30  avg_score:  118.95  frames:  212  epsilon:  0.19
episode:  1639  score:  -38.30  avg_score:  117.67  frames:  205  epsilon:  0.19
episode:  1640  score:  -32.79  avg_score:  117.24  frames:  136  epsilon:  0.19
episode:  1641  score:  32.25  avg_score:  117.13  frames:  102  epsilon:  0.19
episode:  1642  score:  116.38  avg_score:  116.16  frames:  999  epsilon:  0.19
episode:  1643  score:  189.03  avg_score:  116.79  frames:  730  epsilon:  0.19
episode:  1644  score:  183.98  avg_score:  117.50  frames:  863  epsilon:  0.19
episode:  1645  score:  196.56  avg_score:  117.68  frames:  446  epsilon:  0.19
episode:  1646  score:  209.37  avg_score:  118.94  frames:  462  epsilon:  0.19
episode:  1647  score:  72.01  avg_score:  117.40  frames:  999  epsilon:  0.19
episode:  1648  score:  235.87  avg_score:  117.31  frames:  247  epsilon:  0.19
episode:  1649  score:  217.78  avg_score:  118.44  frames:  729  epsilon:  0.19
episode:  1650  score:  215.52  avg_score:  120.16  frames:  466  epsilon:  0.19
episode:  1651  score:  215.80  avg_score:  119.50  frames:  194  epsilon:  0.19
episode:  1652  score:  29.22  avg_score:  117.60  frames:  135  epsilon:  0.19
episode:  1653  score:  95.52  avg_score:  116.51  frames:  999  epsilon:  0.19
episode:  1654  score:  268.19  avg_score:  117.15  frames:  481  epsilon:  0.19
episode:  1655  score:  265.32  avg_score:  118.79  frames:  345  epsilon:  0.19
episode:  1656  score:  255.83  avg_score:  120.11  frames:  308  epsilon:  0.19
episode:  1657  score:  262.28  avg_score:  120.44  frames:  309  epsilon:  0.19
episode:  1658  score:  241.06  avg_score:  123.14  frames:  686  epsilon:  0.19
episode:  1659  score:  229.26  avg_score:  125.77  frames:  448  epsilon:  0.19
episode:  1660  score:  182.08  avg_score:  125.04  frames:  815  epsilon:  0.19
episode:  1661  score:  114.50  avg_score:  124.04  frames:  999  epsilon:  0.19
episode:  1662  score:  129.09  avg_score:  124.59  frames:  999  epsilon:  0.19
episode:  1663  score:  6.40  avg_score:  125.44  frames:  119  epsilon:  0.19
episode:  1664  score:  -217.85  avg_score:  122.86  frames:  360  epsilon:  0.19
episode:  1665  score:  264.62  avg_score:  122.96  frames:  234  epsilon:  0.19
episode:  1666  score:  79.49  avg_score:  123.03  frames:  999  epsilon:  0.19
episode:  1667  score:  16.92  avg_score:  123.04  frames:  112  epsilon:  0.19
episode:  1668  score:  236.72  avg_score:  125.28  frames:  483  epsilon:  0.19
episode:  1669  score:  229.30  avg_score:  127.38  frames:  827  epsilon:  0.19
episode:  1670  score:  -121.29  avg_score:  125.84  frames:  440  epsilon:  0.19
episode:  1671  score:  174.33  avg_score:  127.81  frames:  823  epsilon:  0.19
episode:  1672  score:  93.78  avg_score:  126.63  frames:  999  epsilon:  0.19
episode:  1673  score:  160.68  avg_score:  126.09  frames:  999  epsilon:  0.19
episode:  1674  score:  202.60  avg_score:  127.25  frames:  254  epsilon:  0.19
episode:  1675  score:  213.89  avg_score:  127.40  frames:  435  epsilon:  0.19
episode:  1676  score:  200.46  avg_score:  127.21  frames:  315  epsilon:  0.19
episode:  1677  score:  -1.98  avg_score:  125.62  frames:  111  epsilon:  0.19
episode:  1678  score:  13.18  avg_score:  123.23  frames:  126  epsilon:  0.19
episode:  1679  score:  200.40  avg_score:  124.10  frames:  630  epsilon:  0.19
episode:  1680  score:  115.32  avg_score:  125.25  frames:  999  epsilon:  0.19
episode:  1681  score:  224.38  avg_score:  125.79  frames:  599  epsilon:  0.19
episode:  1682  score:  77.30  avg_score:  123.88  frames:  999  epsilon:  0.19
episode:  1683  score:  253.76  avg_score:  125.70  frames:  850  epsilon:  0.19
episode:  1684  score:  -165.93  avg_score:  126.65  frames:  145  epsilon:  0.19
episode:  1685  score:  219.74  avg_score:  127.64  frames:  955  epsilon:  0.19
episode:  1686  score:  256.96  avg_score:  128.34  frames:  357  epsilon:  0.19
episode:  1687  score:  -150.58  avg_score:  126.64  frames:  341  epsilon:  0.19
episode:  1688  score:  -2.92  avg_score:  125.48  frames:  999  epsilon:  0.18
episode:  1689  score:  270.49  avg_score:  128.09  frames:  788  epsilon:  0.18
episode:  1690  score:  -28.39  avg_score:  127.17  frames:  138  epsilon:  0.18
episode:  1691  score:  232.51  avg_score:  127.25  frames:  397  epsilon:  0.18
episode:  1692  score:  203.56  avg_score:  127.80  frames:  269  epsilon:  0.18
episode:  1693  score:  -53.12  avg_score:  127.48  frames:  126  epsilon:  0.18
episode:  1694  score:  136.83  avg_score:  127.70  frames:  999  epsilon:  0.18
episode:  1695  score:  -50.47  avg_score:  126.15  frames:  294  epsilon:  0.18
episode:  1696  score:  249.57  avg_score:  127.13  frames:  697  epsilon:  0.18
episode:  1697  score:  221.98  avg_score:  129.86  frames:  563  epsilon:  0.18
episode:  1698  score:  -20.11  avg_score:  129.51  frames:  119  epsilon:  0.18
episode:  1699  score:  253.70  avg_score:  129.80  frames:  296  epsilon:  0.18
episode:  1700  score:  28.23  avg_score:  127.62  frames:  117  epsilon:  0.18
episode:  1701  score:  -82.75  avg_score:  126.60  frames:  574  epsilon:  0.18
episode:  1702  score:  93.39  avg_score:  125.32  frames:  999  epsilon:  0.18
episode:  1703  score:  230.25  avg_score:  125.34  frames:  346  epsilon:  0.18
episode:  1704  score:  5.41  avg_score:  125.00  frames:  115  epsilon:  0.18
episode:  1705  score:  179.07  avg_score:  125.66  frames:  588  epsilon:  0.18
episode:  1706  score:  222.09  avg_score:  128.04  frames:  595  epsilon:  0.18
episode:  1707  score:  118.26  avg_score:  126.94  frames:  999  epsilon:  0.18
episode:  1708  score:  169.36  avg_score:  127.69  frames:  321  epsilon:  0.18
episode:  1709  score:  214.37  avg_score:  127.50  frames:  611  epsilon:  0.18
episode:  1710  score:  189.09  avg_score:  127.92  frames:  580  epsilon:  0.18
episode:  1711  score:  15.01  avg_score:  126.40  frames:  84  epsilon:  0.18
episode:  1712  score:  239.04  avg_score:  126.53  frames:  309  epsilon:  0.18
episode:  1713  score:  199.77  avg_score:  128.15  frames:  451  epsilon:  0.18
episode:  1714  score:  215.69  avg_score:  129.94  frames:  375  epsilon:  0.18
episode:  1715  score:  170.10  avg_score:  130.59  frames:  917  epsilon:  0.18
episode:  1716  score:  111.18  avg_score:  130.56  frames:  999  epsilon:  0.18
episode:  1717  score:  201.65  avg_score:  132.66  frames:  509  epsilon:  0.18
episode:  1718  score:  217.21  avg_score:  133.74  frames:  772  epsilon:  0.18
episode:  1719  score:  228.95  avg_score:  133.94  frames:  743  epsilon:  0.18
episode:  1720  score:  230.13  avg_score:  136.21  frames:  997  epsilon:  0.18
episode:  1721  score:  232.14  avg_score:  135.97  frames:  286  epsilon:  0.18
episode:  1722  score:  239.76  avg_score:  136.38  frames:  314  epsilon:  0.18
episode:  1723  score:  235.34  avg_score:  136.91  frames:  837  epsilon:  0.18
episode:  1724  score:  -36.63  avg_score:  134.62  frames:  98  epsilon:  0.18
episode:  1725  score:  271.45  avg_score:  136.69  frames:  776  epsilon:  0.18
episode:  1726  score:  67.19  avg_score:  137.31  frames:  999  epsilon:  0.18
episode:  1727  score:  100.83  avg_score:  135.64  frames:  999  epsilon:  0.18
episode:  1728  score:  171.53  avg_score:  137.49  frames:  864  epsilon:  0.18
episode:  1729  score:  71.22  avg_score:  138.41  frames:  999  epsilon:  0.18
episode:  1730  score:  -2.56  avg_score:  137.31  frames:  144  epsilon:  0.18
episode:  1731  score:  -25.97  avg_score:  135.49  frames:  67  epsilon:  0.18
episode:  1732  score:  21.90  avg_score:  133.36  frames:  123  epsilon:  0.18
episode:  1733  score:  -122.63  avg_score:  130.75  frames:  91  epsilon:  0.18
episode:  1734  score:  219.87  avg_score:  131.15  frames:  871  epsilon:  0.18
episode:  1735  score:  5.01  avg_score:  132.50  frames:  76  epsilon:  0.18
episode:  1736  score:  224.39  avg_score:  132.16  frames:  360  epsilon:  0.18
episode:  1737  score:  -11.53  avg_score:  130.10  frames:  103  epsilon:  0.18
episode:  1738  score:  222.13  avg_score:  129.61  frames:  389  epsilon:  0.18
episode:  1739  score:  108.53  avg_score:  131.08  frames:  999  epsilon:  0.18
episode:  1740  score:  195.01  avg_score:  133.36  frames:  419  epsilon:  0.18
episode:  1741  score:  191.72  avg_score:  134.95  frames:  616  epsilon:  0.18
episode:  1742  score:  148.14  avg_score:  135.27  frames:  999  epsilon:  0.18
episode:  1743  score:  232.11  avg_score:  135.70  frames:  655  epsilon:  0.18
episode:  1744  score:  264.32  avg_score:  136.50  frames:  397  epsilon:  0.17
episode:  1745  score:  224.94  avg_score:  136.79  frames:  194  epsilon:  0.17
episode:  1746  score:  -20.96  avg_score:  134.48  frames:  128  epsilon:  0.17
episode:  1747  score:  280.65  avg_score:  136.57  frames:  784  epsilon:  0.17
episode:  1748  score:  226.69  avg_score:  136.48  frames:  921  epsilon:  0.17
episode:  1749  score:  -165.84  avg_score:  132.64  frames:  117  epsilon:  0.17
episode:  1750  score:  202.49  avg_score:  132.51  frames:  643  epsilon:  0.17
episode:  1751  score:  242.46  avg_score:  132.78  frames:  637  epsilon:  0.17
episode:  1752  score:  116.94  avg_score:  133.65  frames:  999  epsilon:  0.17
episode:  1753  score:  243.62  avg_score:  135.14  frames:  694  epsilon:  0.17
episode:  1754  score:  193.67  avg_score:  134.39  frames:  292  epsilon:  0.17
episode:  1755  score:  221.26  avg_score:  133.95  frames:  184  epsilon:  0.17
episode:  1756  score:  219.40  avg_score:  133.59  frames:  248  epsilon:  0.17
episode:  1757  score:  231.69  avg_score:  133.28  frames:  364  epsilon:  0.17
episode:  1758  score:  241.85  avg_score:  133.29  frames:  869  epsilon:  0.17
episode:  1759  score:  246.87  avg_score:  133.46  frames:  961  epsilon:  0.17
episode:  1760  score:  234.18  avg_score:  133.98  frames:  312  epsilon:  0.17
episode:  1761  score:  -21.44  avg_score:  132.63  frames:  108  epsilon:  0.17
episode:  1762  score:  224.00  avg_score:  133.57  frames:  733  epsilon:  0.17
episode:  1763  score:  197.21  avg_score:  135.48  frames:  896  epsilon:  0.17
episode:  1764  score:  229.50  avg_score:  139.96  frames:  404  epsilon:  0.17
episode:  1765  score:  16.85  avg_score:  137.48  frames:  100  epsilon:  0.17
episode:  1766  score:  242.65  avg_score:  139.11  frames:  372  epsilon:  0.17
episode:  1767  score:  -135.20  avg_score:  137.59  frames:  347  epsilon:  0.17
episode:  1768  score:  212.53  avg_score:  137.35  frames:  471  epsilon:  0.17
episode:  1769  score:  247.15  avg_score:  137.53  frames:  304  epsilon:  0.17
episode:  1770  score:  -194.39  avg_score:  136.79  frames:  417  epsilon:  0.17
episode:  1771  score:  2.85  avg_score:  135.08  frames:  104  epsilon:  0.17
episode:  1772  score:  13.41  avg_score:  134.28  frames:  112  epsilon:  0.17
episode:  1773  score:  230.79  avg_score:  134.98  frames:  394  epsilon:  0.17
episode:  1774  score:  277.76  avg_score:  135.73  frames:  276  epsilon:  0.17
episode:  1775  score:  260.67  avg_score:  136.20  frames:  538  epsilon:  0.17
episode:  1776  score:  203.04  avg_score:  136.22  frames:  517  epsilon:  0.17
episode:  1777  score:  246.18  avg_score:  138.70  frames:  400  epsilon:  0.17
episode:  1778  score:  239.88  avg_score:  140.97  frames:  169  epsilon:  0.17
episode:  1779  score:  231.00  avg_score:  141.28  frames:  298  epsilon:  0.17
episode:  1780  score:  189.60  avg_score:  142.02  frames:  867  epsilon:  0.17
episode:  1781  score:  240.43  avg_score:  142.18  frames:  263  epsilon:  0.17
episode:  1782  score:  117.48  avg_score:  142.58  frames:  999  epsilon:  0.17
episode:  1783  score:  223.17  avg_score:  142.28  frames:  714  epsilon:  0.17
episode:  1784  score:  3.66  avg_score:  143.97  frames:  124  epsilon:  0.17
episode:  1785  score:  39.23  avg_score:  142.17  frames:  106  epsilon:  0.17
episode:  1786  score:  8.09  avg_score:  139.68  frames:  102  epsilon:  0.17
episode:  1787  score:  107.82  avg_score:  142.26  frames:  999  epsilon:  0.17
episode:  1788  score:  -72.61  avg_score:  141.57  frames:  452  epsilon:  0.17
episode:  1789  score:  231.13  avg_score:  141.17  frames:  369  epsilon:  0.17
episode:  1790  score:  236.01  avg_score:  143.82  frames:  345  epsilon:  0.17
episode:  1791  score:  235.15  avg_score:  143.84  frames:  217  epsilon:  0.17
episode:  1792  score:  47.92  avg_score:  142.29  frames:  89  epsilon:  0.17
episode:  1793  score:  -11.59  avg_score:  142.70  frames:  118  epsilon:  0.17
episode:  1794  score:  237.28  avg_score:  143.71  frames:  270  epsilon:  0.17
episode:  1795  score:  247.49  avg_score:  146.69  frames:  345  epsilon:  0.17
episode:  1796  score:  -4.41  avg_score:  144.15  frames:  96  epsilon:  0.17
episode:  1797  score:  275.75  avg_score:  144.68  frames:  506  epsilon:  0.17
episode:  1798  score:  232.07  avg_score:  147.20  frames:  303  epsilon:  0.17
episode:  1799  score:  53.29  avg_score:  145.20  frames:  135  epsilon:  0.17
episode:  1800  score:  -112.93  avg_score:  143.79  frames:  429  epsilon:  0.17
episode:  1801  score:  264.33  avg_score:  147.26  frames:  274  epsilon:  0.17
episode:  1802  score:  29.71  avg_score:  146.62  frames:  91  epsilon:  0.17
episode:  1803  score:  -198.16  avg_score:  142.34  frames:  255  epsilon:  0.16
episode:  1804  score:  242.87  avg_score:  144.71  frames:  223  epsilon:  0.16
episode:  1805  score:  214.03  avg_score:  145.06  frames:  538  epsilon:  0.16
episode:  1806  score:  16.57  avg_score:  143.01  frames:  152  epsilon:  0.16
episode:  1807  score:  250.16  avg_score:  144.33  frames:  201  epsilon:  0.16
episode:  1808  score:  109.22  avg_score:  143.73  frames:  999  epsilon:  0.16
episode:  1809  score:  -35.19  avg_score:  141.23  frames:  114  epsilon:  0.16
episode:  1810  score:  256.84  avg_score:  141.91  frames:  277  epsilon:  0.16
episode:  1811  score:  254.52  avg_score:  144.30  frames:  187  epsilon:  0.16
episode:  1812  score:  238.53  avg_score:  144.30  frames:  497  epsilon:  0.16
episode:  1813  score:  32.11  avg_score:  142.62  frames:  212  epsilon:  0.16
episode:  1814  score:  199.90  avg_score:  142.46  frames:  427  epsilon:  0.16
episode:  1815  score:  238.69  avg_score:  143.15  frames:  381  epsilon:  0.16
episode:  1816  score:  248.83  avg_score:  144.53  frames:  649  epsilon:  0.16
episode:  1817  score:  263.41  avg_score:  145.14  frames:  523  epsilon:  0.16
episode:  1818  score:  -131.50  avg_score:  141.66  frames:  209  epsilon:  0.16
episode:  1819  score:  -17.55  avg_score:  139.19  frames:  110  epsilon:  0.16
episode:  1820  score:  248.53  avg_score:  139.37  frames:  880  epsilon:  0.16
episode:  1821  score:  255.91  avg_score:  139.61  frames:  328  epsilon:  0.16
episode:  1822  score:  117.71  avg_score:  138.39  frames:  999  epsilon:  0.16
episode:  1823  score:  225.73  avg_score:  138.30  frames:  330  epsilon:  0.16
episode:  1824  score:  289.57  avg_score:  141.56  frames:  460  epsilon:  0.16
episode:  1825  score:  259.12  avg_score:  141.43  frames:  329  epsilon:  0.16
episode:  1826  score:  81.50  avg_score:  141.58  frames:  999  epsilon:  0.16
episode:  1827  score:  84.38  avg_score:  141.41  frames:  999  epsilon:  0.16
episode:  1828  score:  251.74  avg_score:  142.21  frames:  367  epsilon:  0.16
episode:  1829  score:  3.30  avg_score:  141.54  frames:  881  epsilon:  0.16
episode:  1830  score:  226.47  avg_score:  143.83  frames:  185  epsilon:  0.16
episode:  1831  score:  38.88  avg_score:  144.47  frames:  124  epsilon:  0.16
episode:  1832  score:  272.05  avg_score:  146.98  frames:  473  epsilon:  0.16
episode:  1833  score:  1.22  avg_score:  148.21  frames:  110  epsilon:  0.16
episode:  1834  score:  256.72  avg_score:  148.58  frames:  220  epsilon:  0.16
episode:  1835  score:  208.92  avg_score:  150.62  frames:  190  epsilon:  0.16
episode:  1836  score:  10.36  avg_score:  148.48  frames:  113  epsilon:  0.16
episode:  1837  score:  133.65  avg_score:  149.93  frames:  813  epsilon:  0.16
episode:  1838  score:  262.51  avg_score:  150.34  frames:  430  epsilon:  0.16
episode:  1839  score:  212.48  avg_score:  151.38  frames:  237  epsilon:  0.16
episode:  1840  score:  241.86  avg_score:  151.85  frames:  267  epsilon:  0.16
episode:  1841  score:  225.91  avg_score:  152.19  frames:  202  epsilon:  0.16
episode:  1842  score:  22.14  avg_score:  150.93  frames:  96  epsilon:  0.16
episode:  1843  score:  240.70  avg_score:  151.01  frames:  793  epsilon:  0.16
episode:  1844  score:  266.15  avg_score:  151.03  frames:  216  epsilon:  0.16
episode:  1845  score:  233.62  avg_score:  151.12  frames:  196  epsilon:  0.16
episode:  1846  score:  206.93  avg_score:  153.40  frames:  474  epsilon:  0.16
episode:  1847  score:  280.54  avg_score:  153.40  frames:  334  epsilon:  0.16
episode:  1848  score:  46.71  avg_score:  151.60  frames:  999  epsilon:  0.16
episode:  1849  score:  211.66  avg_score:  155.37  frames:  475  epsilon:  0.16
episode:  1850  score:  174.75  avg_score:  155.09  frames:  822  epsilon:  0.16
episode:  1851  score:  264.92  avg_score:  155.32  frames:  321  epsilon:  0.16
episode:  1852  score:  57.60  avg_score:  154.73  frames:  90  epsilon:  0.16
episode:  1853  score:  271.81  avg_score:  155.01  frames:  485  epsilon:  0.16
episode:  1854  score:  228.97  avg_score:  155.36  frames:  226  epsilon:  0.16
episode:  1855  score:  253.13  avg_score:  155.68  frames:  397  epsilon:  0.16
episode:  1856  score:  239.50  avg_score:  155.88  frames:  219  epsilon:  0.16
episode:  1857  score:  11.71  avg_score:  153.68  frames:  86  epsilon:  0.16
episode:  1858  score:  275.07  avg_score:  154.01  frames:  202  epsilon:  0.16
episode:  1859  score:  253.05  avg_score:  154.07  frames:  214  epsilon:  0.16
episode:  1860  score:  255.14  avg_score:  154.28  frames:  180  epsilon:  0.16
episode:  1861  score:  259.70  avg_score:  157.10  frames:  729  epsilon:  0.16
episode:  1862  score:  204.83  avg_score:  156.90  frames:  481  epsilon:  0.16
episode:  1863  score:  266.87  avg_score:  157.60  frames:  206  epsilon:  0.16
episode:  1864  score:  255.37  avg_score:  157.86  frames:  286  epsilon:  0.16
episode:  1865  score:  -12.61  avg_score:  157.56  frames:  92  epsilon:  0.15
episode:  1866  score:  216.44  avg_score:  157.30  frames:  324  epsilon:  0.15
episode:  1867  score:  251.50  avg_score:  161.17  frames:  195  epsilon:  0.15
episode:  1868  score:  195.82  avg_score:  161.00  frames:  424  epsilon:  0.15
episode:  1869  score:  102.18  avg_score:  159.55  frames:  999  epsilon:  0.15
episode:  1870  score:  273.19  avg_score:  164.23  frames:  320  epsilon:  0.15
episode:  1871  score:  265.87  avg_score:  166.86  frames:  283  epsilon:  0.15
episode:  1872  score:  19.80  avg_score:  166.92  frames:  84  epsilon:  0.15
episode:  1873  score:  189.94  avg_score:  166.51  frames:  905  epsilon:  0.15
episode:  1874  score:  31.51  avg_score:  164.05  frames:  126  epsilon:  0.15
episode:  1875  score:  244.45  avg_score:  163.89  frames:  267  epsilon:  0.15
episode:  1876  score:  246.22  avg_score:  164.32  frames:  275  epsilon:  0.15
episode:  1877  score:  227.84  avg_score:  164.14  frames:  816  epsilon:  0.15
episode:  1878  score:  266.52  avg_score:  164.40  frames:  202  epsilon:  0.15
episode:  1879  score:  201.80  avg_score:  164.11  frames:  213  epsilon:  0.15
episode:  1880  score:  240.83  avg_score:  164.62  frames:  286  epsilon:  0.15
episode:  1881  score:  250.08  avg_score:  164.72  frames:  247  epsilon:  0.15
episode:  1882  score:  207.66  avg_score:  165.62  frames:  408  epsilon:  0.15
episode:  1883  score:  189.51  avg_score:  165.29  frames:  609  epsilon:  0.15
episode:  1884  score:  237.74  avg_score:  167.63  frames:  355  epsilon:  0.15
episode:  1885  score:  218.38  avg_score:  169.42  frames:  811  epsilon:  0.15
episode:  1886  score:  272.26  avg_score:  172.06  frames:  427  epsilon:  0.15
episode:  1887  score:  2.09  avg_score:  171.00  frames:  110  epsilon:  0.15
episode:  1888  score:  267.30  avg_score:  174.40  frames:  316  epsilon:  0.15
episode:  1889  score:  247.08  avg_score:  174.56  frames:  291  epsilon:  0.15
episode:  1890  score:  258.88  avg_score:  174.79  frames:  696  epsilon:  0.15
episode:  1891  score:  271.71  avg_score:  175.16  frames:  320  epsilon:  0.15
episode:  1892  score:  254.46  avg_score:  177.22  frames:  205  epsilon:  0.15
episode:  1893  score:  277.86  avg_score:  180.12  frames:  301  epsilon:  0.15
episode:  1894  score:  261.54  avg_score:  180.36  frames:  244  epsilon:  0.15
episode:  1895  score:  240.54  avg_score:  180.29  frames:  284  epsilon:  0.15
episode:  1896  score:  220.66  avg_score:  182.54  frames:  967  epsilon:  0.15
episode:  1897  score:  275.67  avg_score:  182.54  frames:  502  epsilon:  0.15
episode:  1898  score:  286.56  avg_score:  183.08  frames:  299  epsilon:  0.15
episode:  1899  score:  222.81  avg_score:  184.78  frames:  338  epsilon:  0.15
episode:  1900  score:  114.03  avg_score:  187.05  frames:  999  epsilon:  0.15
episode:  1901  score:  281.46  avg_score:  187.22  frames:  192  epsilon:  0.15
episode:  1902  score:  214.82  avg_score:  189.07  frames:  391  epsilon:  0.15
episode:  1903  score:  89.47  avg_score:  191.95  frames:  999  epsilon:  0.15
episode:  1904  score:  73.60  avg_score:  190.25  frames:  953  epsilon:  0.15
episode:  1905  score:  -24.58  avg_score:  187.87  frames:  123  epsilon:  0.15
episode:  1906  score:  -245.71  avg_score:  185.25  frames:  161  epsilon:  0.15
episode:  1907  score:  189.18  avg_score:  184.64  frames:  560  epsilon:  0.15
episode:  1908  score:  -104.56  avg_score:  182.50  frames:  999  epsilon:  0.15
episode:  1909  score:  231.56  avg_score:  185.17  frames:  956  epsilon:  0.15
episode:  1910  score:  96.65  avg_score:  183.56  frames:  999  epsilon:  0.15
episode:  1911  score:  197.50  avg_score:  182.99  frames:  311  epsilon:  0.15
episode:  1912  score:  228.65  avg_score:  182.89  frames:  366  epsilon:  0.15
episode:  1913  score:  -36.49  avg_score:  182.21  frames:  113  epsilon:  0.15
episode:  1914  score:  236.68  avg_score:  182.58  frames:  297  epsilon:  0.15
episode:  1915  score:  260.01  avg_score:  182.79  frames:  333  epsilon:  0.15
episode:  1916  score:  109.88  avg_score:  181.40  frames:  999  epsilon:  0.15
episode:  1917  score:  -3.40  avg_score:  178.73  frames:  140  epsilon:  0.15
episode:  1918  score:  244.30  avg_score:  182.49  frames:  207  epsilon:  0.15
episode:  1919  score:  270.88  avg_score:  185.37  frames:  196  epsilon:  0.15
episode:  1920  score:  127.32  avg_score:  184.16  frames:  999  epsilon:  0.15
episode:  1921  score:  212.48  avg_score:  183.73  frames:  269  epsilon:  0.15
episode:  1922  score:  229.05  avg_score:  184.84  frames:  593  epsilon:  0.15
episode:  1923  score:  272.85  avg_score:  185.31  frames:  361  epsilon:  0.15
episode:  1924  score:  117.24  avg_score:  183.59  frames:  999  epsilon:  0.15
episode:  1925  score:  206.68  avg_score:  183.06  frames:  310  epsilon:  0.15
episode:  1926  score:  232.44  avg_score:  184.57  frames:  575  epsilon:  0.15
episode:  1927  score:  0.54  avg_score:  183.74  frames:  88  epsilon:  0.15
episode:  1928  score:  9.19  avg_score:  181.31  frames:  304  epsilon:  0.15
episode:  1929  score:  79.20  avg_score:  182.07  frames:  999  epsilon:  0.15
episode:  1930  score:  259.77  avg_score:  182.40  frames:  343  epsilon:  0.15
episode:  1931  score:  -46.12  avg_score:  181.55  frames:  408  epsilon:  0.15
episode:  1932  score:  193.55  avg_score:  180.77  frames:  618  epsilon:  0.14
episode:  1933  score:  197.37  avg_score:  182.73  frames:  907  epsilon:  0.14
episode:  1934  score:  255.38  avg_score:  182.72  frames:  176  epsilon:  0.14
episode:  1935  score:  179.41  avg_score:  182.42  frames:  935  epsilon:  0.14
episode:  1936  score:  131.38  avg_score:  183.63  frames:  999  epsilon:  0.14
episode:  1937  score:  220.66  avg_score:  184.50  frames:  388  epsilon:  0.14
episode:  1938  score:  246.50  avg_score:  184.34  frames:  310  epsilon:  0.14
episode:  1939  score:  205.98  avg_score:  184.28  frames:  288  epsilon:  0.14
episode:  1940  score:  179.46  avg_score:  183.65  frames:  424  epsilon:  0.14
episode:  1941  score:  187.52  avg_score:  183.27  frames:  400  epsilon:  0.14
episode:  1942  score:  112.51  avg_score:  184.17  frames:  999  epsilon:  0.14
episode:  1943  score:  205.63  avg_score:  183.82  frames:  392  epsilon:  0.14
episode:  1944  score:  266.87  avg_score:  183.83  frames:  280  epsilon:  0.14
episode:  1945  score:  247.34  avg_score:  183.96  frames:  438  epsilon:  0.14
episode:  1946  score:  253.11  avg_score:  184.43  frames:  467  epsilon:  0.14
episode:  1947  score:  208.53  avg_score:  183.71  frames:  765  epsilon:  0.14
episode:  1948  score:  246.77  avg_score:  185.71  frames:  186  epsilon:  0.14
episode:  1949  score:  221.37  avg_score:  185.80  frames:  370  epsilon:  0.14
episode:  1950  score:  231.63  avg_score:  186.37  frames:  213  epsilon:  0.14
episode:  1951  score:  5.61  avg_score:  183.78  frames:  113  epsilon:  0.14
episode:  1952  score:  226.55  avg_score:  185.47  frames:  444  epsilon:  0.14
episode:  1953  score:  237.95  avg_score:  185.13  frames:  337  epsilon:  0.14
episode:  1954  score:  239.08  avg_score:  185.23  frames:  260  epsilon:  0.14
episode:  1955  score:  243.64  avg_score:  185.14  frames:  250  epsilon:  0.14
episode:  1956  score:  259.75  avg_score:  185.34  frames:  773  epsilon:  0.14
episode:  1957  score:  218.46  avg_score:  187.41  frames:  180  epsilon:  0.14
episode:  1958  score:  258.66  avg_score:  187.24  frames:  224  epsilon:  0.14
episode:  1959  score:  225.42  avg_score:  186.97  frames:  399  epsilon:  0.14
episode:  1960  score:  90.71  avg_score:  185.32  frames:  999  epsilon:  0.14
episode:  1961  score:  271.53  avg_score:  185.44  frames:  163  epsilon:  0.14
episode:  1962  score:  -18.36  avg_score:  183.21  frames:  297  epsilon:  0.14
episode:  1963  score:  267.50  avg_score:  183.21  frames:  182  epsilon:  0.14
episode:  1964  score:  214.75  avg_score:  182.81  frames:  253  epsilon:  0.14
episode:  1965  score:  5.91  avg_score:  182.99  frames:  109  epsilon:  0.14
episode:  1966  score:  239.06  avg_score:  183.22  frames:  785  epsilon:  0.14
episode:  1967  score:  110.36  avg_score:  181.81  frames:  999  epsilon:  0.14
episode:  1968  score:  226.40  avg_score:  182.11  frames:  185  epsilon:  0.14
episode:  1969  score:  233.86  avg_score:  183.43  frames:  224  epsilon:  0.14
episode:  1970  score:  245.46  avg_score:  183.15  frames:  226  epsilon:  0.14
episode:  1971  score:  85.37  avg_score:  181.35  frames:  999  epsilon:  0.14
episode:  1972  score:  267.47  avg_score:  183.83  frames:  312  epsilon:  0.14
episode:  1973  score:  253.84  avg_score:  184.46  frames:  480  epsilon:  0.14
episode:  1974  score:  239.91  avg_score:  186.55  frames:  331  epsilon:  0.14
episode:  1975  score:  228.50  avg_score:  186.39  frames:  238  epsilon:  0.14
episode:  1976  score:  280.12  avg_score:  186.73  frames:  379  epsilon:  0.14
episode:  1977  score:  251.15  avg_score:  186.96  frames:  282  epsilon:  0.14
episode:  1978  score:  282.74  avg_score:  187.12  frames:  399  epsilon:  0.14
episode:  1979  score:  28.24  avg_score:  185.39  frames:  186  epsilon:  0.14
episode:  1980  score:  264.45  avg_score:  185.62  frames:  182  epsilon:  0.14
episode:  1981  score:  284.83  avg_score:  185.97  frames:  241  epsilon:  0.14
episode:  1982  score:  185.56  avg_score:  185.75  frames:  785  epsilon:  0.14
episode:  1983  score:  271.39  avg_score:  186.57  frames:  228  epsilon:  0.14
episode:  1984  score:  260.73  avg_score:  186.80  frames:  221  epsilon:  0.14
episode:  1985  score:  12.36  avg_score:  184.74  frames:  112  epsilon:  0.14
episode:  1986  score:  284.25  avg_score:  184.86  frames:  251  epsilon:  0.14
episode:  1987  score:  209.32  avg_score:  186.93  frames:  623  epsilon:  0.14
episode:  1988  score:  235.76  avg_score:  186.62  frames:  290  epsilon:  0.14
episode:  1989  score:  224.26  avg_score:  186.39  frames:  731  epsilon:  0.14
episode:  1990  score:  266.12  avg_score:  186.46  frames:  290  epsilon:  0.14
episode:  1991  score:  216.36  avg_score:  185.91  frames:  250  epsilon:  0.14
episode:  1992  score:  31.43  avg_score:  183.68  frames:  108  epsilon:  0.14
episode:  1993  score:  220.16  avg_score:  183.10  frames:  191  epsilon:  0.14
episode:  1994  score:  287.10  avg_score:  183.35  frames:  160  epsilon:  0.14
episode:  1995  score:  233.78  avg_score:  183.29  frames:  362  epsilon:  0.14
episode:  1996  score:  96.76  avg_score:  182.05  frames:  999  epsilon:  0.14
episode:  1997  score:  254.93  avg_score:  181.84  frames:  187  epsilon:  0.14
episode:  1998  score:  259.70  avg_score:  181.57  frames:  516  epsilon:  0.14
episode:  1999  score:  234.64  avg_score:  181.69  frames:  463  epsilon:  0.14
episode:  2000  score:  258.72  avg_score:  183.14  frames:  236  epsilon:  0.14
episode:  2001  score:  291.13  avg_score:  183.23  frames:  155  epsilon:  0.14
episode:  2002  score:  247.70  avg_score:  183.56  frames:  232  epsilon:  0.14
episode:  2003  score:  267.42  avg_score:  185.34  frames:  162  epsilon:  0.14
episode:  2004  score:  212.65  avg_score:  186.73  frames:  403  epsilon:  0.13
episode:  2005  score:  246.09  avg_score:  189.44  frames:  231  epsilon:  0.13
episode:  2006  score:  255.85  avg_score:  194.46  frames:  294  epsilon:  0.13
episode:  2007  score:  7.87  avg_score:  192.64  frames:  102  epsilon:  0.13
episode:  2008  score:  252.30  avg_score:  196.21  frames:  996  epsilon:  0.13
episode:  2009  score:  259.66  avg_score:  196.49  frames:  264  epsilon:  0.13
episode:  2010  score:  249.92  avg_score:  198.02  frames:  282  epsilon:  0.13
episode:  2011  score:  264.29  avg_score:  198.69  frames:  366  epsilon:  0.13
episode:  2012  score:  264.76  avg_score:  199.05  frames:  346  epsilon:  0.13
episode:  2013  score:  232.00  avg_score:  201.74  frames:  810  epsilon:  0.13
episode:  2014  score:  283.20  avg_score:  202.20  frames:  287  epsilon:  0.13
episode:  2015  score:  241.04  avg_score:  202.01  frames:  193  epsilon:  0.13
episode:  2016  score:  5.18  avg_score:  200.97  frames:  96  epsilon:  0.13
episode:  2017  score:  273.61  avg_score:  203.74  frames:  465  epsilon:  0.13
episode:  2018  score:  -11.41  avg_score:  201.18  frames:  130  epsilon:  0.13
episode:  2019  score:  -6.72  avg_score:  198.40  frames:  120  epsilon:  0.13
episode:  2020  score:  285.79  avg_score:  199.99  frames:  241  epsilon:  0.13
episode:  2021  score:  43.11  avg_score:  198.29  frames:  103  epsilon:  0.13
episode:  2022  score:  -10.96  avg_score:  195.89  frames:  121  epsilon:  0.13
episode:  2023  score:  255.22  avg_score:  195.72  frames:  275  epsilon:  0.13
episode:  2024  score:  51.38  avg_score:  195.06  frames:  130  epsilon:  0.13
episode:  2025  score:  7.20  avg_score:  193.07  frames:  91  epsilon:  0.13
episode:  2026  score:  24.86  avg_score:  190.99  frames:  103  epsilon:  0.13
episode:  2027  score:  38.76  avg_score:  191.37  frames:  121  epsilon:  0.13
episode:  2028  score:  187.13  avg_score:  193.15  frames:  925  epsilon:  0.13
episode:  2029  score:  -26.36  avg_score:  192.10  frames:  118  epsilon:  0.13
episode:  2030  score:  -2.62  avg_score:  189.47  frames:  93  epsilon:  0.13
episode:  2031  score:  249.08  avg_score:  192.42  frames:  257  epsilon:  0.13
episode:  2032  score:  229.10  avg_score:  192.78  frames:  421  epsilon:  0.13
episode:  2033  score:  261.10  avg_score:  193.42  frames:  160  epsilon:  0.13
episode:  2034  score:  244.89  avg_score:  193.31  frames:  463  epsilon:  0.13
episode:  2035  score:  4.03  avg_score:  191.56  frames:  96  epsilon:  0.13
episode:  2036  score:  245.59  avg_score:  192.70  frames:  261  epsilon:  0.13
episode:  2037  score:  -114.84  avg_score:  189.34  frames:  254  epsilon:  0.13
episode:  2038  score:  263.51  avg_score:  189.51  frames:  198  epsilon:  0.13
episode:  2039  score:  267.43  avg_score:  190.13  frames:  394  epsilon:  0.13
episode:  2040  score:  225.10  avg_score:  190.59  frames:  587  epsilon:  0.13
episode:  2041  score:  240.04  avg_score:  191.11  frames:  246  epsilon:  0.13
episode:  2042  score:  234.70  avg_score:  192.33  frames:  194  epsilon:  0.13
episode:  2043  score:  265.31  avg_score:  192.93  frames:  188  epsilon:  0.13
episode:  2044  score:  240.56  avg_score:  192.67  frames:  226  epsilon:  0.13
episode:  2045  score:  271.33  avg_score:  192.91  frames:  310  epsilon:  0.13
episode:  2046  score:  -11.17  avg_score:  190.26  frames:  73  epsilon:  0.13
episode:  2047  score:  275.27  avg_score:  190.93  frames:  137  epsilon:  0.13
episode:  2048  score:  250.23  avg_score:  190.97  frames:  238  epsilon:  0.13
episode:  2049  score:  213.71  avg_score:  190.89  frames:  562  epsilon:  0.13
episode:  2050  score:  255.72  avg_score:  191.13  frames:  343  epsilon:  0.13
episode:  2051  score:  253.52  avg_score:  193.61  frames:  210  epsilon:  0.13
episode:  2052  score:  260.08  avg_score:  193.94  frames:  262  epsilon:  0.13
episode:  2053  score:  29.51  avg_score:  191.86  frames:  88  epsilon:  0.13
episode:  2054  score:  258.54  avg_score:  192.05  frames:  324  epsilon:  0.13
episode:  2055  score:  239.33  avg_score:  192.01  frames:  222  epsilon:  0.13
episode:  2056  score:  264.46  avg_score:  192.06  frames:  401  epsilon:  0.13
episode:  2057  score:  308.56  avg_score:  192.96  frames:  361  epsilon:  0.13
episode:  2058  score:  257.49  avg_score:  192.95  frames:  786  epsilon:  0.13
episode:  2059  score:  30.92  avg_score:  191.00  frames:  121  epsilon:  0.13
episode:  2060  score:  171.76  avg_score:  191.81  frames:  415  epsilon:  0.13
episode:  2061  score:  203.83  avg_score:  191.14  frames:  522  epsilon:  0.13
episode:  2062  score:  258.13  avg_score:  193.90  frames:  509  epsilon:  0.13
episode:  2063  score:  248.10  avg_score:  193.71  frames:  259  epsilon:  0.13
episode:  2064  score:  3.40  avg_score:  191.59  frames:  999  epsilon:  0.13
episode:  2065  score:  266.09  avg_score:  194.20  frames:  245  epsilon:  0.13
episode:  2066  score:  252.01  avg_score:  194.33  frames:  254  epsilon:  0.13
episode:  2067  score:  218.01  avg_score:  195.40  frames:  581  epsilon:  0.13
episode:  2068  score:  208.62  avg_score:  195.22  frames:  244  epsilon:  0.13
episode:  2069  score:  221.41  avg_score:  195.10  frames:  601  epsilon:  0.13
episode:  2070  score:  283.67  avg_score:  195.48  frames:  248  epsilon:  0.13
episode:  2071  score:  255.39  avg_score:  197.18  frames:  348  epsilon:  0.13
episode:  2072  score:  150.41  avg_score:  196.01  frames:  611  epsilon:  0.13
episode:  2073  score:  296.55  avg_score:  196.44  frames:  824  epsilon:  0.13
episode:  2074  score:  13.55  avg_score:  194.17  frames:  147  epsilon:  0.13
episode:  2075  score:  130.43  avg_score:  193.19  frames:  999  epsilon:  0.13
episode:  2076  score:  235.65  avg_score:  192.75  frames:  324  epsilon:  0.13
episode:  2077  score:  264.03  avg_score:  192.88  frames:  296  epsilon:  0.13
episode:  2078  score:  -257.49  avg_score:  187.48  frames:  116  epsilon:  0.13
episode:  2079  score:  184.34  avg_score:  189.04  frames:  491  epsilon:  0.13
episode:  2080  score:  250.62  avg_score:  188.90  frames:  250  epsilon:  0.13
episode:  2081  score:  258.20  avg_score:  188.63  frames:  224  epsilon:  0.12
episode:  2082  score:  294.95  avg_score:  189.73  frames:  623  epsilon:  0.12
episode:  2083  score:  255.16  avg_score:  189.56  frames:  388  epsilon:  0.12
episode:  2084  score:  265.33  avg_score:  189.61  frames:  176  epsilon:  0.12
episode:  2085  score:  172.98  avg_score:  191.22  frames:  324  epsilon:  0.12
episode:  2086  score:  175.59  avg_score:  190.13  frames:  328  epsilon:  0.12
episode:  2087  score:  232.03  avg_score:  190.36  frames:  326  epsilon:  0.12
episode:  2088  score:  211.94  avg_score:  190.12  frames:  339  epsilon:  0.12
episode:  2089  score:  214.47  avg_score:  190.02  frames:  965  epsilon:  0.12
episode:  2090  score:  227.38  avg_score:  189.63  frames:  382  epsilon:  0.12
episode:  2091  score:  194.77  avg_score:  189.42  frames:  353  epsilon:  0.12
episode:  2092  score:  148.52  avg_score:  190.59  frames:  474  epsilon:  0.12
episode:  2093  score:  254.41  avg_score:  190.93  frames:  506  epsilon:  0.12
episode:  2094  score:  228.64  avg_score:  190.35  frames:  321  epsilon:  0.12
episode:  2095  score:  274.06  avg_score:  190.75  frames:  211  epsilon:  0.12
episode:  2096  score:  239.24  avg_score:  192.17  frames:  639  epsilon:  0.12
episode:  2097  score:  196.07  avg_score:  191.58  frames:  343  epsilon:  0.12
episode:  2098  score:  13.82  avg_score:  189.13  frames:  144  epsilon:  0.12
episode:  2099  score:  248.50  avg_score:  189.26  frames:  496  epsilon:  0.12
episode:  2100  score:  262.21  avg_score:  189.30  frames:  295  epsilon:  0.12
episode:  2101  score:  213.87  avg_score:  188.53  frames:  326  epsilon:  0.12
episode:  2102  score:  250.85  avg_score:  188.56  frames:  363  epsilon:  0.12
episode:  2103  score:  249.69  avg_score:  188.38  frames:  446  epsilon:  0.12
episode:  2104  score:  224.56  avg_score:  188.50  frames:  341  epsilon:  0.12
episode:  2105  score:  203.72  avg_score:  188.08  frames:  303  epsilon:  0.12
episode:  2106  score:  264.80  avg_score:  188.17  frames:  708  epsilon:  0.12
episode:  2107  score:  252.08  avg_score:  190.61  frames:  204  epsilon:  0.12
episode:  2108  score:  220.87  avg_score:  190.29  frames:  338  epsilon:  0.12
episode:  2109  score:  253.09  avg_score:  190.23  frames:  295  epsilon:  0.12
episode:  2110  score:  218.72  avg_score:  189.92  frames:  473  epsilon:  0.12
episode:  2111  score:  229.84  avg_score:  189.57  frames:  338  epsilon:  0.12
episode:  2112  score:  248.55  avg_score:  189.41  frames:  475  epsilon:  0.12
episode:  2113  score:  247.66  avg_score:  189.57  frames:  234  epsilon:  0.12
episode:  2114  score:  220.07  avg_score:  188.94  frames:  511  epsilon:  0.12
episode:  2115  score:  247.90  avg_score:  189.00  frames:  227  epsilon:  0.12
episode:  2116  score:  279.78  avg_score:  191.75  frames:  296  epsilon:  0.12
episode:  2117  score:  269.35  avg_score:  191.71  frames:  227  epsilon:  0.12
episode:  2118  score:  207.59  avg_score:  193.90  frames:  429  epsilon:  0.12
episode:  2119  score:  232.42  avg_score:  196.29  frames:  712  epsilon:  0.12
episode:  2120  score:  242.69  avg_score:  195.86  frames:  198  epsilon:  0.12
episode:  2121  score:  239.62  avg_score:  197.82  frames:  400  epsilon:  0.12
episode:  2122  score:  240.91  avg_score:  200.34  frames:  324  epsilon:  0.12
episode:  2123  score:  -11.29  avg_score:  197.68  frames:  124  epsilon:  0.12
episode:  2124  score:  240.63  avg_score:  199.57  frames:  834  epsilon:  0.12
episode:  2125  score:  278.77  avg_score:  202.28  frames:  283  epsilon:  0.12
episode:  2126  score:  265.85  avg_score:  204.69  frames:  264  epsilon:  0.12
episode:  2127  score:  281.08  avg_score:  207.12  frames:  230  epsilon:  0.12
episode:  2128  score:  -161.22  avg_score:  203.63  frames:  459  epsilon:  0.12
episode:  2129  score:  13.83  avg_score:  204.04  frames:  109  epsilon:  0.12
episode:  2130  score:  232.44  avg_score:  206.39  frames:  273  epsilon:  0.12
episode:  2131  score:  264.65  avg_score:  206.54  frames:  685  epsilon:  0.12
episode:  2132  score:  287.13  avg_score:  207.12  frames:  264  epsilon:  0.12
episode:  2133  score:  240.32  avg_score:  206.91  frames:  239  epsilon:  0.12
episode:  2134  score:  -241.97  avg_score:  202.05  frames:  498  epsilon:  0.12
episode:  2135  score:  -2.63  avg_score:  201.98  frames:  319  epsilon:  0.12
episode:  2136  score:  256.98  avg_score:  202.09  frames:  219  epsilon:  0.12
episode:  2137  score:  270.17  avg_score:  205.94  frames:  465  epsilon:  0.12
episode:  2138  score:  266.68  avg_score:  205.98  frames:  318  epsilon:  0.12
episode:  2139  score:  -938.93  avg_score:  193.91  frames:  296  epsilon:  0.12
episode:  2140  score:  228.40  avg_score:  193.94  frames:  181  epsilon:  0.12
episode:  2141  score:  42.45  avg_score:  191.97  frames:  90  epsilon:  0.12
episode:  2142  score:  281.81  avg_score:  192.44  frames:  249  epsilon:  0.12
episode:  2143  score:  240.78  avg_score:  192.19  frames:  372  epsilon:  0.12
episode:  2144  score:  264.29  avg_score:  192.43  frames:  258  epsilon:  0.12
episode:  2145  score:  244.57  avg_score:  192.16  frames:  196  epsilon:  0.12
episode:  2146  score:  190.27  avg_score:  194.18  frames:  575  epsilon:  0.12
episode:  2147  score:  203.47  avg_score:  193.46  frames:  670  epsilon:  0.12
episode:  2148  score:  287.39  avg_score:  193.83  frames:  208  epsilon:  0.12
episode:  2149  score:  261.40  avg_score:  194.31  frames:  342  epsilon:  0.12
episode:  2150  score:  275.69  avg_score:  194.51  frames:  517  epsilon:  0.12
episode:  2151  score:  272.56  avg_score:  194.70  frames:  281  epsilon:  0.12
episode:  2152  score:  246.04  avg_score:  194.56  frames:  206  epsilon:  0.12
episode:  2153  score:  251.44  avg_score:  196.78  frames:  273  epsilon:  0.12
episode:  2154  score:  262.60  avg_score:  196.82  frames:  307  epsilon:  0.12
episode:  2155  score:  289.00  avg_score:  197.32  frames:  179  epsilon:  0.12
episode:  2156  score:  217.37  avg_score:  196.84  frames:  560  epsilon:  0.12
episode:  2157  score:  231.69  avg_score:  196.08  frames:  398  epsilon:  0.12
episode:  2158  score:  -17.25  avg_score:  193.33  frames:  97  epsilon:  0.12
episode:  2159  score:  -3.03  avg_score:  192.99  frames:  116  epsilon:  0.12
episode:  2160  score:  235.71  avg_score:  193.63  frames:  513  epsilon:  0.12
episode:  2161  score:  64.63  avg_score:  192.24  frames:  999  epsilon:  0.12
episode:  2162  score:  68.51  avg_score:  190.34  frames:  999  epsilon:  0.12
episode:  2163  score:  249.52  avg_score:  190.35  frames:  348  epsilon:  0.12
episode:  2164  score:  224.81  avg_score:  192.57  frames:  236  epsilon:  0.11
episode:  2165  score:  215.81  avg_score:  192.07  frames:  386  epsilon:  0.11
episode:  2166  score:  163.74  avg_score:  191.18  frames:  694  epsilon:  0.11
episode:  2167  score:  276.58  avg_score:  191.77  frames:  361  epsilon:  0.11
episode:  2168  score:  253.27  avg_score:  192.22  frames:  654  epsilon:  0.11
episode:  2169  score:  115.10  avg_score:  191.15  frames:  898  epsilon:  0.11
episode:  2170  score:  245.68  avg_score:  190.77  frames:  286  epsilon:  0.11
episode:  2171  score:  242.65  avg_score:  190.64  frames:  233  epsilon:  0.11
episode:  2172  score:  263.97  avg_score:  191.78  frames:  232  epsilon:  0.11
episode:  2173  score:  227.95  avg_score:  191.09  frames:  442  epsilon:  0.11
episode:  2174  score:  268.25  avg_score:  193.64  frames:  252  epsilon:  0.11
episode:  2175  score:  289.20  avg_score:  195.23  frames:  148  epsilon:  0.11
episode:  2176  score:  259.49  avg_score:  195.47  frames:  218  epsilon:  0.11
episode:  2177  score:  -5.48  avg_score:  192.77  frames:  719  epsilon:  0.11
episode:  2178  score:  195.01  avg_score:  197.30  frames:  510  epsilon:  0.11
episode:  2179  score:  252.73  avg_score:  197.98  frames:  380  epsilon:  0.11
episode:  2180  score:  259.44  avg_score:  198.07  frames:  554  epsilon:  0.11
episode:  2181  score:  176.33  avg_score:  197.25  frames:  587  epsilon:  0.11
episode:  2182  score:  -0.83  avg_score:  194.29  frames:  173  epsilon:  0.11
episode:  2183  score:  -29.72  avg_score:  191.44  frames:  72  epsilon:  0.11
episode:  2184  score:  278.44  avg_score:  191.58  frames:  166  epsilon:  0.11
episode:  2185  score:  198.83  avg_score:  191.83  frames:  451  epsilon:  0.11
episode:  2186  score:  253.49  avg_score:  192.61  frames:  311  epsilon:  0.11
episode:  2187  score:  -25.21  avg_score:  190.04  frames:  102  epsilon:  0.11
episode:  2188  score:  219.60  avg_score:  190.12  frames:  204  epsilon:  0.11
episode:  2189  score:  21.40  avg_score:  188.19  frames:  92  epsilon:  0.11
episode:  2190  score:  205.48  avg_score:  187.97  frames:  413  epsilon:  0.11
episode:  2191  score:  261.34  avg_score:  188.63  frames:  310  epsilon:  0.11
episode:  2192  score:  290.61  avg_score:  190.05  frames:  290  epsilon:  0.11
episode:  2193  score:  276.42  avg_score:  190.27  frames:  234  epsilon:  0.11
episode:  2194  score:  258.80  avg_score:  190.58  frames:  339  epsilon:  0.11
episode:  2195  score:  247.22  avg_score:  190.31  frames:  387  epsilon:  0.11
episode:  2196  score:  252.14  avg_score:  190.44  frames:  291  epsilon:  0.11
episode:  2197  score:  212.35  avg_score:  190.60  frames:  222  epsilon:  0.11
episode:  2198  score:  257.84  avg_score:  193.04  frames:  360  epsilon:  0.11
episode:  2199  score:  5.67  avg_score:  190.61  frames:  999  epsilon:  0.11
episode:  2200  score:  -219.08  avg_score:  185.80  frames:  318  epsilon:  0.11
episode:  2201  score:  149.81  avg_score:  185.16  frames:  999  epsilon:  0.11
episode:  2202  score:  277.34  avg_score:  185.42  frames:  331  epsilon:  0.11
episode:  2203  score:  7.09  avg_score:  183.00  frames:  172  epsilon:  0.11
episode:  2204  score:  246.59  avg_score:  183.22  frames:  336  epsilon:  0.11
episode:  2205  score:  285.37  avg_score:  184.03  frames:  195  epsilon:  0.11
episode:  2206  score:  231.92  avg_score:  183.70  frames:  618  epsilon:  0.11
episode:  2207  score:  240.94  avg_score:  183.59  frames:  286  epsilon:  0.11
episode:  2208  score:  208.21  avg_score:  183.47  frames:  556  epsilon:  0.11
episode:  2209  score:  220.81  avg_score:  183.14  frames:  749  epsilon:  0.11
episode:  2210  score:  104.69  avg_score:  182.00  frames:  929  epsilon:  0.11
episode:  2211  score:  213.97  avg_score:  181.84  frames:  464  epsilon:  0.11
episode:  2212  score:  281.84  avg_score:  182.18  frames:  294  epsilon:  0.11
episode:  2213  score:  189.80  avg_score:  181.60  frames:  490  epsilon:  0.11
episode:  2214  score:  195.12  avg_score:  181.35  frames:  420  epsilon:  0.11
episode:  2215  score:  185.38  avg_score:  180.72  frames:  487  epsilon:  0.11
episode:  2216  score:  217.24  avg_score:  180.10  frames:  563  epsilon:  0.11
episode:  2217  score:  229.48  avg_score:  179.70  frames:  416  epsilon:  0.11
episode:  2218  score:  209.64  avg_score:  179.72  frames:  370  epsilon:  0.11
episode:  2219  score:  203.60  avg_score:  179.43  frames:  447  epsilon:  0.11
episode:  2220  score:  219.49  avg_score:  179.20  frames:  293  epsilon:  0.11
episode:  2221  score:  213.73  avg_score:  178.94  frames:  290  epsilon:  0.11
episode:  2222  score:  278.68  avg_score:  179.32  frames:  219  epsilon:  0.11
episode:  2223  score:  196.50  avg_score:  181.40  frames:  666  epsilon:  0.11
episode:  2224  score:  152.81  avg_score:  180.52  frames:  641  epsilon:  0.11
episode:  2225  score:  23.13  avg_score:  177.96  frames:  149  epsilon:  0.11
episode:  2226  score:  290.54  avg_score:  178.21  frames:  251  epsilon:  0.11
episode:  2227  score:  237.08  avg_score:  177.77  frames:  229  epsilon:  0.11
episode:  2228  score:  200.03  avg_score:  181.38  frames:  373  epsilon:  0.11
episode:  2229  score:  225.65  avg_score:  183.50  frames:  323  epsilon:  0.11
episode:  2230  score:  256.61  avg_score:  183.74  frames:  370  epsilon:  0.11
episode:  2231  score:  139.82  avg_score:  182.49  frames:  800  epsilon:  0.11
episode:  2232  score:  232.61  avg_score:  181.95  frames:  243  epsilon:  0.11
episode:  2233  score:  253.82  avg_score:  182.08  frames:  222  epsilon:  0.11
episode:  2234  score:  227.53  avg_score:  186.78  frames:  413  epsilon:  0.11
episode:  2235  score:  210.48  avg_score:  188.91  frames:  473  epsilon:  0.11
episode:  2236  score:  248.36  avg_score:  188.82  frames:  311  epsilon:  0.11
episode:  2237  score:  214.75  avg_score:  188.27  frames:  374  epsilon:  0.11
episode:  2238  score:  244.56  avg_score:  188.05  frames:  162  epsilon:  0.11
episode:  2239  score:  251.68  avg_score:  199.95  frames:  498  epsilon:  0.11
episode:  2240  score:  -228.40  avg_score:  195.39  frames:  399  epsilon:  0.11
episode:  2241  score:  269.34  avg_score:  197.65  frames:  238  epsilon:  0.11
episode:  2242  score:  199.09  avg_score:  196.83  frames:  385  epsilon:  0.11
episode:  2243  score:  201.10  avg_score:  196.43  frames:  303  epsilon:  0.11
episode:  2244  score:  -7.19  avg_score:  193.72  frames:  62  epsilon:  0.11
episode:  2245  score:  229.11  avg_score:  193.56  frames:  323  epsilon:  0.11
episode:  2246  score:  227.65  avg_score:  193.93  frames:  576  epsilon:  0.11
episode:  2247  score:  260.61  avg_score:  194.51  frames:  232  epsilon:  0.11
episode:  2248  score:  264.81  avg_score:  194.28  frames:  290  epsilon:  0.11
episode:  2249  score:  245.78  avg_score:  194.12  frames:  363  epsilon:  0.11
episode:  2250  score:  131.59  avg_score:  192.68  frames:  737  epsilon:  0.11
episode:  2251  score:  253.24  avg_score:  192.49  frames:  233  epsilon:  0.11
episode:  2252  score:  257.56  avg_score:  192.61  frames:  318  epsilon:  0.11
episode:  2253  score:  251.24  avg_score:  192.60  frames:  342  epsilon:  0.11
episode:  2254  score:  199.85  avg_score:  191.98  frames:  272  epsilon:  0.11
episode:  2255  score:  257.42  avg_score:  191.66  frames:  394  epsilon:  0.10
episode:  2256  score:  285.25  avg_score:  192.34  frames:  484  epsilon:  0.10
episode:  2257  score:  193.97  avg_score:  191.96  frames:  248  epsilon:  0.10
episode:  2258  score:  8.09  avg_score:  192.21  frames:  152  epsilon:  0.10
episode:  2259  score:  230.85  avg_score:  194.55  frames:  482  epsilon:  0.10
episode:  2260  score:  261.04  avg_score:  194.81  frames:  315  epsilon:  0.10
episode:  2261  score:  230.31  avg_score:  196.46  frames:  602  epsilon:  0.10
episode:  2262  score:  242.16  avg_score:  198.20  frames:  207  epsilon:  0.10
episode:  2263  score:  226.33  avg_score:  197.97  frames:  264  epsilon:  0.10
episode:  2264  score:  230.77  avg_score:  198.03  frames:  230  epsilon:  0.10
episode:  2265  score:  42.03  avg_score:  196.29  frames:  999  epsilon:  0.10
episode:  2266  score:  211.36  avg_score:  196.77  frames:  755  epsilon:  0.10
episode:  2267  score:  222.62  avg_score:  196.23  frames:  288  epsilon:  0.10
episode:  2268  score:  202.41  avg_score:  195.72  frames:  851  epsilon:  0.10
episode:  2269  score:  217.08  avg_score:  196.74  frames:  322  epsilon:  0.10
episode:  2270  score:  235.58  avg_score:  196.64  frames:  341  epsilon:  0.10
episode:  2271  score:  209.06  avg_score:  196.30  frames:  332  epsilon:  0.10
episode:  2272  score:  232.97  avg_score:  195.99  frames:  437  epsilon:  0.10
episode:  2273  score:  134.26  avg_score:  195.05  frames:  999  epsilon:  0.10
episode:  2274  score:  247.12  avg_score:  194.84  frames:  267  epsilon:  0.10
episode:  2275  score:  220.81  avg_score:  194.16  frames:  296  epsilon:  0.10
episode:  2276  score:  222.58  avg_score:  193.79  frames:  583  epsilon:  0.10
episode:  2277  score:  107.44  avg_score:  194.92  frames:  999  epsilon:  0.10
episode:  2278  score:  224.75  avg_score:  195.22  frames:  407  epsilon:  0.10
episode:  2279  score:  251.81  avg_score:  195.21  frames:  262  epsilon:  0.10
episode:  2280  score:  252.29  avg_score:  195.14  frames:  277  epsilon:  0.10
episode:  2281  score:  253.89  avg_score:  195.91  frames:  337  epsilon:  0.10
episode:  2282  score:  142.13  avg_score:  197.34  frames:  999  epsilon:  0.10
episode:  2283  score:  11.65  avg_score:  197.75  frames:  169  epsilon:  0.10
episode:  2284  score:  267.45  avg_score:  197.64  frames:  258  epsilon:  0.10
episode:  2285  score:  243.29  avg_score:  198.09  frames:  383  epsilon:  0.10
episode:  2286  score:  285.29  avg_score:  198.41  frames:  183  epsilon:  0.10
episode:  2287  score:  198.68  avg_score:  200.65  frames:  647  epsilon:  0.10
episode:  2288  score:  12.56  avg_score:  198.58  frames:  118  epsilon:  0.10
episode:  2289  score:  240.48  avg_score:  200.77  frames:  428  epsilon:  0.10
episode:  2290  score:  34.42  avg_score:  199.06  frames:  128  epsilon:  0.10
episode:  2291  score:  184.57  avg_score:  198.29  frames:  396  epsilon:  0.10
episode:  2292  score:  241.01  avg_score:  197.79  frames:  282  epsilon:  0.10
episode:  2293  score:  253.75  avg_score:  197.57  frames:  208  epsilon:  0.10
episode:  2294  score:  256.68  avg_score:  197.54  frames:  236  epsilon:  0.10
episode:  2295  score:  285.50  avg_score:  197.93  frames:  226  epsilon:  0.10
episode:  2296  score:  212.18  avg_score:  197.53  frames:  348  epsilon:  0.10
episode:  2297  score:  251.96  avg_score:  197.92  frames:  294  epsilon:  0.10
episode:  2298  score:  203.24  avg_score:  197.38  frames:  686  epsilon:  0.10
episode:  2299  score:  238.36  avg_score:  199.70  frames:  343  epsilon:  0.10
episode:  2300  score:  237.48  avg_score:  204.27  frames:  259  epsilon:  0.10
episode:  2301  score:  212.39  avg_score:  204.90  frames:  325  epsilon:  0.10
episode:  2302  score:  238.92  avg_score:  204.51  frames:  348  epsilon:  0.10
episode:  2303  score:  254.32  avg_score:  206.98  frames:  157  epsilon:  0.10
episode:  2304  score:  215.87  avg_score:  206.68  frames:  376  epsilon:  0.10
episode:  2305  score:  263.98  avg_score:  206.46  frames:  317  epsilon:  0.10
episode:  2306  score:  276.35  avg_score:  206.91  frames:  391  epsilon:  0.10
episode:  2307  score:  182.41  avg_score:  206.32  frames:  374  epsilon:  0.10
episode:  2308  score:  286.11  avg_score:  207.10  frames:  242  epsilon:  0.10
episode:  2309  score:  241.37  avg_score:  207.31  frames:  203  epsilon:  0.10
episode:  2310  score:  24.98  avg_score:  206.51  frames:  97  epsilon:  0.10
episode:  2311  score:  159.67  avg_score:  205.97  frames:  353  epsilon:  0.10
episode:  2312  score:  247.28  avg_score:  205.62  frames:  293  epsilon:  0.10
episode:  2313  score:  196.13  avg_score:  205.68  frames:  294  epsilon:  0.10
episode:  2314  score:  210.54  avg_score:  205.84  frames:  433  epsilon:  0.10
episode:  2315  score:  241.50  avg_score:  206.40  frames:  186  epsilon:  0.10
episode:  2316  score:  245.54  avg_score:  206.68  frames:  188  epsilon:  0.10
episode:  2317  score:  234.99  avg_score:  206.74  frames:  250  epsilon:  0.10
episode:  2318  score:  269.98  avg_score:  207.34  frames:  256  epsilon:  0.10
episode:  2319  score:  250.77  avg_score:  207.81  frames:  208  epsilon:  0.10
episode:  2320  score:  271.78  avg_score:  208.34  frames:  150  epsilon:  0.10
episode:  2321  score:  15.24  avg_score:  206.35  frames:  170  epsilon:  0.10
episode:  2322  score:  18.50  avg_score:  203.75  frames:  113  epsilon:  0.10
episode:  2323  score:  237.93  avg_score:  204.16  frames:  364  epsilon:  0.10
episode:  2324  score:  257.16  avg_score:  205.21  frames:  196  epsilon:  0.10
episode:  2325  score:  235.21  avg_score:  207.33  frames:  265  epsilon:  0.10
episode:  2326  score:  216.53  avg_score:  206.59  frames:  377  epsilon:  0.10
episode:  2327  score:  248.93  avg_score:  206.71  frames:  145  epsilon:  0.10
episode:  2328  score:  18.89  avg_score:  204.89  frames:  130  epsilon:  0.10
episode:  2329  score:  260.80  avg_score:  205.25  frames:  236  epsilon:  0.10
episode:  2330  score:  254.91  avg_score:  205.23  frames:  337  epsilon:  0.10
episode:  2331  score:  265.87  avg_score:  206.49  frames:  182  epsilon:  0.10
episode:  2332  score:  269.79  avg_score:  206.86  frames:  493  epsilon:  0.10
episode:  2333  score:  270.11  avg_score:  207.02  frames:  510  epsilon:  0.10
episode:  2334  score:  204.93  avg_score:  206.80  frames:  607  epsilon:  0.10
episode:  2335  score:  216.19  avg_score:  206.86  frames:  343  epsilon:  0.10
episode:  2336  score:  267.11  avg_score:  207.04  frames:  311  epsilon:  0.10
episode:  2337  score:  20.05  avg_score:  205.10  frames:  245  epsilon:  0.10
episode:  2338  score:  281.11  avg_score:  205.46  frames:  333  epsilon:  0.10
episode:  2339  score:  244.01  avg_score:  205.38  frames:  223  epsilon:  0.10
episode:  2340  score:  48.12  avg_score:  208.15  frames:  109  epsilon:  0.10
episode:  2341  score:  147.24  avg_score:  206.93  frames:  773  epsilon:  0.10
episode:  2342  score:  264.14  avg_score:  207.58  frames:  206  epsilon:  0.10
episode:  2343  score:  260.28  avg_score:  208.17  frames:  219  epsilon:  0.10
episode:  2344  score:  41.99  avg_score:  208.66  frames:  128  epsilon:  0.10
episode:  2345  score:  239.67  avg_score:  208.77  frames:  375  epsilon:  0.10
episode:  2346  score:  228.29  avg_score:  208.78  frames:  181  epsilon:  0.10
episode:  2347  score:  252.16  avg_score:  208.69  frames:  328  epsilon:  0.10
episode:  2348  score:  209.90  avg_score:  208.14  frames:  835  epsilon:  0.10
episode:  2349  score:  233.38  avg_score:  208.02  frames:  651  epsilon:  0.10
episode:  2350  score:  227.25  avg_score:  208.97  frames:  333  epsilon:  0.10
episode:  2351  score:  239.99  avg_score:  208.84  frames:  188  epsilon:  0.10
episode:  2352  score:  265.96  avg_score:  208.93  frames:  195  epsilon:  0.10
episode:  2353  score:  226.73  avg_score:  208.68  frames:  307  epsilon:  0.10
episode:  2354  score:  241.03  avg_score:  209.09  frames:  214  epsilon:  0.10
episode:  2355  score:  251.72  avg_score:  209.04  frames:  209  epsilon:  0.09
episode:  2356  score:  221.37  avg_score:  208.40  frames:  350  epsilon:  0.09
episode:  2357  score:  255.86  avg_score:  209.02  frames:  201  epsilon:  0.09
episode:  2358  score:  31.07  avg_score:  209.25  frames:  140  epsilon:  0.09
episode:  2359  score:  227.92  avg_score:  209.22  frames:  391  epsilon:  0.09
episode:  2360  score:  238.65  avg_score:  208.99  frames:  494  epsilon:  0.09
episode:  2361  score:  284.43  avg_score:  209.53  frames:  299  epsilon:  0.09
episode:  2362  score:  232.32  avg_score:  209.44  frames:  271  epsilon:  0.09
episode:  2363  score:  243.61  avg_score:  209.61  frames:  242  epsilon:  0.09
episode:  2364  score:  234.39  avg_score:  209.64  frames:  210  epsilon:  0.09
episode:  2365  score:  250.67  avg_score:  211.73  frames:  154  epsilon:  0.09
episode:  2366  score:  232.22  avg_score:  211.94  frames:  525  epsilon:  0.09
episode:  2367  score:  248.04  avg_score:  212.19  frames:  224  epsilon:  0.09
episode:  2368  score:  19.07  avg_score:  210.36  frames:  98  epsilon:  0.09
episode:  2369  score:  25.78  avg_score:  208.45  frames:  119  epsilon:  0.09
episode:  2370  score:  37.13  avg_score:  206.46  frames:  136  epsilon:  0.09
episode:  2371  score:  240.66  avg_score:  206.78  frames:  304  epsilon:  0.09
episode:  2372  score:  210.13  avg_score:  206.55  frames:  437  epsilon:  0.09
episode:  2373  score:  295.55  avg_score:  208.16  frames:  289  epsilon:  0.09
episode:  2374  score:  210.31  avg_score:  207.79  frames:  608  epsilon:  0.09
episode:  2375  score:  262.57  avg_score:  208.21  frames:  217  epsilon:  0.09
episode:  2376  score:  190.65  avg_score:  207.89  frames:  197  epsilon:  0.09
episode:  2377  score:  251.77  avg_score:  209.34  frames:  187  epsilon:  0.09
episode:  2378  score:  264.72  avg_score:  209.74  frames:  270  epsilon:  0.09
episode:  2379  score:  234.92  avg_score:  209.57  frames:  403  epsilon:  0.09
episode:  2380  score:  286.31  avg_score:  209.91  frames:  387  epsilon:  0.09
episode:  2381  score:  10.03  avg_score:  207.47  frames:  126  epsilon:  0.09
episode:  2382  score:  230.74  avg_score:  208.35  frames:  578  epsilon:  0.09
episode:  2383  score:  254.03  avg_score:  210.78  frames:  249  epsilon:  0.09
episode:  2384  score:  231.86  avg_score:  210.42  frames:  238  epsilon:  0.09
episode:  2385  score:  38.02  avg_score:  208.37  frames:  104  epsilon:  0.09
episode:  2386  score:  217.76  avg_score:  207.69  frames:  246  epsilon:  0.09
episode:  2387  score:  261.04  avg_score:  208.32  frames:  169  epsilon:  0.09
episode:  2388  score:  251.10  avg_score:  210.70  frames:  795  epsilon:  0.09
episode:  2389  score:  244.40  avg_score:  210.74  frames:  277  epsilon:  0.09
episode:  2390  score:  264.52  avg_score:  213.04  frames:  179  epsilon:  0.09
episode:  2391  score:  17.47  avg_score:  211.37  frames:  375  epsilon:  0.09
episode:  2392  score:  235.78  avg_score:  211.32  frames:  388  epsilon:  0.09
episode:  2393  score:  264.40  avg_score:  211.43  frames:  211  epsilon:  0.09
episode:  2394  score:  227.52  avg_score:  211.14  frames:  261  epsilon:  0.09
episode:  2395  score:  274.39  avg_score:  211.02  frames:  299  epsilon:  0.09
episode:  2396  score:  274.02  avg_score:  211.64  frames:  132  epsilon:  0.09
episode:  2397  score:  248.55  avg_score:  211.61  frames:  368  epsilon:  0.09
episode:  2398  score:  276.00  avg_score:  212.34  frames:  698  epsilon:  0.09
episode:  2399  score:  206.50  avg_score:  212.02  frames:  394  epsilon:  0.09
episode:  2400  score:  255.02  avg_score:  212.19  frames:  213  epsilon:  0.09
episode:  2401  score:  287.11  avg_score:  212.94  frames:  473  epsilon:  0.09
episode:  2402  score:  257.32  avg_score:  213.12  frames:  361  epsilon:  0.09
episode:  2403  score:  272.45  avg_score:  213.31  frames:  341  epsilon:  0.09
episode:  2404  score:  198.51  avg_score:  213.13  frames:  482  epsilon:  0.09
episode:  2405  score:  220.75  avg_score:  212.70  frames:  384  epsilon:  0.09
episode:  2406  score:  186.18  avg_score:  211.80  frames:  253  epsilon:  0.09
episode:  2407  score:  8.50  avg_score:  210.06  frames:  125  epsilon:  0.09
episode:  2408  score:  254.28  avg_score:  209.74  frames:  211  epsilon:  0.09
episode:  2409  score:  248.64  avg_score:  209.81  frames:  279  epsilon:  0.09
episode:  2410  score:  279.04  avg_score:  212.35  frames:  191  epsilon:  0.09
episode:  2411  score:  250.16  avg_score:  213.26  frames:  174  epsilon:  0.09
episode:  2412  score:  263.85  avg_score:  213.42  frames:  393  epsilon:  0.09
episode:  2413  score:  120.43  avg_score:  212.67  frames:  999  epsilon:  0.09
episode:  2414  score:  233.94  avg_score:  212.90  frames:  406  epsilon:  0.09
episode:  2415  score:  237.62  avg_score:  212.86  frames:  291  epsilon:  0.09
episode:  2416  score:  197.67  avg_score:  212.38  frames:  771  epsilon:  0.09
episode:  2417  score:  243.94  avg_score:  212.47  frames:  301  epsilon:  0.09
episode:  2418  score:  220.15  avg_score:  211.98  frames:  262  epsilon:  0.09
episode:  2419  score:  126.08  avg_score:  210.73  frames:  999  epsilon:  0.09
episode:  2420  score:  260.99  avg_score:  210.62  frames:  248  epsilon:  0.09
episode:  2421  score:  246.28  avg_score:  212.93  frames:  202  epsilon:  0.09
episode:  2422  score:  42.88  avg_score:  213.17  frames:  103  epsilon:  0.09
episode:  2423  score:  277.53  avg_score:  213.57  frames:  232  epsilon:  0.09
episode:  2424  score:  262.08  avg_score:  213.62  frames:  183  epsilon:  0.09
episode:  2425  score:  270.19  avg_score:  213.97  frames:  239  epsilon:  0.09
episode:  2426  score:  246.27  avg_score:  214.27  frames:  491  epsilon:  0.09
episode:  2427  score:  43.46  avg_score:  212.21  frames:  79  epsilon:  0.09
episode:  2428  score:  242.87  avg_score:  214.45  frames:  235  epsilon:  0.09
episode:  2429  score:  40.36  avg_score:  212.25  frames:  82  epsilon:  0.09
episode:  2430  score:  240.33  avg_score:  212.10  frames:  208  epsilon:  0.09
episode:  2431  score:  240.42  avg_score:  211.85  frames:  247  epsilon:  0.09
episode:  2432  score:  248.46  avg_score:  211.63  frames:  143  epsilon:  0.09
episode:  2433  score:  238.01  avg_score:  211.31  frames:  281  epsilon:  0.09
episode:  2434  score:  12.56  avg_score:  209.39  frames:  111  epsilon:  0.09
episode:  2435  score:  -3.35  avg_score:  207.19  frames:  111  epsilon:  0.09
episode:  2436  score:  -31.00  avg_score:  204.21  frames:  102  epsilon:  0.09
episode:  2437  score:  254.61  avg_score:  206.56  frames:  304  epsilon:  0.09
episode:  2438  score:  18.88  avg_score:  203.94  frames:  117  epsilon:  0.09
episode:  2439  score:  206.79  avg_score:  203.56  frames:  302  epsilon:  0.09
episode:  2440  score:  46.70  avg_score:  203.55  frames:  88  epsilon:  0.09
episode:  2441  score:  19.17  avg_score:  202.27  frames:  76  epsilon:  0.09
episode:  2442  score:  2.54  avg_score:  199.65  frames:  92  epsilon:  0.09
episode:  2443  score:  47.54  avg_score:  197.53  frames:  88  epsilon:  0.09
episode:  2444  score:  54.46  avg_score:  197.65  frames:  96  epsilon:  0.09
episode:  2445  score:  228.41  avg_score:  197.54  frames:  214  epsilon:  0.09
episode:  2446  score:  265.70  avg_score:  197.91  frames:  200  epsilon:  0.09
episode:  2447  score:  32.61  avg_score:  195.72  frames:  142  epsilon:  0.09
episode:  2448  score:  195.46  avg_score:  195.57  frames:  377  epsilon:  0.09
episode:  2449  score:  29.56  avg_score:  193.53  frames:  112  epsilon:  0.09
episode:  2450  score:  282.19  avg_score:  194.08  frames:  273  epsilon:  0.09
episode:  2451  score:  2.38  avg_score:  191.71  frames:  163  epsilon:  0.09
episode:  2452  score:  271.20  avg_score:  191.76  frames:  240  epsilon:  0.09
episode:  2453  score:  269.82  avg_score:  192.19  frames:  298  epsilon:  0.09
episode:  2454  score:  42.10  avg_score:  190.20  frames:  119  epsilon:  0.09
episode:  2455  score:  38.18  avg_score:  188.07  frames:  98  epsilon:  0.09
episode:  2456  score:  268.12  avg_score:  188.53  frames:  283  epsilon:  0.09
episode:  2457  score:  231.92  avg_score:  188.29  frames:  434  epsilon:  0.09
episode:  2458  score:  253.11  avg_score:  190.51  frames:  239  epsilon:  0.09
episode:  2459  score:  -140.82  avg_score:  186.83  frames:  395  epsilon:  0.09
episode:  2460  score:  120.02  avg_score:  185.64  frames:  999  epsilon:  0.09
episode:  2461  score:  281.67  avg_score:  185.61  frames:  378  epsilon:  0.09
episode:  2462  score:  235.32  avg_score:  185.64  frames:  251  epsilon:  0.09
episode:  2463  score:  -10.70  avg_score:  183.10  frames:  999  epsilon:  0.09
episode:  2464  score:  28.06  avg_score:  181.04  frames:  999  epsilon:  0.09
episode:  2465  score:  233.49  avg_score:  180.86  frames:  480  epsilon:  0.09
episode:  2466  score:  230.53  avg_score:  180.85  frames:  699  epsilon:  0.09
episode:  2467  score:  249.32  avg_score:  180.86  frames:  344  epsilon:  0.08
episode:  2468  score:  212.30  avg_score:  182.79  frames:  341  epsilon:  0.08
episode:  2469  score:  -7.73  avg_score:  182.46  frames:  999  epsilon:  0.08
episode:  2470  score:  227.53  avg_score:  184.36  frames:  282  epsilon:  0.08
episode:  2471  score:  225.39  avg_score:  184.21  frames:  311  epsilon:  0.08
episode:  2472  score:  -22.71  avg_score:  181.88  frames:  128  epsilon:  0.08
episode:  2473  score:  20.60  avg_score:  179.13  frames:  182  epsilon:  0.08
episode:  2474  score:  132.24  avg_score:  178.35  frames:  973  epsilon:  0.08
episode:  2475  score:  -5.06  avg_score:  175.67  frames:  999  epsilon:  0.08
episode:  2476  score:  218.80  avg_score:  175.96  frames:  643  epsilon:  0.08
episode:  2477  score:  150.60  avg_score:  174.94  frames:  959  epsilon:  0.08
episode:  2478  score:  172.24  avg_score:  174.02  frames:  824  epsilon:  0.08
episode:  2479  score:  104.18  avg_score:  172.71  frames:  999  epsilon:  0.08
episode:  2480  score:  223.03  avg_score:  172.08  frames:  317  epsilon:  0.08
episode:  2481  score:  250.60  avg_score:  174.48  frames:  281  epsilon:  0.08
episode:  2482  score:  196.31  avg_score:  174.14  frames:  461  epsilon:  0.08
episode:  2483  score:  121.36  avg_score:  172.81  frames:  999  epsilon:  0.08
episode:  2484  score:  149.05  avg_score:  171.99  frames:  692  epsilon:  0.08
episode:  2485  score:  235.29  avg_score:  173.96  frames:  332  epsilon:  0.08
episode:  2486  score:  260.03  avg_score:  174.38  frames:  380  epsilon:  0.08
episode:  2487  score:  202.04  avg_score:  173.79  frames:  284  epsilon:  0.08
episode:  2488  score:  128.62  avg_score:  172.57  frames:  999  epsilon:  0.08
episode:  2489  score:  243.02  avg_score:  172.55  frames:  313  epsilon:  0.08
episode:  2490  score:  260.59  avg_score:  172.51  frames:  222  epsilon:  0.08
episode:  2491  score:  262.43  avg_score:  174.96  frames:  194  epsilon:  0.08
episode:  2492  score:  225.90  avg_score:  174.86  frames:  197  epsilon:  0.08
episode:  2493  score:  204.43  avg_score:  174.26  frames:  443  epsilon:  0.08
episode:  2494  score:  220.53  avg_score:  174.19  frames:  634  epsilon:  0.08
episode:  2495  score:  251.80  avg_score:  173.97  frames:  242  epsilon:  0.08
episode:  2496  score:  4.18  avg_score:  171.27  frames:  96  epsilon:  0.08
episode:  2497  score:  249.96  avg_score:  171.28  frames:  233  epsilon:  0.08
episode:  2498  score:  271.40  avg_score:  171.24  frames:  320  epsilon:  0.08
episode:  2499  score:  255.07  avg_score:  171.72  frames:  364  epsilon:  0.08
episode:  2500  score:  256.90  avg_score:  171.74  frames:  779  epsilon:  0.08
episode:  2501  score:  119.07  avg_score:  170.06  frames:  803  epsilon:  0.08
episode:  2502  score:  251.64  avg_score:  170.00  frames:  154  epsilon:  0.08
episode:  2503  score:  258.44  avg_score:  169.86  frames:  234  epsilon:  0.08
episode:  2504  score:  244.33  avg_score:  170.32  frames:  211  epsilon:  0.08
episode:  2505  score:  238.72  avg_score:  170.50  frames:  316  epsilon:  0.08
episode:  2506  score:  242.26  avg_score:  171.06  frames:  238  epsilon:  0.08
episode:  2507  score:  140.81  avg_score:  172.39  frames:  999  epsilon:  0.08
episode:  2508  score:  47.63  avg_score:  170.32  frames:  117  epsilon:  0.08
episode:  2509  score:  216.27  avg_score:  170.00  frames:  365  epsilon:  0.08
episode:  2510  score:  251.12  avg_score:  169.72  frames:  341  epsilon:  0.08
episode:  2511  score:  261.38  avg_score:  169.83  frames:  371  epsilon:  0.08
episode:  2512  score:  260.29  avg_score:  169.79  frames:  214  epsilon:  0.08
episode:  2513  score:  255.42  avg_score:  171.14  frames:  300  epsilon:  0.08
episode:  2514  score:  221.81  avg_score:  171.02  frames:  673  epsilon:  0.08
episode:  2515  score:  262.19  avg_score:  171.27  frames:  239  epsilon:  0.08
episode:  2516  score:  261.32  avg_score:  171.90  frames:  247  epsilon:  0.08
episode:  2517  score:  243.02  avg_score:  171.90  frames:  318  epsilon:  0.08
episode:  2518  score:  289.27  avg_score:  172.59  frames:  194  epsilon:  0.08
episode:  2519  score:  247.59  avg_score:  173.80  frames:  241  epsilon:  0.08
episode:  2520  score:  246.53  avg_score:  173.66  frames:  489  epsilon:  0.08
episode:  2521  score:  248.34  avg_score:  173.68  frames:  222  epsilon:  0.08
episode:  2522  score:  262.21  avg_score:  175.87  frames:  183  epsilon:  0.08
episode:  2523  score:  267.67  avg_score:  175.77  frames:  239  epsilon:  0.08
episode:  2524  score:  268.61  avg_score:  175.84  frames:  158  epsilon:  0.08
episode:  2525  score:  201.51  avg_score:  175.15  frames:  346  epsilon:  0.08
episode:  2526  score:  229.77  avg_score:  174.99  frames:  538  epsilon:  0.08
episode:  2527  score:  197.91  avg_score:  176.53  frames:  555  epsilon:  0.08
episode:  2528  score:  177.43  avg_score:  175.88  frames:  755  epsilon:  0.08
episode:  2529  score:  259.10  avg_score:  178.06  frames:  346  epsilon:  0.08
episode:  2530  score:  156.18  avg_score:  177.22  frames:  603  epsilon:  0.08
episode:  2531  score:  223.56  avg_score:  177.05  frames:  398  epsilon:  0.08
episode:  2532  score:  275.00  avg_score:  177.32  frames:  185  epsilon:  0.08
episode:  2533  score:  259.56  avg_score:  177.53  frames:  175  epsilon:  0.08
episode:  2534  score:  260.48  avg_score:  180.01  frames:  208  epsilon:  0.08
episode:  2535  score:  243.27  avg_score:  182.48  frames:  217  epsilon:  0.08
episode:  2536  score:  220.54  avg_score:  184.99  frames:  233  epsilon:  0.08
episode:  2537  score:  17.40  avg_score:  182.62  frames:  172  epsilon:  0.08
episode:  2538  score:  286.94  avg_score:  185.30  frames:  358  epsilon:  0.08
episode:  2539  score:  251.19  avg_score:  185.75  frames:  237  epsilon:  0.08
episode:  2540  score:  223.63  avg_score:  187.52  frames:  720  epsilon:  0.08
episode:  2541  score:  209.46  avg_score:  189.42  frames:  840  epsilon:  0.08
episode:  2542  score:  264.77  avg_score:  192.04  frames:  235  epsilon:  0.08
episode:  2543  score:  256.53  avg_score:  194.13  frames:  185  epsilon:  0.08
episode:  2544  score:  259.29  avg_score:  196.18  frames:  279  epsilon:  0.08
episode:  2545  score:  251.89  avg_score:  196.41  frames:  314  epsilon:  0.08
episode:  2546  score:  -6.73  avg_score:  193.69  frames:  169  epsilon:  0.08
episode:  2547  score:  276.68  avg_score:  196.13  frames:  302  epsilon:  0.08
episode:  2548  score:  282.44  avg_score:  197.00  frames:  345  epsilon:  0.08
episode:  2549  score:  249.74  avg_score:  199.20  frames:  271  epsilon:  0.08
episode:  2550  score:  268.52  avg_score:  199.07  frames:  198  epsilon:  0.08
episode:  2551  score:  289.55  avg_score:  201.94  frames:  155  epsilon:  0.08
episode:  2552  score:  251.49  avg_score:  201.74  frames:  166  epsilon:  0.08
episode:  2553  score:  280.16  avg_score:  201.84  frames:  208  epsilon:  0.08
episode:  2554  score:  224.72  avg_score:  203.67  frames:  302  epsilon:  0.08
episode:  2555  score:  258.08  avg_score:  205.87  frames:  251  epsilon:  0.08
episode:  2556  score:  240.59  avg_score:  205.59  frames:  352  epsilon:  0.08
episode:  2557  score:  262.90  avg_score:  205.90  frames:  263  epsilon:  0.08
episode:  2558  score:  293.41  avg_score:  206.31  frames:  226  epsilon:  0.08
episode:  2559  score:  240.06  avg_score:  210.12  frames:  169  epsilon:  0.08
episode:  2560  score:  155.40  avg_score:  210.47  frames:  999  epsilon:  0.08
episode:  2561  score:  253.01  avg_score:  210.18  frames:  279  epsilon:  0.08
episode:  2562  score:  253.28  avg_score:  210.36  frames:  183  epsilon:  0.08
episode:  2563  score:  302.73  avg_score:  213.50  frames:  533  epsilon:  0.08
episode:  2564  score:  26.23  avg_score:  213.48  frames:  124  epsilon:  0.08
episode:  2565  score:  244.75  avg_score:  213.59  frames:  186  epsilon:  0.08
episode:  2566  score:  -12.81  avg_score:  211.16  frames:  104  epsilon:  0.08
episode:  2567  score:  250.99  avg_score:  211.17  frames:  285  epsilon:  0.08
episode:  2568  score:  266.07  avg_score:  211.71  frames:  238  epsilon:  0.08
episode:  2569  score:  281.71  avg_score:  214.61  frames:  175  epsilon:  0.08
episode:  2570  score:  248.75  avg_score:  214.82  frames:  185  epsilon:  0.08
episode:  2571  score:  269.60  avg_score:  215.26  frames:  215  epsilon:  0.08
episode:  2572  score:  261.88  avg_score:  218.11  frames:  164  epsilon:  0.08
episode:  2573  score:  237.72  avg_score:  220.28  frames:  304  epsilon:  0.08
episode:  2574  score:  253.41  avg_score:  221.49  frames:  207  epsilon:  0.08
episode:  2575  score:  273.67  avg_score:  224.28  frames:  190  epsilon:  0.08
episode:  2576  score:  267.13  avg_score:  224.76  frames:  372  epsilon:  0.08
episode:  2577  score:  271.35  avg_score:  225.97  frames:  199  epsilon:  0.08
episode:  2578  score:  260.75  avg_score:  226.85  frames:  250  epsilon:  0.08
episode:  2579  score:  15.74  avg_score:  225.97  frames:  116  epsilon:  0.08
episode:  2580  score:  242.25  avg_score:  226.16  frames:  233  epsilon:  0.08
episode:  2581  score:  246.63  avg_score:  226.12  frames:  218  epsilon:  0.08
episode:  2582  score:  33.59  avg_score:  224.49  frames:  136  epsilon:  0.08
episode:  2583  score:  246.94  avg_score:  225.75  frames:  197  epsilon:  0.08
episode:  2584  score:  282.12  avg_score:  227.08  frames:  220  epsilon:  0.08
episode:  2585  score:  278.31  avg_score:  227.51  frames:  253  epsilon:  0.08
episode:  2586  score:  236.76  avg_score:  227.28  frames:  264  epsilon:  0.08
episode:  2587  score:  282.85  avg_score:  228.09  frames:  225  epsilon:  0.08
episode:  2588  score:  248.80  avg_score:  229.29  frames:  219  epsilon:  0.08
episode:  2589  score:  261.99  avg_score:  229.48  frames:  247  epsilon:  0.08
episode:  2590  score:  288.39  avg_score:  229.75  frames:  274  epsilon:  0.08
episode:  2591  score:  262.00  avg_score:  229.75  frames:  187  epsilon:  0.08
episode:  2592  score:  282.32  avg_score:  230.31  frames:  252  epsilon:  0.07
episode:  2593  score:  259.14  avg_score:  230.86  frames:  231  epsilon:  0.07
episode:  2594  score:  197.31  avg_score:  230.63  frames:  408  epsilon:  0.07
episode:  2595  score:  291.72  avg_score:  231.03  frames:  227  epsilon:  0.07
episode:  2596  score:  289.38  avg_score:  233.88  frames:  166  epsilon:  0.07
episode:  2597  score:  285.56  avg_score:  234.24  frames:  210  epsilon:  0.07
episode:  2598  score:  210.35  avg_score:  233.63  frames:  472  epsilon:  0.07
episode:  2599  score:  278.50  avg_score:  233.86  frames:  198  epsilon:  0.07
episode:  2600  score:  292.10  avg_score:  234.21  frames:  165  epsilon:  0.07
episode:  2601  score:  269.42  avg_score:  235.72  frames:  600  epsilon:  0.07
episode:  2602  score:  151.78  avg_score:  234.72  frames:  478  epsilon:  0.07
episode:  2603  score:  208.39  avg_score:  234.22  frames:  253  epsilon:  0.07
episode:  2604  score:  198.06  avg_score:  233.75  frames:  535  epsilon:  0.07
episode:  2605  score:  248.83  avg_score:  233.86  frames:  319  epsilon:  0.07
episode:  2606  score:  240.78  avg_score:  233.84  frames:  204  epsilon:  0.07
episode:  2607  score:  244.32  avg_score:  234.88  frames:  362  epsilon:  0.07
episode:  2608  score:  275.73  avg_score:  237.16  frames:  226  epsilon:  0.07
episode:  2609  score:  227.93  avg_score:  237.27  frames:  284  epsilon:  0.07
episode:  2610  score:  210.98  avg_score:  236.87  frames:  409  epsilon:  0.07
episode:  2611  score:  298.46  avg_score:  237.24  frames:  278  epsilon:  0.07
episode:  2612  score:  284.69  avg_score:  237.49  frames:  181  epsilon:  0.07
episode:  2613  score:  240.59  avg_score:  237.34  frames:  267  epsilon:  0.07
episode:  2614  score:  275.34  avg_score:  237.87  frames:  182  epsilon:  0.07
episode:  2615  score:  245.35  avg_score:  237.71  frames:  399  epsilon:  0.07
episode:  2616  score:  249.52  avg_score:  237.59  frames:  208  epsilon:  0.07
episode:  2617  score:  257.82  avg_score:  237.74  frames:  175  epsilon:  0.07
episode:  2618  score:  253.12  avg_score:  237.37  frames:  199  epsilon:  0.07
episode:  2619  score:  252.44  avg_score:  237.42  frames:  207  epsilon:  0.07
episode:  2620  score:  -30.44  avg_score:  234.65  frames:  62  epsilon:  0.07
episode:  2621  score:  243.57  avg_score:  234.61  frames:  269  epsilon:  0.07
episode:  2622  score:  253.36  avg_score:  234.52  frames:  312  epsilon:  0.07
episode:  2623  score:  20.79  avg_score:  232.05  frames:  143  epsilon:  0.07
episode:  2624  score:  252.54  avg_score:  231.89  frames:  389  epsilon:  0.07
episode:  2625  score:  264.57  avg_score:  232.52  frames:  222  epsilon:  0.07
episode:  2626  score:  221.18  avg_score:  232.43  frames:  260  epsilon:  0.07
episode:  2627  score:  -88.09  avg_score:  229.57  frames:  62  epsilon:  0.07
episode:  2628  score:  278.75  avg_score:  230.59  frames:  191  epsilon:  0.07
episode:  2629  score:  244.30  avg_score:  230.44  frames:  259  epsilon:  0.07
episode:  2630  score:  231.10  avg_score:  231.19  frames:  729  epsilon:  0.07
episode:  2631  score:  262.01  avg_score:  231.57  frames:  294  epsilon:  0.07
episode:  2632  score:  244.11  avg_score:  231.26  frames:  527  epsilon:  0.07
episode:  2633  score:  245.32  avg_score:  231.12  frames:  322  epsilon:  0.07
episode:  2634  score:  240.86  avg_score:  230.92  frames:  231  epsilon:  0.07
episode:  2635  score:  243.37  avg_score:  230.92  frames:  648  epsilon:  0.07
episode:  2636  score:  261.09  avg_score:  231.33  frames:  317  epsilon:  0.07
episode:  2637  score:  277.99  avg_score:  233.94  frames:  225  epsilon:  0.07
episode:  2638  score:  240.26  avg_score:  233.47  frames:  258  epsilon:  0.07
episode:  2639  score:  272.61  avg_score:  233.68  frames:  304  epsilon:  0.07
episode:  2640  score:  210.67  avg_score:  233.55  frames:  184  epsilon:  0.07
episode:  2641  score:  5.74  avg_score:  231.52  frames:  117  epsilon:  0.07
episode:  2642  score:  267.49  avg_score:  231.54  frames:  214  epsilon:  0.07
episode:  2643  score:  212.91  avg_score:  231.11  frames:  205  epsilon:  0.07
episode:  2644  score:  232.01  avg_score:  230.83  frames:  187  epsilon:  0.07
episode:  2645  score:  27.71  avg_score:  228.59  frames:  203  epsilon:  0.07
episode:  2646  score:  235.47  avg_score:  231.01  frames:  366  epsilon:  0.07
episode:  2647  score:  245.95  avg_score:  230.71  frames:  455  epsilon:  0.07
episode:  2648  score:  238.93  avg_score:  230.27  frames:  521  epsilon:  0.07
episode:  2649  score:  241.23  avg_score:  230.19  frames:  705  epsilon:  0.07
episode:  2650  score:  241.02  avg_score:  229.91  frames:  298  epsilon:  0.07
episode:  2651  score:  263.47  avg_score:  229.65  frames:  322  epsilon:  0.07
episode:  2652  score:  207.99  avg_score:  229.22  frames:  548  epsilon:  0.07
episode:  2653  score:  261.06  avg_score:  229.03  frames:  240  epsilon:  0.07
episode:  2654  score:  245.44  avg_score:  229.23  frames:  274  epsilon:  0.07
episode:  2655  score:  276.65  avg_score:  229.42  frames:  220  epsilon:  0.07
episode:  2656  score:  274.10  avg_score:  229.75  frames:  234  epsilon:  0.07
episode:  2657  score:  250.86  avg_score:  229.63  frames:  197  epsilon:  0.07
episode:  2658  score:  282.72  avg_score:  229.53  frames:  205  epsilon:  0.07
episode:  2659  score:  -100.13  avg_score:  226.12  frames:  66  epsilon:  0.07
episode:  2660  score:  265.75  avg_score:  227.23  frames:  387  epsilon:  0.07
episode:  2661  score:  235.12  avg_score:  227.05  frames:  973  epsilon:  0.07
episode:  2662  score:  276.56  avg_score:  227.28  frames:  266  epsilon:  0.07
episode:  2663  score:  267.42  avg_score:  226.93  frames:  247  epsilon:  0.07
episode:  2664  score:  260.88  avg_score:  229.28  frames:  275  epsilon:  0.07
episode:  2665  score:  245.52  avg_score:  229.28  frames:  247  epsilon:  0.07
episode:  2666  score:  241.07  avg_score:  231.82  frames:  204  epsilon:  0.07
episode:  2667  score:  229.61  avg_score:  231.61  frames:  227  epsilon:  0.07
episode:  2668  score:  261.15  avg_score:  231.56  frames:  175  epsilon:  0.07
episode:  2669  score:  269.08  avg_score:  231.43  frames:  191  epsilon:  0.07
episode:  2670  score:  238.95  avg_score:  231.33  frames:  378  epsilon:  0.07
episode:  2671  score:  213.55  avg_score:  230.77  frames:  213  epsilon:  0.07
episode:  2672  score:  240.27  avg_score:  230.56  frames:  191  epsilon:  0.07
episode:  2673  score:  285.92  avg_score:  231.04  frames:  201  epsilon:  0.07
episode:  2674  score:  204.68  avg_score:  230.55  frames:  235  epsilon:  0.07
episode:  2675  score:  258.05  avg_score:  230.40  frames:  336  epsilon:  0.07
episode:  2676  score:  196.74  avg_score:  229.69  frames:  216  epsilon:  0.07
episode:  2677  score:  197.86  avg_score:  228.96  frames:  275  epsilon:  0.07
episode:  2678  score:  275.12  avg_score:  229.10  frames:  236  epsilon:  0.07
episode:  2679  score:  266.05  avg_score:  231.60  frames:  648  epsilon:  0.07
episode:  2680  score:  258.29  avg_score:  231.76  frames:  228  epsilon:  0.07
episode:  2681  score:  243.64  avg_score:  231.73  frames:  258  epsilon:  0.07
episode:  2682  score:  231.59  avg_score:  233.71  frames:  193  epsilon:  0.07
episode:  2683  score:  226.97  avg_score:  233.52  frames:  270  epsilon:  0.07
episode:  2684  score:  305.77  avg_score:  233.75  frames:  297  epsilon:  0.07
episode:  2685  score:  232.78  avg_score:  233.30  frames:  355  epsilon:  0.07
episode:  2686  score:  224.82  avg_score:  233.18  frames:  424  epsilon:  0.07
episode:  2687  score:  295.16  avg_score:  233.30  frames:  375  epsilon:  0.07
episode:  2688  score:  232.99  avg_score:  233.14  frames:  267  epsilon:  0.07
episode:  2689  score:  229.90  avg_score:  232.82  frames:  305  epsilon:  0.07
episode:  2690  score:  317.18  avg_score:  233.11  frames:  252  epsilon:  0.07
episode:  2691  score:  287.29  avg_score:  233.36  frames:  277  epsilon:  0.07
episode:  2692  score:  239.25  avg_score:  232.93  frames:  390  epsilon:  0.07
episode:  2693  score:  107.18  avg_score:  231.41  frames:  999  epsilon:  0.07
episode:  2694  score:  240.83  avg_score:  231.85  frames:  383  epsilon:  0.07
episode:  2695  score:  304.75  avg_score:  231.98  frames:  253  epsilon:  0.07
episode:  2696  score:  220.29  avg_score:  231.29  frames:  194  epsilon:  0.07
episode:  2697  score:  260.64  avg_score:  231.04  frames:  227  epsilon:  0.07
episode:  2698  score:  264.93  avg_score:  231.58  frames:  963  epsilon:  0.07
episode:  2699  score:  257.81  avg_score:  231.38  frames:  237  epsilon:  0.07
episode:  2700  score:  229.57  avg_score:  230.75  frames:  193  epsilon:  0.07
episode:  2701  score:  299.40  avg_score:  231.05  frames:  264  epsilon:  0.07
episode:  2702  score:  274.27  avg_score:  232.28  frames:  210  epsilon:  0.07
episode:  2703  score:  225.48  avg_score:  232.45  frames:  283  epsilon:  0.07
episode:  2704  score:  274.58  avg_score:  233.21  frames:  219  epsilon:  0.07
episode:  2705  score:  260.49  avg_score:  233.33  frames:  196  epsilon:  0.07
episode:  2706  score:  255.57  avg_score:  233.48  frames:  219  epsilon:  0.07
episode:  2707  score:  300.68  avg_score:  234.04  frames:  213  epsilon:  0.07
episode:  2708  score:  260.42  avg_score:  233.89  frames:  195  epsilon:  0.07
episode:  2709  score:  259.21  avg_score:  234.20  frames:  217  epsilon:  0.07
episode:  2710  score:  283.91  avg_score:  234.93  frames:  285  epsilon:  0.07
episode:  2711  score:  243.44  avg_score:  234.38  frames:  712  epsilon:  0.07
episode:  2712  score:  286.86  avg_score:  234.40  frames:  240  epsilon:  0.07
episode:  2713  score:  285.19  avg_score:  234.85  frames:  246  epsilon:  0.07
episode:  2714  score:  230.47  avg_score:  234.40  frames:  221  epsilon:  0.07
episode:  2715  score:  261.51  avg_score:  234.56  frames:  166  epsilon:  0.07
episode:  2716  score:  254.56  avg_score:  234.61  frames:  208  epsilon:  0.07
episode:  2717  score:  259.69  avg_score:  234.63  frames:  245  epsilon:  0.07
episode:  2718  score:  235.36  avg_score:  234.45  frames:  281  epsilon:  0.07
episode:  2719  score:  203.93  avg_score:  233.97  frames:  585  epsilon:  0.07
episode:  2720  score:  270.79  avg_score:  236.98  frames:  289  epsilon:  0.07
episode:  2721  score:  251.38  avg_score:  237.06  frames:  242  epsilon:  0.07
episode:  2722  score:  274.09  avg_score:  237.26  frames:  302  epsilon:  0.07
episode:  2723  score:  273.96  avg_score:  239.79  frames:  388  epsilon:  0.07
episode:  2724  score:  267.77  avg_score:  239.95  frames:  314  epsilon:  0.07
episode:  2725  score:  242.57  avg_score:  239.73  frames:  228  epsilon:  0.07
episode:  2726  score:  279.23  avg_score:  240.31  frames:  738  epsilon:  0.07
episode:  2727  score:  300.92  avg_score:  244.20  frames:  353  epsilon:  0.07
episode:  2728  score:  263.91  avg_score:  244.05  frames:  177  epsilon:  0.07
episode:  2729  score:  237.53  avg_score:  243.98  frames:  212  epsilon:  0.07
episode:  2730  score:  236.28  avg_score:  244.03  frames:  283  epsilon:  0.07
episode:  2731  score:  283.83  avg_score:  244.25  frames:  237  epsilon:  0.07
episode:  2732  score:  -140.76  avg_score:  240.40  frames:  72  epsilon:  0.07
episode:  2733  score:  293.64  avg_score:  240.89  frames:  236  epsilon:  0.07
episode:  2734  score:  249.32  avg_score:  240.97  frames:  188  epsilon:  0.07
episode:  2735  score:  256.75  avg_score:  241.10  frames:  208  epsilon:  0.06
episode:  2736  score:  267.48  avg_score:  241.17  frames:  231  epsilon:  0.06
episode:  2737  score:  236.78  avg_score:  240.76  frames:  182  epsilon:  0.06
episode:  2738  score:  262.82  avg_score:  240.98  frames:  188  epsilon:  0.06
episode:  2739  score:  300.66  avg_score:  241.26  frames:  230  epsilon:  0.06
episode:  2740  score:  204.89  avg_score:  241.20  frames:  665  epsilon:  0.06
episode:  2741  score:  271.42  avg_score:  243.86  frames:  733  epsilon:  0.06
episode:  2742  score:  285.26  avg_score:  244.04  frames:  186  epsilon:  0.06
episode:  2743  score:  244.09  avg_score:  244.35  frames:  257  epsilon:  0.06
episode:  2744  score:  242.24  avg_score:  244.45  frames:  184  epsilon:  0.06
episode:  2745  score:  232.74  avg_score:  246.50  frames:  270  epsilon:  0.06
episode:  2746  score:  295.30  avg_score:  247.10  frames:  294  epsilon:  0.06
episode:  2747  score:  226.65  avg_score:  246.91  frames:  214  epsilon:  0.06
episode:  2748  score:  274.61  avg_score:  247.27  frames:  264  epsilon:  0.06
episode:  2749  score:  258.67  avg_score:  247.44  frames:  241  epsilon:  0.06
episode:  2750  score:  312.32  avg_score:  248.15  frames:  256  epsilon:  0.06
episode:  2751  score:  239.66  avg_score:  247.91  frames:  170  epsilon:  0.06
episode:  2752  score:  252.06  avg_score:  248.36  frames:  208  epsilon:  0.06
episode:  2753  score:  231.84  avg_score:  248.06  frames:  264  epsilon:  0.06
episode:  2754  score:  248.57  avg_score:  248.09  frames:  285  epsilon:  0.06
episode:  2755  score:  271.48  avg_score:  248.04  frames:  235  epsilon:  0.06
episode:  2756  score:  211.88  avg_score:  247.42  frames:  226  epsilon:  0.06
episode:  2757  score:  259.95  avg_score:  247.51  frames:  238  epsilon:  0.06
episode:  2758  score:  279.68  avg_score:  247.48  frames:  238  epsilon:  0.06
episode:  2759  score:  306.64  avg_score:  251.55  frames:  236  epsilon:  0.06
episode:  2760  score:  283.99  avg_score:  251.73  frames:  255  epsilon:  0.06
episode:  2761  score:  250.00  avg_score:  251.88  frames:  217  epsilon:  0.06
episode:  2762  score:  278.20  avg_score:  251.90  frames:  187  epsilon:  0.06
episode:  2763  score:  264.02  avg_score:  251.86  frames:  204  epsilon:  0.06
episode:  2764  score:  245.18  avg_score:  251.71  frames:  278  epsilon:  0.06
episode:  2765  score:  239.97  avg_score:  251.65  frames:  236  epsilon:  0.06
episode:  2766  score:  258.36  avg_score:  251.82  frames:  188  epsilon:  0.06
episode:  2767  score:  306.70  avg_score:  252.59  frames:  363  epsilon:  0.06
episode:  2768  score:  293.21  avg_score:  252.91  frames:  320  epsilon:  0.06
episode:  2769  score:  266.41  avg_score:  252.89  frames:  213  epsilon:  0.06
episode:  2770  score:  233.69  avg_score:  252.83  frames:  206  epsilon:  0.06
episode:  2771  score:  249.71  avg_score:  253.20  frames:  172  epsilon:  0.06
episode:  2772  score:  266.31  avg_score:  253.46  frames:  172  epsilon:  0.06
episode:  2773  score:  295.87  avg_score:  253.56  frames:  201  epsilon:  0.06
episode:  2774  score:  223.05  avg_score:  253.74  frames:  264  epsilon:  0.06
episode:  2775  score:  258.13  avg_score:  253.74  frames:  246  epsilon:  0.06
episode:  2776  score:  -96.50  avg_score:  250.81  frames:  57  epsilon:  0.06
episode:  2777  score:  269.26  avg_score:  251.52  frames:  272  epsilon:  0.06
episode:  2778  score:  294.64  avg_score:  251.72  frames:  319  epsilon:  0.06
episode:  2779  score:  245.99  avg_score:  251.52  frames:  261  epsilon:  0.06
episode:  2780  score:  244.17  avg_score:  251.38  frames:  211  epsilon:  0.06
episode:  2781  score:  226.91  avg_score:  251.21  frames:  469  epsilon:  0.06
episode:  2782  score:  268.16  avg_score:  251.57  frames:  176  epsilon:  0.06
episode:  2783  score:  275.64  avg_score:  252.06  frames:  269  epsilon:  0.06
episode:  2784  score:  239.48  avg_score:  251.40  frames:  209  epsilon:  0.06
episode:  2785  score:  286.70  avg_score:  251.94  frames:  169  epsilon:  0.06
episode:  2786  score:  302.74  avg_score:  252.72  frames:  242  epsilon:  0.06
episode:  2787  score:  -20.13  avg_score:  249.56  frames:  151  epsilon:  0.06
episode:  2788  score:  249.89  avg_score:  249.73  frames:  217  epsilon:  0.06
episode:  2789  score:  244.47  avg_score:  249.88  frames:  851  epsilon:  0.06
episode:  2790  score:  249.79  avg_score:  249.20  frames:  202  epsilon:  0.06
episode:  2791  score:  263.34  avg_score:  248.97  frames:  262  epsilon:  0.06
episode:  2792  score:  245.80  avg_score:  249.03  frames:  374  epsilon:  0.06
episode:  2793  score:  272.51  avg_score:  250.68  frames:  260  epsilon:  0.06
episode:  2794  score:  247.00  avg_score:  250.75  frames:  225  epsilon:  0.06
episode:  2795  score:  268.94  avg_score:  250.39  frames:  238  epsilon:  0.06
episode:  2796  score:  239.30  avg_score:  250.58  frames:  337  epsilon:  0.06
episode:  2797  score:  249.09  avg_score:  250.46  frames:  335  epsilon:  0.06
episode:  2798  score:  254.01  avg_score:  250.35  frames:  216  epsilon:  0.06
episode:  2799  score:  230.81  avg_score:  250.08  frames:  236  epsilon:  0.06
episode:  2800  score:  281.46  avg_score:  250.60  frames:  332  epsilon:  0.06
episode:  2801  score:  262.51  avg_score:  250.23  frames:  224  epsilon:  0.06
episode:  2802  score:  233.56  avg_score:  249.83  frames:  173  epsilon:  0.06
episode:  2803  score:  236.13  avg_score:  249.93  frames:  162  epsilon:  0.06
episode:  2804  score:  311.45  avg_score:  250.30  frames:  212  epsilon:  0.06
episode:  2805  score:  253.61  avg_score:  250.23  frames:  162  epsilon:  0.06
episode:  2806  score:  254.43  avg_score:  250.22  frames:  208  epsilon:  0.06
episode:  2807  score:  276.48  avg_score:  249.98  frames:  260  epsilon:  0.06
episode:  2808  score:  258.47  avg_score:  249.96  frames:  212  epsilon:  0.06
episode:  2809  score:  238.26  avg_score:  249.75  frames:  193  epsilon:  0.06
episode:  2810  score:  297.24  avg_score:  249.88  frames:  262  epsilon:  0.06
episode:  2811  score:  3.00  avg_score:  247.48  frames:  138  epsilon:  0.06
episode:  2812  score:  269.24  avg_score:  247.30  frames:  461  epsilon:  0.06
episode:  2813  score:  233.85  avg_score:  246.79  frames:  215  epsilon:  0.06
episode:  2814  score:  268.54  avg_score:  247.17  frames:  310  epsilon:  0.06
episode:  2815  score:  228.33  avg_score:  246.84  frames:  308  epsilon:  0.06
episode:  2816  score:  249.26  avg_score:  246.79  frames:  190  epsilon:  0.06
episode:  2817  score:  290.89  avg_score:  247.10  frames:  206  epsilon:  0.06
episode:  2818  score:  119.93  avg_score:  245.94  frames:  999  epsilon:  0.06
episode:  2819  score:  287.07  avg_score:  246.77  frames:  252  epsilon:  0.06
episode:  2820  score:  277.40  avg_score:  246.84  frames:  173  epsilon:  0.06
episode:  2821  score:  272.54  avg_score:  247.05  frames:  230  epsilon:  0.06
episode:  2822  score:  -102.11  avg_score:  243.29  frames:  395  epsilon:  0.06
episode:  2823  score:  76.71  avg_score:  241.32  frames:  999  epsilon:  0.06
episode:  2824  score:  254.19  avg_score:  241.18  frames:  269  epsilon:  0.06
episode:  2825  score:  234.07  avg_score:  241.10  frames:  254  epsilon:  0.06
episode:  2826  score:  250.12  avg_score:  240.81  frames:  214  epsilon:  0.06
episode:  2827  score:  304.26  avg_score:  240.84  frames:  195  epsilon:  0.06
episode:  2828  score:  284.48  avg_score:  241.04  frames:  281  epsilon:  0.06
episode:  2829  score:  281.78  avg_score:  241.49  frames:  240  epsilon:  0.06
episode:  2830  score:  238.74  avg_score:  241.51  frames:  215  epsilon:  0.06
episode:  2831  score:  22.21  avg_score:  238.90  frames:  91  epsilon:  0.06
episode:  2832  score:  32.35  avg_score:  240.63  frames:  118  epsilon:  0.06
episode:  2833  score:  218.29  avg_score:  239.87  frames:  207  epsilon:  0.06
episode:  2834  score:  -112.39  avg_score:  236.26  frames:  51  epsilon:  0.06
episode:  2835  score:  173.45  avg_score:  235.42  frames:  510  epsilon:  0.06
episode:  2836  score:  26.19  avg_score:  233.01  frames:  116  epsilon:  0.06
episode:  2837  score:  290.30  avg_score:  233.55  frames:  253  epsilon:  0.06
episode:  2838  score:  282.40  avg_score:  233.74  frames:  230  epsilon:  0.06
episode:  2839  score:  260.02  avg_score:  233.33  frames:  334  epsilon:  0.06
episode:  2840  score:  270.13  avg_score:  233.99  frames:  236  epsilon:  0.06
episode:  2841  score:  255.46  avg_score:  233.83  frames:  214  epsilon:  0.06
episode:  2842  score:  234.82  avg_score:  233.32  frames:  255  epsilon:  0.06
episode:  2843  score:  269.70  avg_score:  233.58  frames:  263  epsilon:  0.06
episode:  2844  score:  238.23  avg_score:  233.54  frames:  194  epsilon:  0.06
episode:  2845  score:  259.67  avg_score:  233.81  frames:  178  epsilon:  0.06
episode:  2846  score:  290.36  avg_score:  233.76  frames:  218  epsilon:  0.06
episode:  2847  score:  4.48  avg_score:  231.54  frames:  147  epsilon:  0.06
episode:  2848  score:  217.26  avg_score:  230.96  frames:  632  epsilon:  0.06
episode:  2849  score:  287.14  avg_score:  231.25  frames:  268  epsilon:  0.06
episode:  2850  score:  212.54  avg_score:  230.25  frames:  267  epsilon:  0.06
episode:  2851  score:  258.93  avg_score:  230.44  frames:  215  epsilon:  0.06
episode:  2852  score:  192.99  avg_score:  229.85  frames:  538  epsilon:  0.06
episode:  2853  score:  244.48  avg_score:  229.98  frames:  215  epsilon:  0.06
episode:  2854  score:  298.68  avg_score:  230.48  frames:  220  epsilon:  0.06
episode:  2855  score:  223.10  avg_score:  230.00  frames:  203  epsilon:  0.06
episode:  2856  score:  300.47  avg_score:  230.88  frames:  248  epsilon:  0.06
episode:  2857  score:  283.75  avg_score:  231.12  frames:  269  epsilon:  0.06
episode:  2858  score:  253.10  avg_score:  230.85  frames:  269  epsilon:  0.06
episode:  2859  score:  233.93  avg_score:  230.13  frames:  424  epsilon:  0.06
episode:  2860  score:  269.88  avg_score:  229.99  frames:  193  epsilon:  0.06
episode:  2861  score:  261.30  avg_score:  230.10  frames:  190  epsilon:  0.06
episode:  2862  score:  226.10  avg_score:  229.58  frames:  240  epsilon:  0.06
episode:  2863  score:  287.70  avg_score:  229.82  frames:  208  epsilon:  0.06
episode:  2864  score:  282.52  avg_score:  230.19  frames:  223  epsilon:  0.06
episode:  2865  score:  263.62  avg_score:  230.43  frames:  295  epsilon:  0.06
episode:  2866  score:  272.44  avg_score:  230.57  frames:  232  epsilon:  0.06
episode:  2867  score:  287.18  avg_score:  230.37  frames:  221  epsilon:  0.06
episode:  2868  score:  291.02  avg_score:  230.35  frames:  359  epsilon:  0.06
episode:  2869  score:  269.22  avg_score:  230.38  frames:  221  epsilon:  0.06
episode:  2870  score:  258.10  avg_score:  230.62  frames:  253  epsilon:  0.06
episode:  2871  score:  265.74  avg_score:  230.78  frames:  210  epsilon:  0.06
episode:  2872  score:  272.02  avg_score:  230.84  frames:  257  epsilon:  0.06
episode:  2873  score:  260.92  avg_score:  230.49  frames:  275  epsilon:  0.06
episode:  2874  score:  293.73  avg_score:  231.20  frames:  217  epsilon:  0.06
episode:  2875  score:  254.38  avg_score:  231.16  frames:  199  epsilon:  0.06
episode:  2876  score:  293.88  avg_score:  235.06  frames:  234  epsilon:  0.06
episode:  2877  score:  292.27  avg_score:  235.29  frames:  259  epsilon:  0.06
episode:  2878  score:  267.08  avg_score:  235.02  frames:  205  epsilon:  0.06
episode:  2879  score:  284.89  avg_score:  235.41  frames:  291  epsilon:  0.06
episode:  2880  score:  279.96  avg_score:  235.76  frames:  238  epsilon:  0.06
episode:  2881  score:  279.15  avg_score:  236.29  frames:  352  epsilon:  0.06
episode:  2882  score:  284.44  avg_score:  236.45  frames:  246  epsilon:  0.06
episode:  2883  score:  273.74  avg_score:  236.43  frames:  232  epsilon:  0.06
episode:  2884  score:  248.80  avg_score:  236.52  frames:  195  epsilon:  0.06
episode:  2885  score:  222.25  avg_score:  235.88  frames:  189  epsilon:  0.06
episode:  2886  score:  281.06  avg_score:  235.66  frames:  370  epsilon:  0.06
episode:  2887  score:  288.78  avg_score:  238.75  frames:  226  epsilon:  0.06
episode:  2888  score:  244.91  avg_score:  238.70  frames:  188  epsilon:  0.06
episode:  2889  score:  281.17  avg_score:  239.07  frames:  223  epsilon:  0.06
episode:  2890  score:  285.28  avg_score:  239.42  frames:  241  epsilon:  0.06
episode:  2891  score:  262.00  avg_score:  239.41  frames:  309  epsilon:  0.06
episode:  2892  score:  -8.79  avg_score:  236.86  frames:  108  epsilon:  0.06
episode:  2893  score:  244.21  avg_score:  236.58  frames:  240  epsilon:  0.06
episode:  2894  score:  255.35  avg_score:  236.66  frames:  313  epsilon:  0.06
episode:  2895  score:  290.38  avg_score:  236.88  frames:  270  epsilon:  0.06
episode:  2896  score:  273.21  avg_score:  237.22  frames:  212  epsilon:  0.06
episode:  2897  score:  46.17  avg_score:  235.19  frames:  106  epsilon:  0.06
episode:  2898  score:  282.80  avg_score:  235.48  frames:  233  epsilon:  0.06
episode:  2899  score:  240.93  avg_score:  235.58  frames:  230  epsilon:  0.06
episode:  2900  score:  236.40  avg_score:  235.13  frames:  151  epsilon:  0.06
episode:  2901  score:  247.38  avg_score:  234.98  frames:  203  epsilon:  0.06
episode:  2902  score:  240.36  avg_score:  235.04  frames:  330  epsilon:  0.06
episode:  2903  score:  233.19  avg_score:  235.01  frames:  197  epsilon:  0.05
episode:  2904  score:  303.72  avg_score:  234.94  frames:  239  epsilon:  0.05
episode:  2905  score:  219.19  avg_score:  234.59  frames:  178  epsilon:  0.05
episode:  2906  score:  291.45  avg_score:  234.96  frames:  295  epsilon:  0.05
episode:  2907  score:  295.80  avg_score:  235.16  frames:  243  epsilon:  0.05
episode:  2908  score:  308.00  avg_score:  235.65  frames:  192  epsilon:  0.05
episode:  2909  score:  227.99  avg_score:  235.55  frames:  220  epsilon:  0.05
episode:  2910  score:  275.40  avg_score:  235.33  frames:  188  epsilon:  0.05
episode:  2911  score:  296.71  avg_score:  238.27  frames:  193  epsilon:  0.05
episode:  2912  score:  270.58  avg_score:  238.28  frames:  285  epsilon:  0.05
episode:  2913  score:  235.92  avg_score:  238.30  frames:  289  epsilon:  0.05
episode:  2914  score:  306.19  avg_score:  238.68  frames:  254  epsilon:  0.05
episode:  2915  score:  247.16  avg_score:  238.87  frames:  198  epsilon:  0.05
episode:  2916  score:  272.53  avg_score:  239.10  frames:  211  epsilon:  0.05
episode:  2917  score:  262.61  avg_score:  238.82  frames:  252  epsilon:  0.05
episode:  2918  score:  8.77  avg_score:  237.70  frames:  107  epsilon:  0.05
episode:  2919  score:  276.68  avg_score:  237.60  frames:  407  epsilon:  0.05
episode:  2920  score:  285.38  avg_score:  237.68  frames:  277  epsilon:  0.05
episode:  2921  score:  294.68  avg_score:  237.90  frames:  205  epsilon:  0.05
episode:  2922  score:  266.44  avg_score:  241.59  frames:  285  epsilon:  0.05
episode:  2923  score:  283.68  avg_score:  243.66  frames:  283  epsilon:  0.05
episode:  2924  score:  280.66  avg_score:  243.92  frames:  355  epsilon:  0.05
episode:  2925  score:  252.41  avg_score:  244.11  frames:  216  epsilon:  0.05
episode:  2926  score:  283.75  avg_score:  244.44  frames:  292  epsilon:  0.05
episode:  2927  score:  47.94  avg_score:  241.88  frames:  186  epsilon:  0.05
episode:  2928  score:  27.93  avg_score:  239.31  frames:  132  epsilon:  0.05
episode:  2929  score:  213.24  avg_score:  238.63  frames:  215  epsilon:  0.05
episode:  2930  score:  291.58  avg_score:  239.16  frames:  190  epsilon:  0.05
episode:  2931  score:  284.14  avg_score:  241.78  frames:  271  epsilon:  0.05
episode:  2932  score:  276.12  avg_score:  244.21  frames:  183  epsilon:  0.05
episode:  2933  score:  257.51  avg_score:  244.60  frames:  206  epsilon:  0.05
episode:  2934  score:  237.17  avg_score:  248.10  frames:  185  epsilon:  0.05
episode:  2935  score:  -104.83  avg_score:  245.32  frames:  81  epsilon:  0.05
episode:  2936  score:  299.15  avg_score:  248.05  frames:  242  epsilon:  0.05
episode:  2937  score:  290.00  avg_score:  248.04  frames:  203  epsilon:  0.05
episode:  2938  score:  303.85  avg_score:  248.26  frames:  226  epsilon:  0.05
episode:  2939  score:  255.26  avg_score:  248.21  frames:  204  epsilon:  0.05
episode:  2940  score:  290.37  avg_score:  248.41  frames:  242  epsilon:  0.05
episode:  2941  score:  250.84  avg_score:  248.37  frames:  279  epsilon:  0.05
episode:  2942  score:  37.94  avg_score:  246.40  frames:  169  epsilon:  0.05
episode:  2943  score:  -39.79  avg_score:  243.30  frames:  174  epsilon:  0.05
episode:  2944  score:  290.09  avg_score:  243.82  frames:  323  epsilon:  0.05
episode:  2945  score:  278.65  avg_score:  244.01  frames:  228  epsilon:  0.05
episode:  2946  score:  260.62  avg_score:  243.71  frames:  224  epsilon:  0.05
episode:  2947  score:  253.10  avg_score:  246.20  frames:  191  epsilon:  0.05
episode:  2948  score:  293.53  avg_score:  246.96  frames:  216  epsilon:  0.05
episode:  2949  score:  40.32  avg_score:  244.50  frames:  96  epsilon:  0.05
episode:  2950  score:  181.13  avg_score:  244.18  frames:  607  epsilon:  0.05
episode:  2951  score:  196.50  avg_score:  243.56  frames:  364  epsilon:  0.05
episode:  2952  score:  273.44  avg_score:  244.36  frames:  267  epsilon:  0.05
episode:  2953  score:  269.29  avg_score:  244.61  frames:  246  epsilon:  0.05
episode:  2954  score:  269.50  avg_score:  244.32  frames:  294  epsilon:  0.05
episode:  2955  score:  275.52  avg_score:  244.84  frames:  295  epsilon:  0.05
episode:  2956  score:  277.63  avg_score:  244.61  frames:  286  epsilon:  0.05
episode:  2957  score:  218.51  avg_score:  243.96  frames:  350  epsilon:  0.05
episode:  2958  score:  270.35  avg_score:  244.13  frames:  323  epsilon:  0.05
episode:  2959  score:  -0.64  avg_score:  241.79  frames:  113  epsilon:  0.05
episode:  2960  score:  263.67  avg_score:  241.73  frames:  256  epsilon:  0.05
episode:  2961  score:  236.49  avg_score:  241.48  frames:  240  epsilon:  0.05
episode:  2962  score:  258.25  avg_score:  241.80  frames:  193  epsilon:  0.05
episode:  2963  score:  246.65  avg_score:  241.39  frames:  209  epsilon:  0.05
episode:  2964  score:  265.34  avg_score:  241.22  frames:  257  epsilon:  0.05
episode:  2965  score:  280.58  avg_score:  241.39  frames:  247  epsilon:  0.05
episode:  2966  score:  239.44  avg_score:  241.06  frames:  253  epsilon:  0.05
episode:  2967  score:  -289.42  avg_score:  235.29  frames:  78  epsilon:  0.05
episode:  2968  score:  -184.51  avg_score:  230.54  frames:  103  epsilon:  0.05
episode:  2969  score:  260.74  avg_score:  230.45  frames:  215  epsilon:  0.05
episode:  2970  score:  21.47  avg_score:  228.08  frames:  142  epsilon:  0.05
episode:  2971  score:  -25.52  avg_score:  225.17  frames:  152  epsilon:  0.05
episode:  2972  score:  247.80  avg_score:  224.93  frames:  216  epsilon:  0.05
episode:  2973  score:  4.88  avg_score:  222.37  frames:  133  epsilon:  0.05
episode:  2974  score:  212.30  avg_score:  221.55  frames:  343  epsilon:  0.05
episode:  2975  score:  134.53  avg_score:  220.36  frames:  331  epsilon:  0.05
episode:  2976  score:  43.85  avg_score:  217.86  frames:  138  epsilon:  0.05
episode:  2977  score:  -11.71  avg_score:  214.82  frames:  152  epsilon:  0.05
episode:  2978  score:  -4.46  avg_score:  212.10  frames:  112  epsilon:  0.05
episode:  2979  score:  257.19  avg_score:  211.82  frames:  257  epsilon:  0.05
episode:  2980  score:  241.48  avg_score:  211.44  frames:  262  epsilon:  0.05
episode:  2981  score:  306.13  avg_score:  211.71  frames:  233  epsilon:  0.05
episode:  2982  score:  63.82  avg_score:  209.50  frames:  183  epsilon:  0.05
episode:  2983  score:  -186.61  avg_score:  204.90  frames:  56  epsilon:  0.05
episode:  2984  score:  242.10  avg_score:  204.83  frames:  306  epsilon:  0.05
episode:  2985  score:  311.77  avg_score:  205.73  frames:  261  epsilon:  0.05
episode:  2986  score:  261.77  avg_score:  205.53  frames:  253  epsilon:  0.05
episode:  2987  score:  20.91  avg_score:  202.86  frames:  172  epsilon:  0.05
episode:  2988  score:  46.30  avg_score:  200.87  frames:  189  epsilon:  0.05
episode:  2989  score:  160.61  avg_score:  199.66  frames:  264  epsilon:  0.05
episode:  2990  score:  31.86  avg_score:  197.13  frames:  218  epsilon:  0.05
episode:  2991  score:  50.63  avg_score:  195.02  frames:  212  epsilon:  0.05
episode:  2992  score:  -69.43  avg_score:  194.41  frames:  197  epsilon:  0.05
episode:  2993  score:  201.33  avg_score:  193.98  frames:  383  epsilon:  0.05
episode:  2994  score:  78.88  avg_score:  192.22  frames:  164  epsilon:  0.05
episode:  2995  score:  -161.34  avg_score:  187.70  frames:  429  epsilon:  0.05
episode:  2996  score:  209.90  avg_score:  187.07  frames:  461  epsilon:  0.05
episode:  2997  score:  244.94  avg_score:  189.05  frames:  326  epsilon:  0.05
episode:  2998  score:  168.49  avg_score:  187.91  frames:  409  epsilon:  0.05
episode:  2999  score:  231.12  avg_score:  187.81  frames:  508  epsilon:  0.05
episode:  3000  score:  254.14  avg_score:  187.99  frames:  354  epsilon:  0.05
episode:  3001  score:  245.12  avg_score:  187.97  frames:  405  epsilon:  0.05
episode:  3002  score:  192.87  avg_score:  187.49  frames:  397  epsilon:  0.05
episode:  3003  score:  210.16  avg_score:  187.26  frames:  430  epsilon:  0.05
episode:  3004  score:  281.22  avg_score:  187.04  frames:  346  epsilon:  0.05
episode:  3005  score:  284.88  avg_score:  187.69  frames:  396  epsilon:  0.05
episode:  3006  score:  246.26  avg_score:  187.24  frames:  433  epsilon:  0.05
episode:  3007  score:  139.54  avg_score:  185.68  frames:  448  epsilon:  0.05
episode:  3008  score:  76.77  avg_score:  183.37  frames:  653  epsilon:  0.05
episode:  3009  score:  272.67  avg_score:  183.81  frames:  412  epsilon:  0.05
episode:  3010  score:  247.98  avg_score:  183.54  frames:  428  epsilon:  0.05
episode:  3011  score:  125.77  avg_score:  181.83  frames:  650  epsilon:  0.05
episode:  3012  score:  190.97  avg_score:  181.03  frames:  556  epsilon:  0.05
episode:  3013  score:  231.55  avg_score:  180.99  frames:  478  epsilon:  0.05
episode:  3014  score:  183.46  avg_score:  179.76  frames:  459  epsilon:  0.05
episode:  3015  score:  -7.48  avg_score:  177.22  frames:  999  epsilon:  0.05
episode:  3016  score:  278.01  avg_score:  177.27  frames:  368  epsilon:  0.05
episode:  3017  score:  282.42  avg_score:  177.47  frames:  337  epsilon:  0.05
episode:  3018  score:  244.91  avg_score:  179.83  frames:  531  epsilon:  0.05
episode:  3019  score:  177.19  avg_score:  178.84  frames:  623  epsilon:  0.05
episode:  3020  score:  212.83  avg_score:  178.11  frames:  473  epsilon:  0.05
"""